# 3D Droplet Oscillation (Part 2, Postprocessing)

Results published: hopefully at some point!

This is part of the BoSSS-long-term validation test suite, which consists of several computationally expensive test-cases (runtime in the order of days), which are performed on a regular basis in order to validate the physical correctness of BoSSS simulations.

### Preliminaries

This example can be found in the source code repository as as `Droplet3D-Postprocessing.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [ ]:
//#r "../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
//#r "../../src/L4-application/BoSSSpad/bin/Debug/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Initialization tasks

Loading the `XNSE_Solver` and additional namespace:

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases;
using BoSSS.Solution.NSECommon;
using BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater;
using NUnit.Framework;
using BoSSS.Application.XNSE_Solver.Logging;

Initialization of the Workflow management; there `OscillatingDroplet3D` is the project name which is used name all computations (aka. sessions):

In [ ]:
BoSSSshell.WorkflowMgm.Init("OscillatingDroplet3D");

Project name is set to 'OscillatingDroplet3D'.
Opening existing database 'D:\local\OscillatingDroplet3D'.


In [ ]:
//wmg.Sessions

In [ ]:
int numSess = wmg.Sessions.Count();
numSess

43

In [ ]:
//add database manually:
//OpenOrCreateDatabase(@"\\dc1\userspace\smuda\Databases\OscillatingDroplet3D");
OpenOrCreateDatabase(@"\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D");
//OpenOrCreateDatabase(@"\\130.83.248.207\ValidationTests\OscillatingDroplet3D");

Opening existing database '\\hpccluster\hpccluster-scratch\smuda\OscillatingDroplet3D'.


## Observing sessions

In [ ]:
wmg.Sessions.Skip(numSess)

#0: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit_restart1*	05/17/2022 10:19:07	3704109c...
#1: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr1_m4_Oh056_eta005_thirdOrderInit_restart1*	05/17/2022 10:08:04	76b0e980...
#2: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_thirdOrderInit_restart1*	05/17/2022 10:08:00	3e2bed94...
#3: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_thirdOrderInit_restart1*	05/17/2022 10:07:57	67803604...
#4: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_thirdOrderInit_restart1*	05/17/2022 10:07:54	5186e4b6...
#5: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_restart1*	05/17/2022 10:07:52	8e7bc526...
#6: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_restart1*	05/17/2022 10:07:48	2a51c1d4...
#7: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_restart1*	05/17/2022 10:07:45	5c10c5c8...
#8: OscillatingDroplet3D	OD3D_J432k3_wallBC_amr1_m4_Oh056_eta005_restart1*	05/17/20

In [ ]:
//wmg.Sessions.Skip(numSess).Pick(0).Delete(true);

Session 3704109c-decc-42af-8031-49a960c60336 deleted.


In [ ]:
//var targetDB = databases.Pick(0);
//wmg.Sessions.Skip(numSess).Take(14).CopyAll(targetDB); //Move(targetDB); //MoveAll(targetDB);
//var delSess = wmg.Sessions.Skip(42).Where(s => s.Name.Contains("_Oh01"));
//wmg.Sessions[0].Delete(true)
//wmg.Sessions[8].Timesteps.Count()
//delSess

In [ ]:
//delSess.ElementAt(0).Delete(true);
// foreach(var dS in delSess)
//     dS.Delete(true);

In [ ]:
int sNum = 44;
wmg.Sessions[sNum]

OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04*	04/08/2022 11:34:56	8486c026...

In [ ]:
wmg.Sessions[sNum].GetSessionDirectory()

\\dc1\userspace\smuda\Databases\OscillatingDroplet3D\sessions\8ef825a4-ae9c-485e-99d5-8769c65afa5d

In [ ]:
wmg.Sessions[sNum].DeployPath

\\fdygitrunner\ValidationTests\OscillatingDroplet3D-XNSE_Solver2022Apr08_113220

In [ ]:
wmg.Sessions[sNum].Timesteps.Export().WithSupersampling(2).Do();

Starting export process... Data will be written to the directory: C:\Users\smuda\AppData\Local\BoSSS\plots\sessions\OscillatingDroplet3D__OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04__8486c026-23b3-4414-8c35-a04d33089732


In [ ]:
using System.IO;

In [8]:
/*
// temporary fix for a bug in the output file "SphericalHarmonics.txt"
// (missing separator/tab between fist ans second column)
void FileSanitizer(string _TextFile) {
Console.WriteLine("sanitizing: " + _TextFile);
var TextFile2 = Path.Combine(Path.GetDirectoryName(_TextFile), Path.GetFileNameWithoutExtension(_TextFile) + "-Copy.txt");
File.Copy(_TextFile,TextFile2, true);
var TextFile3 = Path.Combine(Path.GetDirectoryName(_TextFile), Path.GetFileNameWithoutExtension(_TextFile) + "-Sanitized.txt");
using (StreamWriter wrt = new StreamWriter(TextFile3)) {
using (StreamReader reader = new StreamReader(new FileStream(TextFile2, FileMode.Open, FileAccess.Read, FileShare.Read))) {
  int cnt = 0;
  for(string l = reader.ReadLine(); l != null; l = reader.ReadLine()) {
     if(cnt >= 1) {
       string[] parts = l.Split("\t");
       //Console.WriteLine(parts[0] + "    " + parts.Length);
       string P0 = parts[0];
       int L = P0.Length;
       
       string pp1, pp2;
       int exp = P0.IndexOf("E", 0, 5);
       //exp = -1;
       if(exp >= 0) {
          int ppl = P0[exp + 1] == '-' ? 4 : 3;
          pp1 = P0.Substring(0,exp + ppl);
          pp2 = P0.Substring(exp + ppl);
          
       } else {
          int minus = P0.IndexOf("-");
          if(minus >= 0 && P0[minus - 1] == 'E')
             minus = -1;
          if(minus >= 0) {
              pp1 = P0.Substring(0,minus);
              pp2 = P0.Substring(minus);
          } else {
              int comma = P0.IndexOf('.', 2, L - 2);
              pp1 = P0.Substring(0,comma - 1);
              pp2 = P0.Substring(comma - 1);
          }
       }
       try {
          double.Parse(pp1);
       } catch(Exception) {
          Console.Error.WriteLine("line " + (cnt + 1) + " cannot parse 1: " + pp1 + "  from " + P0);
       }
       try {
            double.Parse(pp2);
       } catch(Exception) {
          Console.Error.WriteLine("line " + (cnt + 1) + " cannot parse 2: " + pp2 + "  from " + P0);
       }
       wrt.Write(pp1 + "\t" + pp2);

       
       for(int i = 1; i < parts.Length; i++) {
            wrt.Write("\t");
            wrt.Write(parts[i]);
       }
       wrt.WriteLine();
       
     } else {
       wrt.WriteLine(l);
     }
     cnt++;
  }
}
}

}
*/

In [9]:
/*
foreach(var s in wmg.Sessions) {
    string dir = DatabaseDriver.GetSessionDirectory(s);
    string file = Path.Combine(dir, "SphericalHarmonics.txt");
    FileSanitizer(file);
}
*/

In [10]:
// might take some time
/*
foreach(var s in wmg.Sessions) {
    Console.Write(s.Name + ": ");
    if(s.Timesteps.Count() > 0)
        Console.WriteLine(s.Timesteps.Last());
    else 
        Console.WriteLine("no timestep computed/saved");
    Console.WriteLine();
}
*/

In [11]:
//wmg.Sessions.Single(si => si.Name.Contains("case3")).Export().WithSupersampling(2).Do()

# Study Overview

In [ ]:
var studySess = wmg.Sessions.Skip(numSess).Take(16);
studySess

#0: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005*	04/08/2022 11:37:03	92930af7...
#1: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01*	04/08/2022 11:36:37	15341e01...
#2: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04*	04/08/2022 11:34:56	8486c026...
#3: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04*	04/08/2022 11:34:00	e3a04ed7...
#4: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01*	04/08/2022 11:33:27	59bb73dc...
#5: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02*	04/08/2022 11:33:07	2e275baf...
#6: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04*	04/08/2022 11:32:42	8ef825a4...
#7: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_thirdOrderInit*	04/08/2022 11:32:17	adf2022d...
#8: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_thirdOrderInit*	04/08/2022 11:31:55	106a329f...
#9: OscillatingDroplet3D	OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_thirdOrderInit*	04/08/2022 11:31:22	5f8

In [ ]:
string[] caseS = new string[] { "m2_Oh01_eta04", "m2_Oh01_eta02", "m2_Oh01_eta01", "m3_Oh01_eta04", "m4_Oh01_eta04", "m4_Oh01_eta01", "m4_Oh056_eta005"};
string[] gridS = new string[] { "J686", "J432" };
bool[] ShouldAnaInit = new bool[] { true, false };
bool[] useNewton = new bool[] { false };

# Plotting  droplet metrics - aspect ratio L/W

In [ ]:
//string[] metricKeys = new string[] {"theta0", "theta90x", "theta90y", "volume"};

In [ ]:
Plot2Ddata[,] PlotTableDM = new Plot2Ddata[caseS.Length,1];
for(int iCol = 0; iCol < 1; iCol++) {
for(int iRow = 0; iRow < caseS.Length; iRow++) { // loop over cases
for(int iGrd = 0; iGrd < gridS.Length; iGrd++) {   
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
for(int iNew = 0; iNew < useNewton.Length; iNew++) {

    string _case = caseS[iRow];
    string _grd = gridS[iGrd];

    // if(_case == "case2" || _case == "case3")
    //     continue; 

    //ISessionInfo SI = null;
    var SIs = studySess.Where(sess => sess.Name.Contains(_case) 
                                      && sess.Name.Contains(_grd)
                                      && (sess.Name.Contains("_thirdOrderInit") == ShouldAnaInit[iAna])
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));


    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    if(SIs.IsNullOrEmpty()) 
        continue;

    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    IDictionary<string, IList<double>> tab = null;
    try {
        tab = SI.ReadTabulatedTextFileAsDoubles("DropletMetrics.txt", '\t');
    } catch (Exception e) {
        Console.Error.WriteLine(e.Message);
        Console.WriteLine("Skipping " + SI);
        
        // var dummyPlot = new Plot2Ddata();
        // dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
        // PlotTableDM[iRow,iCol] = dummyPlot;
        continue;
    }
    
    var plot = new Plot2Ddata();
    var time = tab["time"];

    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.LineWidth = 3;
    if (useNewton[iNew]) {
        fmt.LineColor = LineColors.Red;
        //fmt.DashType = DashTypes.Dashed;
    } else {  
        fmt.LineColor = LineColors.Blue;
        //fmt.DashType = DashTypes.Solid;
    }
    if (!ShouldAnaInit[iAna]) {
        fmt.DashType = DashTypes.Solid;
    } else {
        fmt.DashType = DashTypes.Dashed;
    }
    
    // if (_init == "_initCase0")
    //     fmt.LineColor = LineColors.Green;
    // if (_init == "_initCase1")
    //     fmt.LineColor = LineColors.Magenta;
    // if (_init == "_initCase2")
    //     fmt.LineColor = LineColors.Orange;
 
    string name = "BoSSS-"+_grd;
    // if(iAMR > 0)
    //     name = name + "-amr";
    if(ShouldAnaInit[iAna])
        name = name + "-3OrdInit"; 
    if(useNewton[iNew])
        name = name + "-Newton";

    double[] L = tab["L"].ToArray(); 
    double[] Wx = tab["Wx"].ToArray(); 
    double[] Wy = tab["Wy"].ToArray();
    double[] arLW = new double[L.Length];
    for (int i = 0; i < L.Length; i++) {
        arLW[i] = 0.5 * ((L[i] / (2.0*Wx[i])) + (L[i] / (2.0*Wy[i])));
    }
    
    List<double> arLW_lst = new List<double>();
    List<double> time_lst = new List<double>();
    double dataThreshold = 0.01;
    for(int i = 0; i < L.Length; i++) {
        if(arLW[i] > dataThreshold) {
            arLW_lst.Add(arLW[i]);
            time_lst.Add(time[i]);
        }
    }
    arLW = arLW_lst.ToArray();
    time = time_lst.ToArray();

    plot.AddDataGroup(name, time, arLW, fmt);
   
    plot.ShowLegend = true;  

    if(PlotTableDM[iRow,iCol]==null) {
        PlotTableDM[iRow,iCol] = plot;
    } else {
        PlotTableDM[iRow,iCol] = PlotTableDM[iRow,iCol].Merge(plot);
    }
        
    // if(iAMR == 0) {
    //     PlotTableDM[iRow,iCol] = plot;
    // } else {
    //     PlotTableDM[iRow,iCol] = PlotTableDM[iRow,iCol].Merge(plot);
    // }
    
}

if(PlotTableDM[iRow,iCol] != null) {
    PlotTableDM[iRow,iCol].ShowLegend = true; 
    //PlotTableDM[iRow,iCol].LegendFont = 10;
    if(iCol == 0) {
       PlotTableDM[iRow,iCol].Title = caseS[iRow];
       PlotTableDM[iRow,iCol].Xlabel = "time t";
       PlotTableDM[iRow,iCol].Ylabel = "aspect ratio L/W";
    }
}
}
}
}
}

In [ ]:
PlotTableDM

index dataGroups LogX LogY LogBaseX LogBaseY LogX2 LogY2 XrangeMin XrangeMax YrangeMin YrangeMax Y2rangeMin Y2rangeMax X2rangeMin X2rangeMax Xlabel X2label Ylabel Y2label Title .. 0 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m2_Oh01_eta04 1 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m2_Oh01_eta02 2 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 2 <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m2_Oh01_eta01 3 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 3 <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m3_Oh01_eta04 4 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 3 <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m4_Oh01_eta04 5 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 3 <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m4_Oh01_eta01 6 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 3 <null> <null> <null> <null> <null> <null> time t <null> aspect ratio L/W <null> m4_Oh056_eta005

### Period durations

In [ ]:
Plot2Ddata[,] PlotTablePD = new Plot2Ddata[caseS.Length, 1];
for(int iRow = 3; iRow < 4; iRow++) { // loop over cases
    Plot2Ddata plot = PlotTableDM[iRow,0];
    double[] time = plot.dataGroups[0].Abscissas;
    double[] LW = plot.dataGroups[0].Values;

    List<double> times_max = new List<double>();    // times at maxima
    List<double> times_min = new List<double>();    // times at minima
    List<double> times_one = new List<double>();    // times at value 1 crossings
    double value_max = LW[0];
    double value_min = double.MaxValue;
    double time_maxmin = 0;
    bool search_max = true;     // if false the local minima is searched for
    for(int i = 1; i < time.Length; i++) {
        double currVal = LW[i];
        if(search_max && currVal > value_max) {
            value_max = currVal;
            time_maxmin = time[i];
        } else if(!search_max && currVal < value_min) {
            value_min = currVal;
            time_maxmin = time[i];
        }
        double prevVal = LW[i-1];
        if(prevVal >= 1 && currVal < 1) {   // value 1 neg crossing
            times_one.Add(time[i]);
            times_max.Add(time_maxmin);
            value_max = 1.0;
            search_max = false;
        } else if(prevVal < 1 && currVal >= 1) {   // value 1 pos crossing
            times_one.Add(time[i]);
            times_min.Add(time_maxmin);
            value_min = 1.0;
            search_max = true;
        }
    }

    List<double> periodTimes = new List<double>();
    List<double> timesAtPstart = new List<double>(); 

    for(int i = 0; i < times_max.Count - 1; i++) {
        periodTimes.Add(times_max.ElementAt(i+1) - times_max.ElementAt(i)); 
        timesAtPstart.Add(times_max.ElementAt(i));
    }
    for(int i = 0; i < times_min.Count - 1; i++) {
        periodTimes.Add(times_min.ElementAt(i+1) - times_min.ElementAt(i)); 
        timesAtPstart.Add(times_min.ElementAt(i));
    }
    for(int i = 0; i < times_one.Count - 2; i++) {
        periodTimes.Add(times_one.ElementAt(i+2) - times_one.ElementAt(i)); 
        timesAtPstart.Add(times_one.ElementAt(i));
    }

    Plot2Ddata procPlot = new Plot2Ddata();
    var fmt = new PlotFormat();
    fmt.Style = Styles.Points;
    fmt.PointSize = 1;
    procPlot.AddDataGroup(plot.dataGroups[0].Name, timesAtPstart, periodTimes, fmt);

    PlotTablePD[iRow, 0] = procPlot;
}


## Case 1 (m2_Oh01_eta04)

In [ ]:
var pltTab1 = PlotTableDM[0,0];
pltTab1.Title

m2_Oh01_eta04

In [ ]:
//var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/Oh_0dot1_LWinTime.txt");
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m2/AspRatioInTime_m2_Oh01_eta04.txt");
var basaran_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m2/basaranOscOh_0dot1.txt");
var becker_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m2/beckerEtAlOscOh_0dot1.txt");
var meradji_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m2/meradjiOscOh_0dot1.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Black;
pltTab1.AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Magenta;
pltTab1.AddDataGroup("Basaran (1992)", basaran_dat.GetColumn(0), basaran_dat.GetColumn(1), fmt);

fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Green;
pltTab1.AddDataGroup("Becker et. al. (1994)", becker_dat.GetColumn(0), becker_dat.GetColumn(1), fmt);

fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Orange;
pltTab1.AddDataGroup("Meradji et. al. (2001)", meradji_dat.GetColumn(0), meradji_dat.GetColumn(1), fmt);

//var gp = dat_plot.ToGnuplot();
//gp.PlotSVG()

In [ ]:
pltTab1.XrangeMin = 0;
pltTab1.XrangeMax = 2;
var gp = pltTab1.ToGnuplot();
gp.PlotSVG()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m2 O h01 e ta04 
 
 
 BoSSS-J686-3OrdInit 
 
 
 
 
 BoSSS-J686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M433.1,93.1 L486.5,93.1 M71.9,207.1 L73.6,194.9 L77.0,178.2 L82.1,153.0 L83.8,148.4 L85.5,139.4
 L87.2,133.5 L90.6,120.8 L92.2,121.7 L95.6,106.4 L97.3,103.0 L99.0,99.2 L100.7,96.5 L102.4,93.8
 L104.1,91.4 L105.8,89.6 L107.5,87.4 L109.2,85.8 L114.3,80.4 L116.0,88.3 L117.7,79.8 L121.1,77.4
 L126.2,76.3 L127.9,76.7 L129.5,77.3 L131.2,77.8 L132.9,78.3 L134.6,78.7 L136.3,79.1 L138.0,79.6
 L139.7,82.0 L141.4,82.4 L143.1,82.8 L144.8,81.7 L148.2,84.6 L151.6,87.9 L155.0,540.3 L156.7,92.6
 L158.4,94.1 L166.8,102.7 L168.5,104.7 L170.2,106.7 L177.0,115.4 L178.7,117.5 L180.4,119.5 L182.1,121.5
 L183.8,123.4 L185.5,125.3 L194.0,135.9 L195.7,137.6 L199.1,141.7 L207.5,152.0 L210.9,155.8 L212.6,157.6
 L214.3,159.5 L216.0,161.3 L217.7,163.1 L219.4,164.7 L221.1,166.1 L222.8,167.5 L226.2,171.3 L227.9,173.3
 L229.6,175.2 L231.3,176.9 L233.0,178.4 L234.7,179.8 L236.4,181.1 L238.1,182.3 L239.8,183.7 L241.5,185.0
 L243.1,186.2 L244.8,187.5 L246.5,189.4 L248.2,190.6 L253.3,193.9 L275.4,208.3 L277.1,209.3 L278.8,210.2
 L280.4,211.0 L283.8,212.8 L285.5,214.2 L287.2,215.1 L288.9,216.4 L290.6,217.8 L292.3,218.9 L294.0,220.0
 L295.7,220.6 L297.4,221.5 L299.1,222.5 L300.8,223.5 L311.0,229.3 L312.7,230.2 L314.4,231.2 L317.7,233.1
 L319.4,234.0 L321.1,235.0 L322.8,236.0 L326.2,239.2 L327.9,240.3 L329.6,241.4 L331.3,242.4 L333.0,243.5
 L334.7,244.3 L336.4,245.4 L338.1,246.2 L339.8,247.2 L341.5,248.3 L343.2,249.4 L344.9,250.3 L348.3,252.7
 L350.0,253.6 L351.7,255.3 L353.4,256.8 L355.0,258.2 L356.7,259.5 L358.4,260.8 L360.1,262.2 L361.8,263.5
 L363.5,264.8 L365.2,266.1 L366.9,267.4 L378.8,277.7 L380.5,279.3 L382.2,280.8 L383.9,282.1 L385.6,283.7
 L387.3,285.3 L389.0,287.1 L390.7,288.6 L392.3,290.2 L394.0,291.8 L395.7,293.5 L397.4,295.3 L400.8,298.8
 L404.2,302.3 L405.9,304.1 '/> 
 
 BoSSS-J686 
 
 
 BoSSS-J686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M433.1,117.1 L486.5,117.1 M71.9,207.1 L73.6,207.1 L75.3,207.2 L77.0,207.3 L78.7,207.5 L80.4,207.7
 L82.1,207.9 L83.8,208.2 L85.5,208.6 L87.2,209.0 L88.9,209.4 L90.6,209.8 L92.2,210.2 L93.9,210.7
 L95.6,211.2 L97.3,211.7 L99.0,212.3 L100.7,212.8 L102.4,213.4 L104.1,214.0 L105.8,214.6 L107.5,215.2
 L109.2,215.8 L110.9,216.4 L112.6,217.1 L114.3,217.7 L116.0,218.4 L117.7,219.0 L119.4,219.7 L121.1,220.4
 L122.8,221.1 L124.5,221.8 L126.2,222.5 L127.9,223.2 L129.5,223.9 L131.2,224.7 L132.9,225.4 L134.6,226.1
 L136.3,226.9 L138.0,227.6 L139.7,228.4 L141.4,229.1 L143.1,229.9 L144.8,230.6 L146.5,231.4 L148.2,232.2
 L149.9,232.9 L151.6,233.7 L153.3,234.5 L155.0,235.2 L156.7,236.0 L158.4,236.8 L160.1,237.6 L161.8,238.4
 L163.5,239.2 L165.2,240.0 L166.8,240.8 L168.5,241.6 L170.2,242.4 L171.9,243.2 L173.6,244.0 L175.3,244.9
 L177.0,245.7 L178.7,246.5 L180.4,247.3 L182.1,248.2 L183.8,249.0 L185.5,249.8 L187.2,250.7 L188.9,251.6
 L190.6,252.5 L192.3,253.3 L194.0,254.2 L195.7,255.0 L197.4,255.9 L199.1,256.7 L200.8,257.6 L202.5,258.5
 L204.1,259.4 L205.8,260.3 L207.5,261.2 L209.2,262.1 L210.9,263.0 L212.6,263.9 L214.3,264.8 L216.0,265.8
 L217.7,266.7 L219.4,267.6 L221.1,268.6 L222.8,269.6 L224.5,270.6 L226.2,271.6 L227.9,272.6 L229.6,273.7
 L231.3,274.6 L233.0,275.7 L234.7,276.7 L236.4,277.8 L238.1,278.8 L239.8,279.8 L241.5,280.9 L243.1,282.0
 L244.8,283.1 L246.5,284.2 L248.2,285.4 L249.9,286.5 L251.6,287.6 L253.3,288.8 L255.

In [ ]:
// foreach(var grp in pltTab1.dataGroups) {
//     Plot2Ddata dat = new Plot2Ddata();
//     dat.AddDataGroup(grp);
//     string name = "plotData/AspectRatioOverTime/m2/" + pltTab1.Title + "/" + grp.Name + ".txt";
//     dat.SaveToTextFile(name);
// }

## Case 2 (m2_Oh01_eta02)

In [ ]:
var pltTab2 = PlotTableDM[1,0];
pltTab2.Title

m2_Oh01_eta02

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m2/AspRatioInTime_m2_Oh01_eta02.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Black;
PlotTableDM[1,0].AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
pltTab2.XrangeMin = 0;
pltTab2.XrangeMax = 2;
var gp = pltTab2.ToGnuplot();
gp.PlotSVG()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.8 
 
 
 
 
 0.9 
 
 
 
 
 1 
 
 
 
 
 1.1 
 
 
 
 
 1.2 
 
 
 
 
 1.3 
 
 
 
 
 1.4 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m2 O h01 e ta02 
 
 
 BoSSS-J686-3OrdInit 
 
 
 
 
 BoSSS-J686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M466.4,93.1 L519.8,93.1 M71.9,122.0 L73.6,121.2 L75.3,120.2 L77.0,119.3 L78.7,118.6 L80.4,118.0
 L82.1,117.6 L83.8,117.3 L85.5,117.1 L87.2,117.0 L88.9,117.0 L90.6,117.1 L92.2,117.3 L93.9,117.6
 L95.6,118.0 L97.3,118.5 L99.0,119.1 L100.7,119.7 L102.4,120.4 L104.1,121.2 L105.8,122.1 L107.5,123.0
 L109.2,124.0 L110.9,125.0 L112.6,126.0 L114.3,127.2 L116.0,128.3 L117.7,129.5 L119.4,130.7 L121.1,132.0
 L122.8,133.3 L124.5,134.6 L126.2,136.0 L127.9,137.4 L129.5,138.8 L131.2,140.2 L132.9,141.7 L134.6,143.1
 L136.3,144.6 L138.0,146.2 L139.7,147.7 L141.4,149.3 L143.1,150.9 L144.8,152.5 L146.5,154.1 L148.2,155.7
 L149.9,157.4 L151.6,159.0 L153.3,160.7 L155.0,162.5 L156.7,164.2 L158.4,165.9 L160.1,167.7 L161.8,169.5
 L163.5,171.3 L165.2,173.1 L166.8,174.9 L168.5,176.8 L170.2,178.7 L171.9,180.5 L173.6,182.5 L175.3,184.3
 L177.0,186.3 L178.7,188.2 L180.4,190.1 L182.1,192.2 L183.8,194.2 L185.5,196.2 L187.2,198.2 L188.9,200.3
 L190.6,202.4 L192.3,204.4 L194.0,206.5 L195.7,208.6 L197.4,210.8 L199.1,212.9 L200.8,215.1 L202.5,217.3
 L204.1,219.5 L205.8,221.7 L207.5,223.9 L209.2,226.2 L210.9,228.3 L212.6,230.7 L214.3,233.0 L216.0,235.4
 L217.7,237.8 L219.4,240.1 L221.1,242.6 L222.8,245.1 L224.5,247.4 L226.2,249.9 L227.9,252.4 L229.6,254.9
 L231.3,257.4 L233.0,260.0 L234.7,262.5 L236.4,265.1 L238.1,267.6 L239.8,270.0 L241.5,272.6 L243.1,275.3
 L244.8,278.0 L246.5,280.6 L248.2,283.3 L249.9,285.8 L251.6,288.5 L253.3,291.3 L255.0,294.0 L256.7,296.5
 L258.4,299.3 L260.1,302.0 L261.8,304.9 L263.5,307.4 L265.2,310.3 L266.9,313.1 L268.6,316.0 L270.3,318.8
 L272.0,321.5 L273.7,324.3 L275.4,327.1 L277.1,329.7 L278.8,332.5 L280.4,335.2 L282.1,338.0 L283.8,340.7
 L285.5,343.5 L287.2,346.2 L288.9,349.0 L290.6,351.7 L292.3,354.4 L294.0,357.1 L295.7,359.7 L297.4,362.4
 L299.1,365.1 L300.8,367.7 L302.5,370.3 L304.2,373.0 L305.9,375.6 L307.6,378.2 L309.3,380.7 L311.0,383.3
 L312.7,385.8 L314.4,388.4 L316.1,390.9 L317.7,393.4 L319.4,395.9 L321.1,398.3 L322.8,400.8 L324.5,403.2
 L326.2,405.6 L327.9,408.0 L329.6,410.3 L331.3,412.6 L333.0,415.0 L334.7,417.2 L336.4,419.5 L338.1,421.8
 L339.8,423.9 L341.5,426.1 L343.2,428.3 L344.9,430.5 L346.6,432.6 L348.3,434.7 L350.0,436.8 L351.7,438.9
 L353.4,440.9 L355.0,442.9 L356.7,444.9 L358.4,446.9 L360.1,448.8 L361.8,450.7 L363.5,452.6 L365.2,454.4
 L366.9,456.1 L368.6,457.9 L370.3,459.7 L372.0,461.4 L373.7,463.1 L375.4,464.8 L377.1,466.5 L378.8,468.1
 L380.5,469.7 L382.2,471.2 L383.9,472.8 L385.6,474.3 L387.3,475.7 L389.0,477.2 L390.7,478.6 L392.3,480.0
 L394.0,481.4 L395.7,482.7 L397.4,484.0 L399.1,485.3 L400.8,486.6 L402.5,487.8 L404.2,489.0 L405.9,490.2
 L407.6,491.4 L409.3,492.5 L411.0,493.6 L412.7,494.7 L414.4,495.8 L416.1,496.8 L417.8,497.8 L419.5,498.7
 L421.2,499.7 L422.9,500.6 L424.6,501.5 L426.3,502.4 L428.0,503.2 L429.7,504.0 L431.3,504.5 L433.0,505.3
 L434.7,506.1 L436.4,506.8 L438.1,507.5 L439.8,508.2 L441.5,508.8 L443.2,509.4 L444.9,510.0 L446.6,510.5
 L448.3,511.0 L450.0,511.5 L451.7,512.0 L453.4,512.4 L455.1,512.7 L456.8,513.1 L458.5,513.4 L460.2,513.7
 L461.9,514.0 L463.6,514.3 L465.3,514.5 L467.0,514.7 L468.6,514.9 L470.3,515.0 L472.0,515.2 L473.7,515.3
 L475.4,515.3 L499.2,513.6 L500.9,513.3 L502.6,512.9 L504.3,512.5 L505.9,512.2 L507.6,511.7 L509.3,511.3
 L511.0,510.9 L5

In [ ]:
// foreach(var grp in pltTab2.dataGroups) {
//     Plot2Ddata dat = new Plot2Ddata();
//     dat.AddDataGroup(grp);
//     string name = "plotData/AspectRatioOverTime/m2/" + pltTab2.Title + "/" + grp.Name + ".txt";
//     dat.SaveToTextFile(name);
// }

## Case 3 (m2_Oh01_eta01)

In [ ]:
var pltTab3 = PlotTableDM[2,0];
pltTab3.Title

m2_Oh01_eta01

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m2/AspRatioInTime_m2_Oh01_eta01.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Black;
pltTab3.AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
pltTab3.XrangeMin = 0;
pltTab3.XrangeMax = 2;
var gp = pltTab3.ToGnuplot();
gp.PlotSVG()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.9 
 
 
 
 
 0.95 
 
 
 
 
 1 
 
 
 
 
 1.05 
 
 
 
 
 1.1 
 
 
 
 
 1.15 
 
 
 
 
 1.2 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m2 O h01 e ta01 
 
 
 BoSSS-J686-3OrdInit 
 
 
 
 
 BoSSS-J686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M466.4,93.1 L519.8,93.1 M80.2,137.5 L81.9,137.5 L83.5,137.5 L85.2,137.6 L86.9,137.7 L88.6,137.9
 L90.2,138.1 L91.9,138.3 L93.6,138.6 L95.3,138.9 L96.9,139.3 L98.6,139.7 L100.3,140.1 L102.0,140.6
 L103.6,141.2 L105.3,141.8 L107.0,142.4 L108.7,143.0 L110.3,143.7 L112.0,144.4 L113.7,145.2 L115.4,146.0
 L117.0,146.9 L118.7,147.8 L120.4,148.7 L122.1,149.6 L123.7,150.6 L125.4,151.6 L127.1,152.7 L128.8,153.8
 L130.4,154.9 L132.1,156.1 L133.8,157.3 L135.5,158.5 L137.1,159.8 L138.8,161.1 L140.5,162.4 L142.2,163.8
 L143.8,165.2 L145.5,166.6 L147.2,168.0 L148.9,169.5 L150.5,171.0 L152.2,172.6 L153.9,174.2 L155.6,175.8
 L157.2,177.3 L158.9,179.0 L160.6,180.7 L162.3,182.4 L163.9,184.2 L165.6,186.0 L167.3,187.8 L169.0,189.6
 L170.6,191.5 L172.3,193.4 L174.0,195.3 L175.7,197.2 L177.3,199.2 L179.0,201.2 L180.7,203.2 L182.4,205.2
 L184.0,207.3 L185.7,209.4 L187.4,211.5 L189.1,213.6 L190.7,215.8 L192.4,218.0 L194.1,220.2 L195.8,222.4
 L197.4,224.7 L199.1,227.0 L200.8,229.3 L202.5,231.6 L204.1,234.0 L205.8,236.3 L207.5,238.7 L209.2,241.1
 L210.8,243.5 L212.5,245.9 L214.2,248.3 L215.9,250.8 L217.5,253.2 L219.2,255.7 L220.9,258.2 L222.6,260.7
 L224.2,263.3 L225.9,265.9 L227.6,268.4 L229.3,271.0 L230.9,273.6 L232.6,276.2 L234.3,278.8 L236.0,281.4
 L237.6,284.0 L239.3,286.7 L241.0,289.3 L242.7,292.0 L244.3,294.6 L246.0,297.3 L247.7,300.0 L249.3,302.7
 L251.0,305.4 L252.7,308.1 L254.4,310.8 L256.0,313.5 L257.7,316.2 L259.4,318.9 L261.1,321.6 L262.7,324.3
 L264.4,327.0 L266.1,329.7 L267.8,332.4 L269.4,335.1 L271.1,337.8 L272.8,340.5 L274.5,343.2 L276.1,345.9
 L277.8,348.6 L279.5,351.3 L281.2,353.9 L282.8,356.6 L284.5,359.3 L286.2,361.9 L287.9,364.6 L289.5,367.2
 L291.2,369.8 L292.9,372.4 L294.6,375.1 L296.2,377.6 L297.9,380.2 L299.6,382.8 L301.3,385.4 L302.9,387.9
 L304.6,390.5 L306.3,393.0 L308.0,395.5 L309.6,398.0 L311.3,400.5 L313.0,402.9 L314.7,405.4 L316.3,407.8
 L318.0,410.2 L319.7,412.6 L321.4,415.0 L323.0,417.3 L324.7,419.7 L326.4,422.0 L328.1,424.3 L329.7,426.6
 L331.4,429.0 L333.1,431.2 L334.8,433.4 L336.4,435.5 L338.1,437.7 L339.8,439.8 L341.5,441.9 L343.1,444.0
 L344.8,446.1 L346.5,448.3 L348.2,450.3 L349.8,452.2 L351.5,454.2 L353.2,456.3 L354.9,458.1 L356.5,460.0
 L358.2,461.9 L359.9,463.8 L361.6,465.6 L363.2,467.5 L364.9,469.3 L366.6,471.1 L368.3,472.8 L369.9,474.5
 L371.6,476.2 L373.3,477.9 L375.0,479.5 L376.6,481.1 L378.3,482.7 L380.0,484.3 L381.7,485.8 L383.3,487.3
 L385.0,488.7 L386.7,490.1 L388.4,491.5 L390.0,492.9 L391.7,494.3 L393.4,495.6 L395.1,496.9 L396.7,498.1
 L398.4,499.4 L400.1,500.6 L401.8,501.7 L403.4,502.9 L405.1,504.0 L406.8,505.1 L408.5,506.1 L410.1,507.2
 L411.8,508.2 L413.5,509.1 L415.2,510.1 L416.8,511.0 L418.5,511.9 L420.2,512.7 L421.8,513.6 L423.5,514.4
 L425.2,515.1 L426.9,515.9 L428.5,516.6 L430.2,517.2 L431.9,517.9 L433.6,518.5 L435.2,519.1 L436.9,519.7
 L438.6,520.2 L440.3,520.7 L441.9,521.2 L443.6,521.6 L445.3,521.9 L447.0,522.3 L448.6,522.7 L450.3,523.0
 L452.0,523.3 L453.7,523.6 L455.3,523.9 L457.0,524.1 L458.7,524.3 L460.4,524.5 L462.0,524.6 L463.7,524.7
 L465.4,524.8 L467.1,524.8 L468.7,524.9 L470.4,524.9 L472.1,524.8 L473.8,524.8 L475.4,524.7 L477.1,524.6
 L478.8,524.4 L480.5,524.3 L482.1,524.1 L483.8,523.8 L485.5,523.6 L487.2,523.3 L488.8,523.0 L490.5,522.7
 L492.2,

In [ ]:
// foreach(var grp in pltTab3.dataGroups) {
//     Plot2Ddata dat = new Plot2Ddata();
//     dat.AddDataGroup(grp);
//     string name = "plotData/AspectRatioOverTime/m3/" + pltTab3.Title + "/" + grp.Name + ".txt";
//     dat.SaveToTextFile(name);
// }

## Case 4 (m3_Oh01_eta04)

In [ ]:
var pltTab4 = PlotTableDM[3,0];
pltTab4.Title

m3_Oh01_eta04

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m3/AspRatioInTime_m3_Oh01_eta04.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Black;
pltTab4.AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
pltTab4.XrangeMin = 0;
pltTab4.XrangeMax = 3;
var gp = pltTab4.ToGnuplot();
gp.PlotSVG()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.95 
 
 
 
 
 1 
 
 
 
 
 1.05 
 
 
 
 
 1.1 
 
 
 
 
 1.15 
 
 
 
 
 1.2 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m3 O h01 e ta04 
 
 
 BoSSS-J686-3OrdInit 
 
 
 
 
 
 BoSSS-J686 
 
 
 BoSSS-J686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M466.4,93.1 L519.8,93.1 M82.4,447.6 L98.1,358.5 L99.2,349.4 L102.5,330.7 L103.6,324.2 L105.9,310.2
 L107.0,304.9 L108.1,300.6 L109.2,294.8 L113.7,284.2 L114.8,279.5 L115.9,275.0 L117.0,270.4 L118.2,265.4
 L119.3,260.9 L120.4,256.4 L124.9,239.0 L126.0,234.2 L129.3,219.3 L130.4,223.3 L131.6,215.8 L137.1,193.5
 L149.4,175.3 L150.5,173.9 L151.7,172.6 L152.8,170.1 L153.9,169.7 L155.0,168.1 L156.1,167.4 L157.2,166.9
 L158.4,167.7 L172.9,175.9 L181.8,198.4 L236.5,398.7 L244.3,277.3 L246.6,274.9 L247.7,274.4 L248.8,273.8
 L249.9,273.0 L251.0,272.1 L267.8,264.4 L270.0,264.7 L271.1,264.8 L272.2,264.9 L273.4,264.9 L274.5,267.3
 L275.6,267.5 L276.7,267.8 L282.3,277.4 L283.4,278.5 L284.5,279.6 L285.6,280.6 L286.8,281.5 L287.9,282.8
 L289.0,284.1 L290.1,285.5 L291.2,286.4 L292.3,287.7 L301.3,297.3 L302.4,298.4 L303.5,308.2 L304.6,309.2
 L305.7,315.7 L306.8,322.0 L308.0,326.6 L309.1,330.5 L310.2,331.8 L311.3,333.2 L312.4,334.3 L313.5,335.9
 L314.7,336.3 L315.8,337.8 L316.9,339.0 L319.1,341.4 L320.2,343.1 L321.4,345.8 L322.5,347.6 L323.6,349.4
 L331.4,365.0 L332.5,367.4 L333.6,368.8 L339.2,382.1 L340.3,385.2 L341.5,387.5 L342.6,389.8 L343.7,393.1
 L344.8,394.2 L345.9,397.0 L347.0,399.2 L348.2,401.4 L349.3,403.7 L350.4,406.0 L351.5,408.3 L352.6,410.6
 L353.7,412.4 L354.9,414.7 L356.0,417.2 L361.6,429.9 L362.7,431.8 L363.8,433.6 L364.9,435.3 L366.0,437.0
 L367.1,438.7 L368.3,440.3 L377.2,453.3 L378.3,455.0 L379.4,456.6 L380.5,458.4 L381.7,460.1 L382.8,461.6
 L383.9,463.0 L385.0,464.3 L386.1,465.6 L407.3,480.7 L408.5,480.9 L409.6,481.5 L410.7,482.4 L411.8,483.3
 L415.1,486.5 L416.3,486.5 L417.4,486.8 L418.5,487.2 L419.6,487.7 L420.7,488.3 '/> 
 
 WNLT 
 
 
 WNLT 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M466.4,117.1 L519.8,117.1 M80.2,448.3 L82.4,446.5 L84.7,441.1 L86.9,432.6 L89.1,421.2 L91.4,407.5
 L93.6,391.7 L95.8,374.3 L98.1,355.6 L100.3,336.1 L102.5,316.1 L104.8,296.0 L107.0,276.1 L109.2,256.6
 L111.5,238.0 L113.7,220.3 L115.9,203.9 L118.2,188.8 L120.4,175.3 L122.6,163.3 L124.9,153.0 L127.1,144.5
 L129.3,137.6 L131.6,132.4 L133.8,128.9 L136.0,127.0 L138.3,126.6 L140.5,127.5 L142.7,129.8 L145.0,133.3
 L147.2,137.8 L149.4,143.3 L151.7,149.5 L153.9,156.5 L156.1,164.0 L158.4,171.9 L160.6,180.1 L162.8,188.6
 L165.1,197.1 L167.3,205.7 L169.5,214.1 L171.8,222.4 L174.0,230.5 L176.2,238.2 L178.5,245.6 L180.7,252.6
 L182.9,259.1 L185.2,265.2 L187.4,270.8 L189.6,275.9 L191.9,280.4 L194.1,284.5 L196.3,288.0 L198.5,291.1
 L200.8,293.6 L203.0,295.7 L205.2,297.3 L207.5,298.5 L209.7,299.4 L211.9,299.8 L214.2,299.9 L216.4,299.7
 L218.6,299.3 L220.9,298.6 L223.1,297.7 L225.3,296.6 L227.6,295.4 L229.8,294.1 L232.0,292.7 L234.3,291.3
 L236.5,289.9 L238.7,288.6 L241.0,287.2 L243.2,286.0 L245.4,284.9 L247.7,283.9 L249.9,283.1 L252.1,282.4
 L254.4,282.0 L256.6,281.7 L258.8,281.7 L261.1,282.0 L263.3,282.5 L265.5,283.3 L267.8,284.4 L270.0,285.7
 L272.2,287.4 L274.5,289.3 L276.7,291.6 L278.9,294.1 L281.2,297.0 L283.4,300.1 L285.6,303.5 L287.9,307.2
 L290.1,311.1 L292.3,315.3 L294.6,319.6 L296.8,324.2 L299.0,329.0 L301.3,334.0 L303.5,339.1 L305.7,344.3
 L308.0,349.6 L310.2,355.0 L312.4,360.5 L314.7,366.0 L316.9,371.4 L319.1,376.9 L321.4,382.3 L323.6,387.7
 L325.8,393.0 L328.1,398.2 L330.3,403.2 L332.5,408.1 L334.8,412.9 L337.0

In [ ]:
// foreach(var grp in pltTab4.dataGroups) {
//     Plot2Ddata dat = new Plot2Ddata();
//     dat.AddDataGroup(grp);
//     string name = "plotData/AspectRatioOverTime/m3/" + pltTab4.Title + "/" + grp.Name + ".txt";
//     dat.SaveToTextFile(name);
// }

## Case 5 (m4_Oh01_eta04)

In [ ]:
var pltTab5 = PlotTableDM[4,0];
pltTab5.Title

m4_Oh01_eta04

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m4/AspRatioInTime_m4_Oh01_eta04.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Black;
pltTab5.AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
pltTab5.XrangeMin = 0;
pltTab5.XrangeMax = 3;
var gp = pltTab5.ToGnuplot();
gp.PlotSVG()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.9 
 
 
 
 
 0.95 
 
 
 
 
 1 
 
 
 
 
 1.05 
 
 
 
 
 1.1 
 
 
 
 
 1.15 
 
 
 
 
 1.2 
 
 
 
 
 1.25 
 
 
 
 
 1.3 
 
 
 
 
 1.35 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m4 O h01 e ta04 
 
 
 BoSSS-J686-3OrdInit 
 
 
 
 
 BoSSS-J686-3OrdInit 
 
 
 
 
 
 BoSSS-J686 
 
 
 BoSSS-J686 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M466.4,117.1 L519.8,117.1 M80.2,207.1 L81.3,207.1 L82.4,207.1 L83.5,207.0 L84.7,206.9 L85.8,206.9
 L86.9,206.8 L88.0,206.7 L89.1,206.5 L90.2,206.4 L91.4,206.3 L92.5,206.0 L93.6,205.8 L94.7,205.5
 L95.8,205.2 L96.9,204.8 L98.1,204.5 L99.2,204.2 L100.3,203.8 L101.4,203.4 L102.5,203.1 L103.6,202.7
 L104.8,202.5 L105.9,202.0 L107.0,201.5 L108.1,201.0 L109.2,200.5 L110.3,200.1 L111.5,199.6 L112.6,199.2
 L113.7,198.2 L114.8,197.7 L115.9,197.1 L117.0,196.6 L118.2,196.0 L119.3,195.7 L120.4,195.2 L121.5,195.1
 L122.6,195.0 L123.7,194.9 L124.9,194.7 L126.0,195.0 L127.1,195.5 L128.2,196.3 L129.3,197.4 L130.4,198.8
 L131.6,200.5 L132.7,202.5 L133.8,204.7 L134.9,207.2 L136.0,210.0 L137.1,213.1 L138.3,216.7 L139.4,220.7
 L140.5,225.0 L141.6,229.8 L142.7,235.1 L143.8,240.7 L145.0,246.7 L146.1,252.6 L147.2,258.7 L148.3,265.7
 L149.4,273.3 L150.5,281.5 L151.7,290.5 L152.8,298.4 L153.9,304.8 L155.0,312.2 L156.1,320.0 L157.2,328.3
 L158.4,337.1 L159.5,346.1 L160.6,354.9 L161.7,363.5 L162.8,371.6 L163.9,379.3 L165.1,386.5 L166.2,393.2
 L167.3,398.9 L168.4,404.7 L169.5,410.0 L170.6,415.0 L171.8,419.6 L172.9,423.8 L174.0,427.3 L175.1,430.5
 L176.2,433.3 L177.3,435.7 L178.5,437.5 L179.6,438.9 L180.7,439.7 L181.8,440.1 L182.9,439.9 L184.0,439.3
 L185.2,438.2 L186.3,436.7 L187.4,434.9 L188.5,432.7 L189.6,430.2 L190.7,427.4 L191.9,424.4 L193.0,421.2
 L194.1,417.8 L195.2,413.5 L196.3,409.5 L197.4,405.6 L199.7,397.4 L200.8,393.5 L201.9,389.6 L203.0,385.8
 L204.1,382.0 L205.2,378.3 L208.6,365.7 L209.7,362.2 L210.8,358.9 L211.9,355.5 L213.1,352.2 L214.2,348.9
 L215.3,345.7 L216.4,342.5 L217.5,339.3 L218.6,336.2 L219.8,330.5 L220.9,327.7 L222.0,324.5 L223.1,321.7
 L224.2,319.0 L225.3,315.9 L226.5,312.9 L227.6,310.1 L228.7,307.4 L229.8,304.8 L230.9,308.8 L232.0,306.1
 L233.2,303.6 L234.3,301.6 L235.4,299.9 L236.5,301.7 L238.7,299.4 L241.0,297.4 L242.1,296.4 L243.2,294.9
 L244.3,293.4 L245.4,291.9 L246.6,290.9 L247.7,290.0 L248.8,290.4 L249.9,290.3 L251.0,290.3 L252.1,290.3
 L253.3,290.5 L254.4,290.7 L255.5,293.9 L256.6,294.1 L257.7,294.2 L258.8,294.4 L260.0,294.3 L261.1,294.2
 L262.2,294.1 L263.3,294.2 L264.4,294.4 L265.5,295.2 L266.7,295.6 L267.8,296.2 L268.9,296.8 L270.0,297.4
 L271.1,298.1 L272.2,298.9 L273.4,299.7 L274.5,300.6 L275.6,301.5 L276.7,302.4 L277.8,303.4 L278.9,304.4
 L280.1,305.5 L281.2,306.7 L282.3,307.8 L283.4,309.1 L284.5,310.3 L285.6,311.7 L286.8,313.0 L287.9,314.4
 L289.0,315.9 L290.1,317.4 L291.2,318.9 L292.3,320.4 L293.5,322.0 L294.6,323.7 L295.7,325.8 L296.8,327.9
 L297.9,332.8 L299.0,335.1 L300.2,337.0 L301.3,338.5 L302.4,340.2 L303.5,342.0 L304.6,344.8 L305.7,346.9
 L308.0,350.9 L309.1,352.9 L310.2,355.0 L311.3,357.2 L312.4,359.5 L313.5,362.2 L314.7,364.9 L315.8,369.7
 L316.9,372.2 L318.0,374.4 L319.1,376.8 L320.2,379.3 L321.4,381.9 L322.5,384.6 L323.6,387.4 L324.7,390.2
 L325.8,393.0 L326.9,395.8 L328.1,398.7 L329.2,401.5 L330.3,404.3 L331.4,407.0 L332.5,409.8 L333.6,412.5
 L334.8,415.1 L335.9,417.8 L337.0,420.4 L338.1,423.0 L339.2,426.8 L340.3,429.4 L341.5,432.3 L342.6,434.5
 L343.7,437.3 L344.8,439.8 L345.9,442.2 L347.0,444.4 L348.2,446.6 L349.3,448.7 L350.4,450.

In [ ]:
// foreach(var grp in pltTab5.dataGroups) {
//     Plot2Ddata dat = new Plot2Ddata();
//     dat.AddDataGroup(grp);
//     string name = "plotData/AspectRatioOverTime/m4/" + pltTab5.Title + "/" + grp.Name + ".txt";
//     dat.SaveToTextFile(name);
// }

## Case 6 (m4_Oh01_eta01)

In [ ]:
var pltTab6 = PlotTableDM[5,0];
pltTab6.Title

m4_Oh01_eta01

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m4/AspRatioInTime_m4_Oh01_eta01.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Black;
pltTab6.AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
pltTab6.XrangeMin = 0;
pltTab6.XrangeMax = 3;
var gp = pltTab6.ToGnuplot();
gp.PlotSVG()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.97 
 
 
 
 
 0.98 
 
 
 
 
 0.99 
 
 
 
 
 1 
 
 
 
 
 1.01 
 
 
 
 
 1.02 
 
 
 
 
 1.03 
 
 
 
 
 1.04 
 
 
 
 
 1.05 
 
 
 
 
 1.06 
 
 
 
 
 1.07 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m4 O h01 e ta01 
 
 
 BoSSS-J686-3OrdInit 
 
 
 
 
 BoSSS-J686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M466.4,93.1 L519.8,93.1 M80.2,117.6 L81.3,114.6 L82.4,111.9 L83.5,110.3 L84.7,109.7 L85.8,109.9
 L86.9,110.4 L88.0,111.8 L89.1,113.7 L90.2,116.0 L91.4,118.4 L92.5,121.1 L93.6,124.0 L94.7,127.0
 L95.8,129.8 L96.9,133.3 L98.1,137.0 L99.2,140.8 L100.3,144.7 L101.4,148.8 L102.5,153.1 L103.6,157.5
 L104.8,162.2 L105.9,167.0 L107.0,172.0 L108.1,177.4 L109.2,183.4 L110.3,189.4 L111.5,195.3 L112.6,201.8
 L113.7,208.4 L114.8,215.2 L115.9,222.2 L117.0,229.3 L118.2,236.6 L119.3,244.0 L120.4,251.6 L121.5,259.2
 L122.6,267.1 L123.7,275.0 L124.9,283.0 L126.0,291.1 L127.1,299.2 L128.2,307.4 L129.3,315.7 L130.4,323.9
 L131.6,332.2 L132.7,340.4 L133.8,348.6 L134.9,356.7 L136.0,364.8 L137.1,372.8 L138.3,380.2 L139.4,388.1
 L140.5,395.8 L141.6,403.1 L142.7,410.5 L143.8,417.7 L145.0,424.7 L146.1,431.5 L147.2,438.0 L148.3,444.2
 L149.4,450.2 L150.5,455.9 L151.7,461.3 L152.8,466.5 L153.9,470.9 L155.0,475.5 L157.2,483.9 L158.4,487.6
 L159.5,491.0 L160.6,494.1 L161.7,496.8 L162.8,499.1 L163.9,501.1 L165.1,502.8 L166.2,504.1 L167.3,505.1
 L168.4,505.8 L169.5,506.2 L170.6,506.3 L171.8,506.2 L172.9,505.7 L174.0,504.9 L175.1,503.9 L176.2,502.6
 L177.3,501.0 L178.5,499.3 L179.6,497.3 L180.7,495.0 L181.8,492.6 L182.9,490.0 L184.0,487.2 L185.2,484.2
 L186.3,481.1 L187.4,477.9 L188.5,474.5 L189.6,471.0 L190.7,467.4 L191.9,463.7 L193.0,459.9 L194.1,455.4
 L195.2,451.6 L196.3,447.8 L197.4,443.8 L198.5,439.9 L199.7,435.8 L200.8,431.8 L201.9,427.7 L203.0,423.5
 L204.1,419.4 L205.2,415.3 L206.4,411.1 L207.5,407.0 L208.6,403.0 L209.7,398.9 L210.8,394.9 L211.9,391.0
 L213.1,386.4 L214.2,382.8 L215.3,379.2 L216.4,375.6 L217.5,371.4 L218.6,368.2 L219.8,364.9 L220.9,361.7
 L222.0,358.5 L223.1,355.4 L224.2,352.3 L225.3,349.3 L226.5,346.3 L227.6,343.4 L228.7,340.6 L229.8,337.9
 L230.9,335.3 L232.0,332.8 L233.2,330.4 L234.3,328.1 L235.4,325.9 L236.5,323.8 L237.6,321.8 L238.7,319.9
 L239.9,318.1 L241.0,316.4 L242.1,314.9 L243.2,313.4 L244.3,312.0 L245.4,310.8 L246.6,309.6 L247.7,308.6
 L248.8,307.6 L249.9,306.8 L251.0,306.1 L252.1,305.4 L253.3,304.9 L254.4,304.5 L255.5,304.1 L256.6,303.9
 L257.7,303.7 L258.8,303.7 L260.0,303.8 L261.1,303.9 L262.2,304.2 L263.3,304.5 L264.4,304.9 L265.5,305.5
 L266.7,306.1 L267.8,306.8 L268.9,307.6 L270.0,308.5 L271.1,309.4 L272.2,310.5 L273.4,311.6 L274.5,312.8
 L275.6,314.1 L276.7,315.5 L277.8,316.9 L278.9,318.4 L280.1,320.0 L281.2,321.6 L282.3,323.3 L283.4,325.1
 L284.5,326.9 L286.8,330.7 L287.9,332.7 L289.0,334.7 L290.1,336.8 L291.2,338.9 L292.3,341.1 L293.5,343.3
 L294.6,345.5 L295.7,347.8 L296.8,350.1 L297.9,352.4 L299.0,354.7 L300.2,357.0 L301.3,359.4 L302.4,361.7
 L303.5,364.1 L304.6,366.5 L305.7,368.9 L306.8,371.2 L308.0,373.6 L309.1,375.9 L310.2,378.3 L311.3,380.6
 L312.4,382.9 L313.5,385.2 L314.7,387.4 L315.8,389.7 L316.9,391.8 L318.0,394.0 L319.1,396.1 L320.2,398.2
 L321.4,400.3 L322.5,402.3 L323.6,404.2 L324.7,406.1 L325.8,408.0 L326.9,409.8 L331.4,416.4 L332.5,417.9
 L333.6,419.4 L334.8,420.8 L335.9,422.1 L337.0,423.4 L338.1,424.6 L339.2,425.7 L340.3,426.3 L341.5,427.3
 L342.6,428.4 L343.7,429.5 L344.8,430.4 L345.9,431.3 L347.0,432.1 L348.2,43

In [ ]:
// foreach(var grp in pltTab6.dataGroups) {
//     Plot2Ddata dat = new Plot2Ddata();
//     dat.AddDataGroup(grp);
//     string name = "plotData/AspectRatioOverTime/m4/" + pltTab6.Title + "/" + grp.Name + ".txt";
//     dat.SaveToTextFile(name);
// }

## Case 7 (m4_Oh056_eta005)

In [ ]:
var pltTab7 = PlotTableDM[6,0];
pltTab7.Title

m4_Oh056_eta005

In [ ]:
var wnlt_dat = IMatrixExtensions.LoadFromTextFile($"data/LWoverTime/m4/AspRatioInTime_m4_Oh056_eta005.txt");

//var dat_plot = new Plot2Ddata();
var fmt = new PlotFormat();
fmt.Style = Styles.Lines; 
fmt.LineWidth = 3;
fmt.LineColor = LineColors.Black;
pltTab7.AddDataGroup("WNLT", wnlt_dat.GetColumn(0), wnlt_dat.GetColumn(1), fmt);

In [ ]:
pltTab7.XrangeMin = 0;
pltTab7.XrangeMax = 3;
var gp = pltTab7.ToGnuplot();
gp.PlotSVG()

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.97 
 
 
 
 
 0.98 
 
 
 
 
 0.99 
 
 
 
 
 1 
 
 
 
 
 1.01 
 
 
 
 
 1.02 
 
 
 
 
 1.03 
 
 
 
 
 1.04 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 2.5 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 aspect ratio L/W 
 
 
 
 
 time t 
 
 
 
 
 m4 O h056 e ta005 
 
 
 BoSSS-J686-3OrdInit 
 
 
 
 
 BoSSS-J686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M466.4,93.1 L519.8,93.1 M80.2,134.8 L81.3,135.2 L82.4,135.1 L83.5,135.4 L84.7,136.3 L85.8,137.7
 L86.9,139.6 L88.0,141.9 L89.1,144.6 L90.2,147.8 L91.4,151.3 L92.5,155.3 L93.6,159.6 L94.7,164.2
 L95.8,169.1 L96.9,174.4 L98.1,179.9 L99.2,185.6 L100.3,191.5 L101.4,197.7 L102.5,204.0 L103.6,210.5
 L104.8,217.1 L105.9,223.8 L107.0,230.6 L108.1,237.5 L109.2,244.4 L110.3,251.4 L111.5,258.4 L112.6,265.5
 L113.7,272.5 L114.8,279.6 L115.9,286.6 L117.0,293.6 L118.2,300.5 L119.3,307.4 L120.4,314.2 L121.5,321.0
 L122.6,327.7 L123.7,334.3 L124.9,340.9 L126.0,347.3 L127.1,353.7 L128.2,360.0 L129.3,366.1 L130.4,372.2
 L131.6,378.1 L132.7,384.0 L133.8,389.7 L134.9,395.3 L136.0,400.8 L137.1,406.2 L138.3,411.4 L139.4,416.6
 L140.5,421.6 L141.6,426.5 L142.7,431.3 L143.8,435.9 L145.0,440.5 L146.1,444.9 L147.2,449.2 L148.3,453.4
 L149.4,457.5 L150.5,461.5 L151.7,465.3 L152.8,469.1 L153.9,472.7 L155.0,476.2 L156.1,479.6 L157.2,482.9
 L158.4,486.1 L159.5,489.1 L160.6,492.1 L161.7,495.0 L162.8,497.7 L163.9,500.4 L165.1,503.0 L166.2,505.4
 L167.3,507.8 L168.4,510.1 L169.5,512.2 L170.6,514.3 L171.8,516.3 L172.9,518.2 L174.0,520.1 L175.1,521.8
 L176.2,523.5 L177.3,525.0 L178.5,526.5 L179.6,527.9 L180.7,529.3 L181.8,530.5 L182.9,531.7 L184.0,532.8
 L185.2,533.9 L186.3,534.8 L187.4,535.7 L188.5,536.6 L189.6,537.3 L190.7,538.0 L191.9,538.7 L193.0,539.2
 L194.1,539.8 L195.2,540.2 L196.3,540.7 L197.4,540.9 L198.5,541.3 L199.7,541.4 L200.8,541.6 L201.9,541.7
 L203.0,541.8 L204.1,541.8 L205.2,541.8 L206.4,541.8 L207.5,541.7 L208.6,541.5 L209.7,541.3 L210.8,541.1
 L211.9,540.8 L213.1,540.5 L214.2,540.1 L215.3,539.7 L216.4,539.3 L217.5,538.8 L218.6,538.3 L219.8,537.8
 L220.9,537.2 L222.0,536.6 L223.1,536.0 L224.2,535.5 L225.3,534.7 L226.5,534.1 L227.6,533.2 L228.7,532.4
 L229.8,531.6 L230.9,530.8 L232.0,529.9 L233.2,529.1 L234.3,528.2 L235.4,527.2 L236.5,526.3 L237.6,525.3
 L238.7,524.4 L239.9,523.4 L241.0,522.4 L242.1,521.3 L243.2,520.3 L244.3,519.2 L245.4,518.1 L246.6,517.0
 L247.7,515.9 L248.8,514.8 L249.9,513.6 L251.0,512.5 L252.1,511.3 L253.3,510.2 L254.4,509.0 L255.5,507.8
 L256.6,506.6 L257.7,505.3 L258.8,504.1 L260.0,502.9 L261.1,501.6 L262.2,500.4 L263.3,499.1 L264.4,497.9
 L265.5,496.6 L266.7,495.3 L267.8,494.0 L268.9,492.8 L270.0,491.5 L271.1,490.2 L272.2,488.9 L273.4,487.6
 L274.5,486.3 L275.6,484.9 L276.7,483.6 L277.8,482.3 L278.9,481.0 L280.1,479.7 L281.2,478.4 L282.3,477.0
 L283.4,475.7 L284.5,474.4 L285.6,473.1 L286.8,471.8 L287.9,470.5 L289.0,469.1 L290.1,467.8 L291.2,466.5
 L292.3,465.2 L293.5,463.9 L294.6,462.6 L295.7,461.3 L296.8,460.0 L297.9,458.7 L299.0,457.4 L300.2,456.1
 L301.3,454.8 L302.4,453.5 L303.5,452.2 L304.6,450.9 L305.7,449.6 L306.8,448.4 L308.0,447.1 L309.1,445.9
 L310.2,444.6 L311.3,443.3 L312.4,442.1 L313.5,440.9 L314.7,439.6 L315.8,438.4 L316.9,437.2 L318.0,435.9
 L319.1,434.7 L320.2,433.5 L321.4,432.3 L322.5,431.1 L323.6,430.0 L324.7,428.8 L325.8,427.6 L326.9,426.4
 L328.1,425.3 L329.2,424.1 L330.3,423.0 L331.4,421.8 L332.5,420.7 L333.6,419.6 L334.8,418.5 L335.9,417.4
 L337.0,416.3 L338.1,415.2 L339.2,414.1 L340.3,413.0 L341.5,411.9 L342.6,410.9 L343.7,409.8 L344.8,408.8
 L345.9,407.7 L347.0,406.7 L348.2,405.7 L34

In [ ]:
// foreach(var grp in pltTab7.dataGroups) {
//     Plot2Ddata dat = new Plot2Ddata();
//     dat.AddDataGroup(grp);
//     string name = "plotData/AspectRatioOverTime/m4/" + pltTab7.Title + "/" + grp.Name + ".txt";
//     dat.SaveToTextFile(name);
// }

# Plotting of Spherical Harmonics

In [ ]:
bool skipEqualModes = false;
bool skipUnequalModes = true;
bool skipNonPoly = true; // false: plots all Legendgre functions, i.e. indices (*, *)
//                          true:  plots only Legendre polynomials, i.e. inices (*, 0)

In [ ]:
int[] m2cases = new int[] {0, 1, 2};
int[] m3cases = new int[] {3};
int[] m4cases = new int[] {4, 5, 6};

In [ ]:
List<Plot2Ddata>[] PlotTableSH = new List<Plot2Ddata>[caseS.Length];
for(int iRow = 0; iRow < caseS.Length; iRow++) { // loop over cases
for(int iGrd = 0; iGrd < gridS.Length; iGrd++) {
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
for(int iNew = 0; iNew < useNewton.Length; iNew++) {
  
    string _case = caseS[iRow];
    string _grd = gridS[iGrd];

    //ISessionInfo SI = null;
    var SIs = studySess.Where(sess => sess.Name.Contains(_case) 
                                      && sess.Name.Contains(_grd)
                                      && (sess.Name.Contains("_thirdOrderInit") == ShouldAnaInit[iAna])
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));
    
    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }
    
    if(SIs.IsNullOrEmpty()) 
        continue;

    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    IDictionary<string, IList<double>> tab = null;
    try {
        tab = SI.ReadTabulatedTextFileAsDoubles("SphericalHarmonics.txt", '\t');
        if (PlotTableSH[iRow] == null)
            PlotTableSH[iRow] = new List<Plot2Ddata>();
    } catch (Exception e) {
        Console.Error.WriteLine(e.Message);
        Console.WriteLine("Skipping " + SI);
        
        //var dummyPlot = new Plot2Ddata();
        //dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
        //PlotTableSH[iRow] = new List<Plot2Ddata>();
        continue;
    }


    var time = tab["time"];
    int cnt = 0;
    foreach(var column in tab) {
       
        var plot = new Plot2Ddata();

        var fmt = new PlotFormat();
        fmt.Style = Styles.Lines; 
        
        if(column.Key == "time")
            continue;
        if(skipNonPoly) {
            if(!column.Key.Contains(", 0)")) {
                continue;
            }
        }
        // if(skipEqualModes) {
        //     if(column.Key.Contains("(0, 0)") || column.Key.Contains("(2, 0)") 
        //     || column.Key.Contains("(4, 0)") || column.Key.Contains("(6, 0)") 
        //     || column.Key.Contains("(8, 0)") || column.Key.Contains("(10, 0)") 
        //     || column.Key.Contains("(12, 0)")) {
        //         continue;
        //     }
        // }
        if(!m3cases.Contains(iRow)) { 
            if(column.Key.Contains("(1, 0)") || column.Key.Contains("(3, 0)") 
            || column.Key.Contains("(5, 0)") || column.Key.Contains("(7, 0)") 
            || column.Key.Contains("(9, 0)") || column.Key.Contains("(11, 0)")) {
                continue;
            }
        }
            
        if (useNewton[iNew]) {
            fmt.LineColor = LineColors.Red;
            //fmt.DashType = DashTypes.Dashed;
        } else {  
            fmt.LineColor = LineColors.Blue;
            //fmt.DashType = DashTypes.Solid;
        }
        if (!ShouldAnaInit[iAna]) {
            fmt.DashType = DashTypes.Solid;
        } else {
            fmt.DashType = DashTypes.Dashed;
        }

        string name = column.Key;
        if(name.Contains(", 0)"))
            fmt.LineWidth = 3;
        // if(iAMR > 0)
        //     name = name + "-amr";
        if(ShouldAnaInit[iAna])
            name = name + "-3OrdInit";
        if(useNewton[iNew])
            name = name + "-Newton";

        plot.ShowLegend = true;

        if(name.Contains("(0, 0)")) {
            List<double> procVal = new List<double>();
            foreach(double val in column.Value) {
                procVal.Add(val-1);
            }
            plot.AddDataGroup(name, time, procVal, fmt);
        } else    
            plot.AddDataGroup(name, time, column.Value, fmt);

        plot.Ylabel = column.Key;
        plot.XrangeMin = 0;
        plot.XrangeMax = 2;

        if(PlotTableSH[iRow].Count > cnt) {
            PlotTableSH[iRow].ElementAt(cnt).AddDataGroup(plot.dataGroups[0]);
        } else {
            PlotTableSH[iRow].Add(plot);
        }
        cnt++;

   }
   
    
//    if(iAMR == 0) {
//        PlotTableSH[iRow,iCol] = plot;
//    } else {
//        PlotTableSH[iRow,iCol] = PlotTableSH[iRow,iCol].Merge(plot);
//    }
   
}
}
}
}

OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01
OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m4_Oh056_eta005
OD3D_J432k3_wallBC_amr1_m4_Oh056_eta005_thirdOrderInit
OD3D_J432k3_wallBC_amr1_m4_Oh056_eta005


In [ ]:
PlotTableSH[0]

index dataGroups LogX LogY LogBaseX LogBaseY LogX2 LogY2 XrangeMin XrangeMax YrangeMin YrangeMax Y2rangeMin Y2rangeMax X2rangeMin X2rangeMax Xlabel X2label Ylabel Y2label Title .. 0 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 0.5 <null> <null> <null> <null> <null> <null> <null> <null> (0, 0) <null> <null> 1 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 0.5 <null> <null> <null> <null> <null> <null> <null> <null> (2, 0) <null> <null> 2 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 0.5 <null> <null> <null> <null> <null> <null> <null> <null> (4, 0) <null> <null> 3 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 0.5 <null> <null> <null> <null> <null> <null> <null> <null> (6, 0) <null> <null> 4 [ BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues, BoSSS.Solution.Gnuplot.Plot2Ddata+XYvalues ] False False 10 10 False False 0 0.5 <null> <null> <null> <null> <null> <null> <null> <null> (8, 0) <null> <null>

### comparison data WNLT

In [ ]:
for(int iRow = 0; iRow < caseS.Length; iRow++) { // loop over cases
for(int i = 0; i < PlotTableSH[iRow].Count; i++) {
    //Console.WriteLine(PlotTableSH[iRow].ElementAt(i).dataGroups[0].Name);
    MultidimensionalArray dat = MultidimensionalArray.Create(1);
    if(m2cases.Contains(iRow)) {
        try {
            dat = IMatrixExtensions.LoadFromTextFile($"data/ModeDecomposition/m2/modeDecomp{2*i}_{caseS[iRow]}.txt");
        } catch {
            continue;
        }
    } 
    if(m3cases.Contains(iRow)) {
        try {
            dat = IMatrixExtensions.LoadFromTextFile($"data/ModeDecomposition/m3/modeDecomp{i}_{caseS[iRow]}.txt");
        } catch {
            continue;
        }
    } 
    if(m4cases.Contains(iRow)) {
        try {
            dat = IMatrixExtensions.LoadFromTextFile($"data/ModeDecomposition/m4/modeDecomp{2*i}_{caseS[iRow]}.txt");
        } catch {
            continue;
        }
    } 

    var fmt = new PlotFormat();
    fmt.Style = Styles.Lines; 
    fmt.LineWidth = 3;
    fmt.DashType = DashTypes.Dashed;
    fmt.LineColor = LineColors.Black;
    if(m3cases.Contains(iRow)) {
        PlotTableSH[iRow].ElementAt(i).AddDataGroup($"({i}, 0)-WNLT", dat.GetColumn(0), dat.GetColumn(1), fmt);
    } else {
        PlotTableSH[iRow].ElementAt(i).AddDataGroup($"({2*i}, 0)-WNLT", dat.GetColumn(0), dat.GetColumn(1), fmt);
    }
}
}

## Case 1 (m2_Oh01_eta04)

In [ ]:
var gp = PlotTableSH[0].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.04 
 
 
 
 
 -0.035 
 
 
 
 
 -0.03 
 
 
 
 
 -0.025 
 
 
 
 
 -0.02 
 
 
 
 
 -0.015 
 
 
 
 
 -0.01 
 
 
 
 
 -0.005 
 
 
 
 
 0 
 
 
 
 
 0.005 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0)-3OrdInit 
 
 
 
 
 (0, 0)-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M1492.8,60.1 L1546.2,60.1 M106.4,227.7 L110.5,228.6 L114.5,229.7 L118.6,230.7 L122.7,231.6 L126.7,232.5
 L130.8,233.3 L134.8,234.1 L138.9,234.8 L143.0,235.5 L147.0,236.1 L151.1,237.1 L155.2,237.5 L159.2,237.8
 L163.3,238.1 L167.4,238.3 L171.4,238.7 L175.5,239.0 L179.6,239.3 L183.6,239.5 L187.7,239.7 L191.7,239.8
 L195.8,240.0 L199.9,240.0 L203.9,240.4 L208.0,240.6 L212.1,240.6 L216.1,240.4 L220.2,240.6 L224.3,240.7
 L228.3,240.8 L232.4,240.6 L236.5,240.6 L240.5,240.5 L244.6,240.6 L248.6,240.6 L252.7,240.6 L256.8,240.6
 L260.8,240.6 L264.9,240.6 L269.0,240.7 L273.0,240.7 L277.1,240.7 L281.2,240.7 L285.2,240.7 L289.3,240.8
 L293.4,240.8 L297.4,241.0 L301.5,240.9 L305.5,241.0 L309.6,241.1 L313.7,241.1 L317.7,241.2 L321.8,241.3
 L325.9,241.2 L329.9,241.2 L334.0,241.2 L338.1,241.2 L342.1,241.3 L346.2,241.3 L350.3,241.4 L354.3,241.4
 L358.4,241.3 L362.4,241.3 L366.5,241.3 L370.6,241.3 L374.6,241.2 L378.7,241.2 L382.8,241.1 L386.8,240.9
 L390.9,240.8 L395.0,240.7 L399.0,240.5 L403.1,240.3 L407.2,240.1 L411.2,239.9 L415.3,239.7 L419.3,239.4
 L423.4,239.2 L427.5,238.9 L431.5,238.6 L435.6,238.2 L439.7,237.9 L443.7,237.5 L447.8,237.1 L451.9,236.7
 L455.9,236.3 L460.0,235.9 L464.1,235.5 L468.1,235.0 L472.2,234.6 L476.2,234.2 L480.3,233.7 L484.4,233.3
 L488.4,232.8 L492.5,232.3 L496.6,231.8 L500.6,231.3 L504.7,230.8 L508.8,230.3 L512.8,229.7 L516.9,229.2
 L521.0,228.6 L525.0,228.0 L529.1,227.4 L533.1,226.8 L537.2,226.2 L541.3,225.5 L545.3,224.8 L549.4,224.1
 L553.5,223.4 L557.5,222.7 L561.6,221.9 L565.7,221.1 L569.7,220.3 L573.8,219.4 L577.9,218.5 L581.9,217.6
 L586.0,216.7 L590.0,215.8 L594.1,214.7 L598.2,213.7 L602.2,212.7 L606.3,211.6 L610.4,210.6 L614.4,209.5
 L618.5,208.4 L622.6,207.3 L626.6,206.2 L630.7,205.0 L634.8,203.8 L638.8,202.6 L642.9,201.5 L646.9,200.3
 L651.0,199.1 L655.1,197.9 L659.1,196.7 L663.2,195.4 L667.3,194.2 L671.3,193.0 L675.4,191.8 L679.5,190.5
 L683.5,189.3 L687.6,188.0 L691.7,186.7 L695.7,185.4 L699.8,184.1 L703.8,182.7 L707.9,181.4 L712.0,180.1
 L716.0,178.8 L720.1,177.4 L724.2,176.0 L728.2,174.7 L732.3,173.3 L736.4,172.0 L740.4,170.5 L744.5,169.1
 L748.6,167.7 L752.6,166.3 L756.7,164.8 L760.7,163.4 L764.8,161.9 L768.9,160.4 L772.9,159.0 L777.0,157.4
 L781.1,155.9 L785.1,154.3 L789.2,152.8 L793.3,151.2 L797.3,149.5 L801.4,147.9 L805.5,146.3 L809.5,144.6
 L813.6,143.0 L817.6,141.4 L821.7,139.7 L825.8,137.9 L829.8,136.2 L833.9,134.4 L838.0,132.7 L842.0,130.9
 L846.1,129.1 L850.2,127.3 L854.2,125.6 L858.3,123.8 L862.4,121.9 L866.4,120.1 L870.5,118.2 L874.5,116.4
 L878.6,114.6 L882.7,112.7 L886.7,110.9 L890.8,109.1 L894.9,107.3 L898.9,105.5 L903.0,103.6 L907.1,101.9
 '/> 
 
 (0, 0) 
 
 
 (0, 0) 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M1492.8,84.1 L1546.2,84.1 M106.4,227.7 L110.5,227.7 L114.5,227.6 L118.6,227.5 L122.7,227.4 L126.7,227.2
 L130.8,226.9 L134.8,226.6 L138.9,226.2 L143.0,225.9 L147.0,225.4 L151.1,224.9 L155.2,224.4 L159.2,223.9
 L163.3,223.2 L167.4,222.6 L171.4,221.9 L175.5,221.2 L179.6,220.5 L183.6,219.7 L187.7,218.9 L191.7,218.0
 L195.8,217.1 L199.9,216.2 L203.9,215.3 L208.0,214.3 L212.1,213.3 L216.1,212.3 L220.2,211.3 L224.3,210.2
 L228.3,209.1 L232.4,208.0 L236.5,206.8 L240.5,205.7 L244.6,204.5 L248.6,203.3 L252.7,202.1 L256.8,2

In [ ]:
foreach(var lst in PlotTableSH[0]) {
foreach(var grp in lst.dataGroups) {
    Plot2Ddata dat = new Plot2Ddata();
    dat.AddDataGroup(grp);
    string name = "plotData/ModeDecomposition/case1/" + grp.Name + ".txt";
    dat.SaveToTextFile(name);
}
}

## Case 2 (m2_Oh01_eta02)

In [ ]:
var gp = PlotTableSH[1].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.009 
 
 
 
 
 -0.008 
 
 
 
 
 -0.007 
 
 
 
 
 -0.006 
 
 
 
 
 -0.005 
 
 
 
 
 -0.004 
 
 
 
 
 -0.003 
 
 
 
 
 -0.002 
 
 
 
 
 -0.001 
 
 
 
 
 0 
 
 
 
 
 0.001 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0)-3OrdInit 
 
 
 
 
 (0, 0)-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M1492.8,60.1 L1546.2,60.1 M106.4,242.3 L110.5,242.3 L114.5,242.3 L118.6,242.3 L122.7,242.2 L126.7,242.1
 L130.8,241.9 L134.8,241.6 L138.9,241.3 L143.0,241.0 L147.0,240.6 L151.1,240.1 L155.2,239.6 L159.2,239.0
 L163.3,238.3 L167.4,237.7 L171.4,237.0 L175.5,236.2 L179.6,235.4 L183.6,234.5 L187.7,233.6 L191.7,232.6
 L195.8,231.6 L199.9,230.6 L203.9,229.5 L208.0,228.4 L212.1,227.2 L216.1,226.0 L220.2,224.8 L224.3,223.5
 L228.3,222.2 L232.4,220.8 L236.5,219.4 L240.5,218.0 L244.6,216.6 L248.6,215.1 L252.7,213.6 L256.8,212.0
 L260.8,210.4 L264.9,208.8 L269.0,207.2 L273.0,205.5 L277.1,203.8 L281.2,202.1 L285.2,200.4 L289.3,198.7
 L293.4,196.9 L297.4,195.0 L301.5,193.3 L305.5,191.4 L309.6,189.6 L313.7,187.7 L317.7,185.8 L321.8,183.9
 L325.9,182.0 L329.9,180.1 L334.0,178.1 L338.1,176.2 L342.1,174.2 L346.2,172.2 L350.3,170.2 L354.3,168.2
 L358.4,166.2 L362.4,164.2 L366.5,162.2 L370.6,160.2 L374.6,158.2 L378.7,156.2 L382.8,154.2 L386.8,152.1
 L390.9,150.1 L395.0,148.1 L399.0,146.1 L403.1,144.1 L407.2,142.1 L411.2,140.1 L415.3,138.1 L419.3,136.1
 L423.4,134.1 L427.5,132.1 L431.5,130.2 L435.6,128.2 L439.7,126.2 L443.7,124.3 L447.8,122.4 L451.9,120.5
 L455.9,118.7 L460.0,116.7 L464.1,114.9 L468.1,113.1 L472.2,111.3 L476.2,109.5 L480.3,107.8 L484.4,106.0
 L488.4,104.3 L492.5,102.6 L496.6,101.0 L500.6,99.3 L504.7,97.7 L508.8,96.0 L512.8,94.5 L516.9,93.0
 L521.0,91.5 L525.0,90.0 L529.1,88.5 L533.1,87.1 L537.2,85.7 L541.3,84.4 L545.3,83.1 L549.4,81.7
 L553.5,80.5 L557.5,79.3 L561.6,78.1 L565.7,76.9 L569.7,75.9 L573.8,74.8 L577.9,73.8 L581.9,72.8
 L586.0,71.8 L590.0,70.9 L594.1,70.0 L598.2,69.1 L602.2,68.3 L606.3,67.5 L610.4,66.8 L614.4,66.1
 L618.5,65.5 L622.6,64.9 L626.6,64.3 L630.7,63.8 L634.8,63.3 L638.8,62.8 L642.9,62.4 L646.9,62.0
 L651.0,61.7 L655.1,61.4 L659.1,61.1 L663.2,60.9 L667.3,60.8 L671.3,60.6 L675.4,60.5 L679.5,60.5
 L683.5,60.5 L687.6,60.5 L691.7,60.6 L695.7,60.7 L699.8,60.8 L703.8,61.0 L707.9,61.2 L712.0,61.4
 L716.0,61.7 L720.1,62.0 L724.2,62.4 L728.2,62.8 L732.3,63.2 L736.4,63.6 L740.4,64.1 L744.5,64.6
 L748.6,65.2 L752.6,65.8 L756.7,66.4 L760.7,67.0 L764.8,67.7 L768.9,68.4 L772.9,69.1 L777.0,69.8
 L781.1,70.6 L785.1,71.4 L789.2,72.2 L793.3,73.0 L797.3,73.8 L801.4,74.7 L805.5,75.6 L809.5,76.5
 L813.6,77.3 L817.6,78.3 L821.7,79.2 L825.8,80.2 L829.8,81.1 L833.9,82.1 L838.0,83.1 L842.0,84.0
 L846.1,85.0 L850.2,86.0 L854.2,87.0 L858.3,88.1 L862.4,89.1 L866.4,90.1 L870.5,91.1 L874.5,92.1
 L878.6,93.1 L882.7,94.1 L886.7,95.1 L890.8,96.1 L894.9,97.1 L898.9,98.1 L903.0,99.1 L907.1,100.1
 L911.1,101.0 L915.2,102.0 L919.3,102.9 L923.3,103.9 L927.4,104.8 L931.4,105.7 L935.5,106.6 L939.6,107.5
 L943.6,108.3 L947.7,109.2 L951.8,110.0 L955.8,110.8 L959.9,111.6 L964.0,112.3 L968.0,112.9 L972.1,113.7
 L976.1,114.4 L980.2,115.1 L984.3,115.7 L988.3,116.4 L992.4,117.0 L996.5,117.6 L1000.5,118.2 L1004.6,118.7
 L1008.7,119.3 L1012.7,119.7 L1016.8,120.2 L1020.9,120.6 L1024.9,121.0 L1029.0,121.4 L1033.0,121.8 L1037.1,122.1
 L1041.2,122.4 L1045.2,122.7 L1049.3,122.9 L1053.4,123.1 L1057.4,123.3 L1061.5,123.5 L1065.6,123.6 L1069.6,123.7
 L1073.7,123.8 L1077.8,123.8 L1081.8,123.8 L1085.9,123.8 L1089.9,123.8 L1094.0,123.7 L1098.1,123.6 L1102.1,123.5
 L1

In [ ]:
foreach(var lst in PlotTableSH[1]) {
    foreach(var grp in lst.dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/ModeDecomposition/case2/" + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
    }

## Case 3 (m2_Oh01_eta01)

In [ ]:
var gp = PlotTableSH[2].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.0025 
 
 
 
 
 -0.002 
 
 
 
 
 -0.0015 
 
 
 
 
 -0.001 
 
 
 
 
 -0.0005 
 
 
 
 
 0 
 
 
 
 
 0.0005 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0)-3OrdInit 
 
 
 
 
 (0, 0)-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M1492.8,60.1 L1546.2,60.1 M114.7,225.5 L118.7,225.5 L122.8,225.4 L126.8,225.3 L130.9,225.2 L134.9,225.0
 L139.0,224.7 L143.0,224.4 L147.0,224.1 L151.1,223.7 L155.1,223.2 L159.2,222.7 L163.2,222.2 L167.3,221.6
 L171.3,221.0 L175.4,220.3 L179.4,219.5 L183.4,218.8 L187.5,218.0 L191.5,217.1 L195.6,216.2 L199.6,215.2
 L203.7,214.3 L207.7,213.2 L211.7,212.2 L215.8,211.1 L219.8,209.9 L223.9,208.8 L227.9,207.6 L232.0,206.3
 L236.0,205.0 L240.0,203.7 L244.1,202.4 L248.1,201.0 L252.2,199.6 L256.2,198.2 L260.3,196.7 L264.3,195.2
 L268.4,193.7 L272.4,192.2 L276.4,190.6 L280.5,189.0 L284.5,187.4 L288.6,185.8 L292.6,184.1 L296.7,182.5
 L300.7,180.7 L304.7,179.0 L308.8,177.3 L312.8,175.6 L316.9,173.9 L320.9,172.1 L325.0,170.4 L329.0,168.5
 L333.0,166.8 L337.1,165.0 L341.1,163.2 L345.2,161.4 L349.2,159.5 L353.3,157.8 L357.3,156.0 L361.4,154.2
 L365.4,152.3 L369.4,150.5 L373.5,148.7 L377.5,146.9 L381.6,145.1 L385.6,143.3 L389.7,141.5 L393.7,139.7
 L397.7,138.0 L401.8,136.2 L405.8,134.5 L409.9,132.7 L413.9,131.0 L418.0,129.3 L422.0,127.6 L426.0,125.9
 L430.1,124.2 L434.1,122.5 L438.2,120.9 L442.2,119.3 L446.3,117.6 L450.3,116.1 L454.4,114.5 L458.4,113.0
 L462.4,111.5 L466.5,110.0 L470.5,108.5 L474.6,107.1 L478.6,105.6 L482.7,104.2 L486.7,102.9 L490.7,101.5
 L494.8,100.2 L498.8,98.8 L502.9,97.7 L506.9,96.5 L511.0,95.3 L515.0,94.1 L519.1,93.0 L523.1,91.9
 L527.1,90.8 L531.2,89.8 L535.2,88.7 L539.3,87.8 L543.3,86.8 L547.4,85.9 L551.4,85.1 L555.4,84.2
 L559.5,83.4 L563.5,82.7 L567.6,81.9 L571.6,81.2 L575.7,80.6 L579.7,79.9 L583.7,79.4 L587.8,78.8
 L591.8,78.3 L595.9,77.8 L599.9,77.4 L604.0,77.0 L608.0,76.6 L612.1,76.2 L616.1,75.9 L620.1,75.7
 L624.2,75.4 L628.2,75.3 L632.3,75.1 L636.3,75.0 L640.4,74.9 L644.4,74.8 L648.4,74.8 L652.5,74.8
 L656.5,74.9 L660.6,74.9 L664.6,75.1 L668.7,75.2 L672.7,75.4 L676.7,75.6 L680.8,75.8 L684.8,76.1
 L688.9,76.4 L692.9,76.7 L697.0,77.1 L701.0,77.5 L705.1,77.9 L709.1,78.3 L713.1,78.8 L717.2,79.3
 L721.2,79.7 L725.3,80.3 L729.3,80.9 L733.4,81.5 L737.4,82.1 L741.4,82.7 L745.5,83.4 L749.5,84.1
 L753.6,84.7 L757.6,85.2 L761.7,86.0 L765.7,86.8 L769.7,87.6 L773.8,88.2 L777.8,89.0 L781.9,89.9
 L785.9,90.7 L790.0,91.5 L794.0,92.3 L798.1,93.2 L802.1,94.0 L806.1,94.9 L810.2,95.7 L814.2,96.6
 L818.3,97.5 L822.3,98.3 L826.4,99.2 L830.4,100.1 L834.4,101.0 L838.5,101.9 L842.5,102.7 L846.6,103.6
 L850.6,104.5 L854.7,105.4 L858.7,106.2 L862.7,107.1 L866.8,107.9 L870.8,108.8 L874.9,109.6 L878.9,110.5
 L883.0,111.3 L887.0,112.1 L891.1,112.9 L895.1,113.7 L899.1,114.5 L903.2,115.3 L907.2,116.1 L911.3,116.8
 L915.3,117.6 L919.4,118.3 L923.4,119.0 L927.4,119.7 L931.5,120.3 L935.5,121.0 L939.6,121.6 L943.6,122.3
 L947.7,122.9 L951.7,123.5 L955.7,124.0 L959.8,124.6 L963.8,125.1 L967.9,125.6 L971.9,126.1 L976.0,126.6
 L980.0,127.0 L984.1,127.4 L988.1,127.8 L992.1,128.2 L996.2,128.4 L1000.2,128.8 L1004.3,129.1 L1008.3,129.4
 L1012.4,129.7 L1016.4,129.9 L1020.4,130.1 L1024.5,130.3 L1028.5,130.5 L1032.6,130.6 L1036.6,130.8 L1040.7,130.9
 L1044.7,130.9 L1048.7,131.0 L1052.8,131.0 L1056.8,131.0 L1060.9,131.0 L1064.9,131.0 L1069.0,130.9 L1073.0,130.8
 L1077.1,130.7 L1081.1,130.6 L1085.1,130.4 L1089.2,130.2 L1093.2,130.0 L1097.3,129.8 L1101.3,129.6 L1105.4,129.3
 L1109.4,129.0 L1113.4,128.7 L1117.5,128.4 L1121.5,127.9 L1125.6,127.6 L1129.6,127.2 L11

## Case 4 (m3_Oh01_eta04)

In [ ]:
var gp = PlotTableSH[3].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.025 
 
 
 
 
 -0.02 
 
 
 
 
 -0.015 
 
 
 
 
 -0.01 
 
 
 
 
 -0.005 
 
 
 
 
 0 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0)-3OrdInit 
 
 
 
 
 (0, 0)-3OrdInit 
 
 
 
 
 
 (0, 0) 
 
 
 (0, 0) 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M1492.8,82.8 L1546.2,82.8 M106.4,121.2 L110.5,121.2 L114.5,121.1 L118.6,120.8 L122.7,120.4 L126.7,119.9
 L130.8,119.3 L134.8,118.5 L138.9,117.7 L143.0,116.8 L147.0,115.7 L151.1,114.6 L155.2,113.5 L159.2,112.2
 L163.3,110.9 L167.4,109.5 L171.4,108.1 L175.5,106.6 L179.6,105.1 L183.6,103.5 L187.7,102.0 L191.7,100.3
 L195.8,98.7 L199.9,97.1 L203.9,95.4 L208.0,93.7 L212.1,92.1 L216.1,90.4 L220.2,88.7 L224.3,87.0
 L228.3,85.4 L232.4,83.7 L236.5,82.0 L240.5,80.4 L244.6,78.7 L248.6,77.1 L252.7,75.5 L256.8,73.9
 L260.8,72.3 L264.9,70.8 L269.0,69.3 L273.0,67.8 L277.1,66.4 L281.2,64.9 L285.2,63.6 L289.3,62.2
 L293.4,60.9 L297.4,59.6 L301.5,58.4 L305.5,57.2 L309.6,56.1 L313.7,55.0 L317.7,53.9 L321.8,52.9
 L325.9,52.0 L329.9,51.0 L334.0,50.1 L338.1,49.3 L342.1,48.5 L346.2,47.8 L350.3,47.1 L354.3,46.5
 L358.4,45.8 L362.4,45.3 L366.5,44.8 L370.6,44.3 L374.6,43.8 L378.7,43.4 L382.8,43.1 L386.8,42.8
 L390.9,42.5 L395.0,42.3 L399.0,42.1 L403.1,41.9 L407.2,41.8 L411.2,41.8 L415.3,41.7 L419.3,41.7
 L423.4,41.8 L427.5,41.8 L431.5,41.9 L435.6,42.0 L439.7,42.2 L443.7,42.4 L447.8,42.6 L451.9,42.8
 L455.9,43.1 L460.0,43.4 L464.1,43.7 L468.1,44.0 L472.2,44.4 L476.2,44.8 L480.3,45.2 L484.4,45.6
 L488.4,46.0 L492.5,46.5 L496.6,46.9 L500.6,47.4 L504.7,47.9 L508.8,48.4 L512.8,48.9 L516.9,49.4
 L521.0,49.9 L525.0,50.4 L529.1,50.9 L533.1,51.4 L537.2,51.9 L541.3,52.3 L545.3,52.8 L549.4,53.3
 L553.5,53.8 L557.5,54.2 L561.6,54.7 L565.7,55.1 L569.7,55.5 L573.8,55.9 L577.9,56.3 L581.9,56.6
 L586.0,56.9 L590.0,57.3 L594.1,57.6 L598.2,57.8 L602.2,58.1 L606.3,58.4 L610.4,58.6 L614.4,58.8
 L618.5,58.9 L622.6,59.0 L626.6,59.2 L630.7,59.3 L634.8,59.3 L638.8,59.4 L642.9,59.4 L646.9,59.4
 L651.0,59.3 L655.1,59.2 L659.1,59.2 L663.2,59.1 L667.3,58.9 L671.3,58.8 L675.4,58.7 L679.5,58.5
 L683.5,58.3 L687.6,58.1 L691.7,57.8 L695.7,57.5 L699.8,57.3 L703.8,57.0 L707.9,56.7 L712.0,56.4
 L716.0,56.0 L720.1,55.7 L724.2,55.3 L728.2,55.0 L732.3,54.6 L736.4,54.2 L740.4,53.8 L744.5,53.4
 L748.6,53.0 L752.6,52.6 L756.7,52.2 L760.7,51.8 L764.8,51.4 L768.9,51.0 L772.9,50.6 L777.0,50.2
 L781.1,49.7 L785.1,49.3 L789.2,48.9 L793.3,48.5 L797.3,48.1 L801.4,47.7 L805.5,47.3 L809.5,46.9
 L813.6,46.5 L817.6,46.1 L821.7,45.7 L825.8,45.4 L829.8,45.0 L833.9,44.7 L838.0,44.3 L842.0,44.0
 L846.1,43.6 L850.2,43.3 L854.2,43.0 L858.3,42.7 L862.4,42.5 L866.4,42.2 L870.5,41.9 L874.5,41.7
 L878.6,41.5 L882.7,41.3 L886.7,41.1 L890.8,40.8 L894.9,40.6 L898.9,40.5 L903.0,40.3 L907.1,40.2
 L911.1,40.0 L915.2,39.9 L919.3,39.9 L923.3,39.8 L927.4,39.8 L931.4,39.7 L935.5,39.7 L939.6,39.6
 L943.6,39.6 L947.7,39.6 L951.8,39.6 L955.8,39.6 L959.9,39.6 L964.0,39.6 L968.0,39.6 L972.1,39.7
 L976.1,39.7 L980.2,39.8 L984.3,39.8 L988.3,39.9 L992.4,40.0 L996.5,40.1 L1000.5,40.2 L1004.6,40.3
 L1008.7,40.4 L1012.7,40.5 L1016.8,40.5 L1020.9,40.6 L1024.9,40.8 L1029.0,40.9 L1033.0,41.0 L1037.1,41.1
 L1041.2,41.2 L1045.2,41.4 L1049.3,41.5 L1053.4,41.6 L1057.4,41.7 L1061.5,41.8 L1065.6,41.9 L1069.6,42.0
 L1073.7,42.2 L1077.8,42.3 L1081.8,42.4 L1085.9,42.5 L1089.9,42.6 L1094.0,42.7 L1098.1,42.8 L1102.1,42.9
 L1106.2,43.0 L1110.3,43.0 L1114.3,43.1 L1118.4,43.2 L1122.5,43.2 L1126.5,43.3 L1130.6,43.3 L1134.7,43.4
 L1138.7,43.4 L1142.8,43.5 L1146.8,43.5 L1150.9,43.5 L1155.0,43.5 L1159.0,43.5 L1163.1,43.5 L1167.2,43.5
 L1171.2,43.5 L1175.3,43.5 L1179.4,43.5 L1183.4,4

In [ ]:
foreach(var lst in PlotTableSH[3]) {
    foreach(var grp in lst.dataGroups) {
        Plot2Ddata dat = new Plot2Ddata();
        dat.AddDataGroup(grp);
        string name = "plotData/ModeDecomposition/case4/" + grp.Name + ".txt";
        dat.SaveToTextFile(name);
    }
    }

## Case 5 (m4_Oh01_eta04)

In [ ]:
var gp = PlotTableSH[4].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.02 
 
 
 
 
 -0.018 
 
 
 
 
 -0.016 
 
 
 
 
 -0.014 
 
 
 
 
 -0.012 
 
 
 
 
 -0.01 
 
 
 
 
 -0.008 
 
 
 
 
 -0.006 
 
 
 
 
 -0.004 
 
 
 
 
 -0.002 
 
 
 
 
 0 
 
 
 
 
 0.002 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0)-3OrdInit 
 
 
 
 
 (0, 0)-3OrdInit 
 
 
 
 
 
 (0, 0) 
 
 
 (0, 0) 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M1492.8,84.1 L1546.2,84.1 M106.4,242.6 L110.5,242.6 L114.5,242.2 L118.6,241.5 L122.7,240.3 L126.7,238.7
 L130.8,236.8 L134.8,234.6 L138.9,232.0 L143.0,229.1 L147.0,225.8 L151.1,222.4 L155.2,218.5 L159.2,214.6
 L163.3,210.5 L167.4,206.1 L171.4,201.6 L175.5,197.0 L179.6,192.3 L183.6,187.4 L187.7,182.5 L191.7,177.6
 L195.8,172.5 L199.9,167.5 L203.9,162.5 L208.0,157.6 L212.1,152.6 L216.1,147.7 L220.2,142.9 L224.3,138.2
 L228.3,133.5 L232.4,129.0 L236.5,124.6 L240.5,120.3 L244.6,116.2 L248.6,112.3 L252.7,108.5 L256.8,105.0
 L260.8,101.6 L264.9,98.6 L269.0,95.6 L273.0,92.7 L277.1,90.0 L281.2,87.4 L285.2,85.1 L289.3,82.9
 L293.4,80.9 L297.4,79.0 L301.5,77.3 L305.5,75.7 L309.6,74.4 L313.7,73.2 L317.7,72.3 L321.8,71.6
 L325.9,71.0 L329.9,70.5 L334.0,70.4 L338.1,70.6 L342.1,70.8 L346.2,70.9 L350.3,71.3 L354.3,71.9
 L358.4,72.5 L362.4,73.2 L366.5,74.0 L370.6,74.9 L374.6,76.0 L378.7,77.2 L382.8,78.4 L386.8,79.7
 L390.9,81.2 L395.0,82.6 L399.0,84.0 L403.1,85.5 L407.2,87.0 L411.2,88.5 L415.3,90.0 L419.3,91.4
 L423.4,92.6 L427.5,94.1 L431.5,95.4 L435.6,96.7 L439.7,97.6 L443.7,98.7 L447.8,99.9 L451.9,100.8
 L455.9,101.7 L460.0,102.4 L464.1,103.1 L468.1,103.6 L472.2,104.0 L476.2,104.3 L480.3,104.5 L484.4,104.7
 L488.4,104.7 L492.5,104.6 L496.6,104.4 L500.6,104.1 L504.7,103.7 L508.8,103.2 L512.8,102.7 L516.9,102.1
 L521.0,101.4 L525.0,100.0 L529.1,99.4 L533.1,98.6 L537.2,97.5 L541.3,96.7 L545.3,95.8 L549.4,94.9
 L553.5,93.9 L557.5,92.8 L561.6,91.8 L565.7,90.6 L569.7,89.6 L573.8,88.7 L577.9,87.6 L581.9,86.6
 L586.0,85.6 L590.0,84.7 L594.1,83.7 L598.2,82.8 L602.2,81.9 L606.3,81.0 L610.4,80.2 L614.4,79.3
 L618.5,78.6 L622.6,77.8 L626.6,77.0 L630.7,76.3 L634.8,75.7 L638.8,75.1 L642.9,74.6 L646.9,74.0
 L651.0,73.6 L655.1,73.2 L659.1,72.8 L663.2,72.4 L667.3,72.1 L671.3,71.9 L675.4,71.4 L679.5,71.3
 L683.5,71.2 L687.6,71.1 L691.7,71.0 L695.7,71.0 L699.8,70.9 L703.8,70.9 L707.9,70.8 L712.0,70.8
 L716.0,70.9 L720.1,70.9 L724.2,71.0 L728.2,71.0 L732.3,71.1 L736.4,71.2 L740.4,71.3 L744.5,71.5
 L748.6,71.5 L752.6,71.7 L756.7,71.7 L760.7,71.8 L764.8,71.8 L768.9,71.9 L772.9,71.9 L777.0,72.0
 L781.1,72.0 L785.1,72.0 L789.2,72.0 L793.3,72.0 L797.3,72.0 L801.4,72.0 L805.5,71.9 L809.5,71.8
 L813.6,71.7 L817.6,71.6 L821.7,71.5 L825.8,71.4 L829.8,71.2 L833.9,71.0 L838.0,70.8 L842.0,70.6
 L846.1,70.4 L850.2,70.2 L854.2,69.9 L858.3,69.6 L862.4,69.4 L866.4,69.1 L870.5,68.8 L874.5,68.5
 L878.6,68.2 L882.7,67.9 L886.7,67.5 L890.8,67.2 L894.9,66.9 L898.9,66.9 L903.0,66.5 L907.1,66.1
 L911.1,65.7 L915.2,65.4 L919.3,65.0 L923.3,64.9 L927.4,64.5 L931.4,64.1 L935.5,63.8 L939.6,63.5
 L943.6,63.2 L947.7,62.9 L951.8,62.7 L955.8,62.4 L959.9,62.2 L964.0,62.1 L968.0,61.8 L972.1,61.6
 L976.1,61.4 L980.2,61.2 L984.3,61.0 L988.3,60.9 L992.4,60.7 L996.5,60.6 L1000.5,60.5 L1004.6,60.4
 L1008.7,60.3 L1012.7,60.2 L1016.8,60.1 L1020.9,60.1 L1024.9,60.0 L1029.0,60.0 L1033.0,59.9 L1037.1,59.9
 L1041.2,59.9 L1045.2,59.9 L1049.3,60.1 L1053.4,60.1 L1057.4,60.1 L1061.5,60.3 L1065.6,60.4 L1069.6,60.5
 L1073.7,60.5 L1077.8,60.6 L1081.8,60.6 L1085.9,60.7 L1089.9,60.7 L1094.0,60.8 L1098.1,60.8 L1102.1,60.9
 L1106.2,61.0 L1110.3,60.9 L1114.3,61.0 L1118.4,61.1 L1122.5,6

In [ ]:
// foreach(var lst in PlotTableSH[4]) {
//     foreach(var grp in lst.dataGroups) {
//         Plot2Ddata dat = new Plot2Ddata();
//         dat.AddDataGroup(grp);
//         string name = "plotData/ModeDecomposition/m4/" + grp.Name + ".txt";
//         dat.SaveToTextFile(name);
//     }
//     }

## Case 6 (m4_Oh01_eta01)

In [ ]:
var gp = PlotTableSH[5].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.0012 
 
 
 
 
 -0.001 
 
 
 
 
 -0.0008 
 
 
 
 
 -0.0006 
 
 
 
 
 -0.0004 
 
 
 
 
 -0.0002 
 
 
 
 
 0 
 
 
 
 
 0.0002 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0)-3OrdInit 
 
 
 
 
 (0, 0)-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M1492.8,60.1 L1546.2,60.1 M114.7,248.1 L118.7,248.5 L122.8,248.0 L126.8,247.1 L130.9,245.7 L134.9,243.8
 L139.0,241.4 L143.0,238.6 L147.0,235.4 L151.1,231.8 L155.1,227.9 L159.2,223.6 L163.2,219.0 L167.3,214.1
 L171.3,208.8 L175.4,203.5 L179.4,198.0 L183.4,192.3 L187.5,186.5 L191.5,180.7 L195.6,174.7 L199.6,168.7
 L203.7,162.7 L207.7,156.8 L211.7,150.9 L215.8,145.1 L219.8,139.4 L223.9,133.5 L227.9,128.2 L232.0,123.0
 L236.0,118.0 L240.0,113.2 L244.1,108.6 L248.1,104.3 L252.2,100.2 L256.2,96.3 L260.3,92.8 L264.3,89.5
 L268.4,86.4 L272.4,83.7 L276.4,81.2 L280.5,79.1 L284.5,77.2 L288.6,75.6 L292.6,74.3 L296.7,73.3
 L300.7,72.5 L304.7,72.0 L308.8,71.8 L312.8,71.8 L316.9,72.0 L320.9,72.4 L325.0,72.6 L329.0,73.7
 L333.0,74.8 L337.1,75.6 L341.1,77.2 L345.2,78.7 L349.2,80.2 L353.3,81.9 L357.3,83.6 L361.4,85.4
 L365.4,87.2 L369.4,89.0 L373.5,90.8 L377.5,92.6 L381.6,93.7 L385.6,95.7 L389.7,97.5 L393.7,99.1
 L397.7,100.7 L401.8,102.2 L405.8,103.6 L409.9,104.9 L413.9,106.1 L418.0,107.1 L422.0,108.1 L426.0,108.9
 L430.1,109.5 L434.1,110.1 L438.2,110.4 L442.2,110.7 L446.3,110.9 L450.3,110.9 L454.4,110.7 L458.4,110.5
 L462.4,110.1 L466.5,109.6 L470.5,109.0 L474.6,108.4 L478.6,107.6 L482.7,106.7 L486.7,105.7 L490.7,104.7
 L494.8,103.6 L498.8,102.4 L502.9,101.2 L506.9,100.0 L511.0,98.7 L515.0,97.4 L519.1,96.1 L523.1,94.7
 L527.1,93.0 L531.2,91.9 L535.2,90.7 L539.3,89.4 L543.3,88.1 L547.4,86.9 L551.4,85.7 L555.4,84.5
 L559.5,83.4 L563.5,82.3 L567.6,81.3 L571.6,80.3 L575.7,79.4 L579.7,78.5 L583.7,77.8 L587.8,77.0
 L591.8,76.4 L595.9,75.5 L599.9,75.0 L604.0,74.5 L608.0,74.1 L612.1,73.8 L616.1,73.5 L620.1,73.3
 L624.2,73.1 L628.2,73.1 L632.3,73.0 L636.3,73.0 L640.4,73.1 L644.4,73.2 L648.4,73.4 L652.5,73.6
 L656.5,73.8 L660.6,74.1 L664.6,74.4 L668.7,74.7 L672.7,75.0 L676.7,75.4 L680.8,75.8 L684.8,76.2
 L688.9,76.6 L692.9,77.0 L697.0,77.3 L701.0,77.7 L705.1,78.1 L709.1,78.5 L713.1,78.9 L717.2,79.2
 L721.2,79.5 L725.3,79.8 L729.3,80.1 L733.4,80.4 L737.4,80.6 L741.4,80.9 L745.5,81.1 L749.5,81.2
 L753.6,81.3 L757.6,81.5 L761.7,81.5 L765.7,81.6 L769.7,81.6 L773.8,81.6 L777.8,81.6 L781.9,81.5
 L785.9,81.4 L790.0,81.3 L794.0,81.1 L798.1,81.0 L802.1,80.8 L806.1,80.6 L810.2,80.4 L814.2,80.1
 L818.3,79.9 L822.3,79.6 L826.4,79.3 L830.4,79.0 L834.4,78.7 L838.5,78.4 L842.5,78.1 L846.6,77.8
 L850.6,77.5 L854.7,77.1 L858.7,76.8 L862.7,76.5 L866.8,76.2 L870.8,75.9 L874.9,75.6 L878.9,75.3
 L883.0,75.0 L887.0,74.7 L891.1,74.5 L895.1,74.2 L899.1,74.0 L903.2,73.8 L907.2,73.6 L911.3,73.4
 L915.3,73.2 L919.4,73.0 L923.4,72.9 L927.4,72.8 L931.5,72.7 L935.5,72.6 L939.6,72.5 L943.6,72.4
 L947.7,72.3 L951.7,72.3 L955.7,72.3 L959.8,72.3 L963.8,72.3 L967.9,72.3 L971.9,72.3 L976.0,72.3
 L980.0,72.4 L984.1,72.4 L988.1,72.5 L992.1,72.5 L996.2,72.6 L1000.2,72.7 L1004.3,72.8 L1008.3,72.9
 L1012.4,72.9 L1016.4,73.0 L1020.4,73.1 L1024.5,73.2 L1028.5,73.3 L1032.6,73.4 L1036.6,73.5 L1040.7,73.6
 L1044.7,73.6 L1048.7,73.7 L1052.8,73.8 L1056.8,73.6 L1060.9,73.7 L1064.9,73.9 L1069.0,74.0 L1073.0,74.1
 L1077.1,74.1 L1081.1,74.2 L1085.1,74.2 L1089.2,74.2 L1093.2,74.3 L1097.3,74.3 L1101.3,74.3 L1105.4,74.3
 L1109.4,74.3 L1113.4,74.2 L1117.5,74.2 L1121.5,74.2 L1125.6,74.2 L1129.6,73.9 L1133.7,73.8 L1137.7,73.9
 L1141.7,73.9 L1145.8,73.8 L1149.8,73.8 L1153.9,73.7 L1

In [ ]:
// foreach(var lst in PlotTableSH[5]) {
//     foreach(var grp in lst.dataGroups) {
//         Plot2Ddata dat = new Plot2Ddata();
//         dat.AddDataGroup(grp);
//         string name = "plotData/ModeDecomposition/m4/" + grp.Name + ".txt";
//         dat.SaveToTextFile(name);
//     }
//     }

## Case 7 (m4_Oh056_eta005)

In [ ]:
var gp = PlotTableSH[6].ToGnuplot();
gp.PlotSVG(xRes:1800,yRes:1500)

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.0003 
 
 
 
 
 -0.00025 
 
 
 
 
 -0.0002 
 
 
 
 
 -0.00015 
 
 
 
 
 -0.0001 
 
 
 
 
 -5x10 -5 
 
 
 
 
 0 
 
 
 
 
 5x10 -5 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 (0, 0) 
 
 
 
 
 (0, 0)-3OrdInit 
 
 
 
 
 (0, 0)-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M1492.8,60.1 L1546.2,60.1 M123.0,247.9 L127.0,249.1 L131.0,249.4 L135.1,249.4 L139.1,249.1 L143.1,248.5
 L147.1,247.5 L151.2,246.3 L155.2,244.8 L159.2,243.1 L163.2,241.2 L167.3,239.2 L171.3,236.9 L175.3,234.5
 L179.3,232.0 L183.3,229.4 L187.4,226.8 L191.4,224.1 L195.4,221.3 L199.4,218.5 L203.5,215.7 L207.5,213.0
 L211.5,210.2 L215.5,207.4 L219.5,204.7 L223.6,202.1 L227.6,199.5 L231.6,196.9 L235.6,194.4 L239.7,192.0
 L243.7,189.7 L247.7,187.4 L251.7,185.2 L255.8,183.1 L259.8,181.1 L263.8,179.2 L267.8,177.4 L271.8,175.6
 L275.9,173.9 L279.9,172.4 L283.9,170.9 L287.9,169.4 L292.0,168.1 L296.0,166.8 L300.0,165.6 L304.0,164.5
 L308.0,163.5 L312.1,162.5 L316.1,161.6 L320.1,160.8 L324.1,160.0 L328.2,159.2 L332.2,158.5 L336.2,157.9
 L340.2,157.3 L344.3,156.8 L348.3,156.3 L352.3,155.9 L356.3,155.4 L360.3,155.1 L364.4,154.7 L368.4,154.4
 L372.4,154.1 L376.4,153.8 L380.5,153.6 L384.5,153.4 L388.5,153.1 L392.5,153.0 L396.5,152.8 L400.6,152.6
 L404.6,152.4 L408.6,152.3 L412.6,152.2 L416.7,152.0 L420.7,151.9 L424.7,151.7 L428.7,151.6 L432.8,151.5
 L436.8,151.3 L440.8,151.2 L444.8,151.0 L448.8,150.9 L452.9,150.7 L456.9,150.6 L460.9,150.4 L464.9,150.2
 L469.0,150.0 L473.0,149.8 L477.0,149.6 L481.0,149.4 L485.0,149.2 L489.1,149.0 L493.1,148.7 L497.1,148.5
 L501.1,148.2 L505.2,147.9 L509.2,147.6 L513.2,147.3 L517.2,147.0 L521.3,146.7 L525.3,146.4 L529.3,146.0
 L533.3,145.6 L537.3,145.3 L541.4,144.3 L545.4,144.1 L549.4,143.4 L553.4,143.2 L557.5,142.8 L561.5,142.4
 L565.5,142.0 L569.5,141.5 L573.5,141.1 L577.6,140.6 L581.6,140.1 L585.6,139.6 L589.6,139.1 L593.7,138.6
 L597.7,138.0 L601.7,137.5 L605.7,136.9 L609.8,136.4 L613.8,135.8 L617.8,135.2 L621.8,134.7 L625.8,134.1
 L629.9,133.5 L633.9,132.9 L637.9,132.3 L641.9,131.3 L646.0,131.0 L650.0,130.2 L654.0,129.9 L658.0,129.3
 L662.0,128.7 L666.1,128.1 L670.1,127.4 L674.1,126.8 L678.1,126.2 L682.2,125.5 L686.2,124.9 L690.2,124.2
 L694.2,123.6 L698.3,123.0 L702.3,122.3 L706.3,121.7 L710.3,121.0 L714.3,120.4 L718.4,119.7 L722.4,119.1
 L726.4,118.4 L730.4,117.8 L734.5,117.2 L738.5,116.5 L742.5,115.9 L746.5,115.3 L750.5,114.6 L754.6,114.0
 L758.6,113.4 L762.6,112.8 L766.6,112.2 L770.7,111.5 L774.7,110.9 L778.7,110.3 L782.7,109.7 L786.8,109.1
 L790.8,108.5 L794.8,108.0 L798.8,107.4 L802.8,106.8 L806.9,106.2 L810.9,105.7 L814.9,105.1 L818.9,104.6
 L823.0,104.0 L827.0,103.5 L831.0,102.9 L835.0,102.4 L839.0,101.9 L843.1,101.3 L847.1,100.8 L851.1,100.3
 L855.1,99.8 L859.2,99.3 L863.2,98.8 L867.2,98.4 L871.2,97.9 L875.3,97.4 L879.3,96.9 L883.3,96.5
 L887.3,96.0 L891.3,95.6 L895.4,95.1 L899.4,94.7 L903.4,94.3 L907.4,93.9 L911.5,93.5 L915.5,93.0
 L919.5,92.6 L923.5,92.3 L927.6,91.9 L931.6,91.5 L935.6,91.1 L939.6,90.7 L943.6,90.4 L947.7,90.0
 L951.7,89.7 L955.7,89.3 L959.7,89.0 L963.8,88.7 L967.8,88.3 L971.8,88.0 L975.8,87.7 L979.8,87.4
 L983.9,87.1 L987.9,86.8 L991.9,86.5 L995.9,86.2 L1000.0,85.9 L1004.0,85.6 L1008.0,85.4 L1012.0,85.1
 L1016.1,84.9 L1020.1,84.6 L1024.1,84.4 L1028.1,84.1 L1032.1,83.9 L1036.2,83.6 L1040.2,83.4 L1044.2,83.2
 L1048.2,83.0 L1052.3,82.8 L1056.3,82.6 L1060.3,82.4 L1064.3,82.2 L1068.3,82.0 L1072.4,81.8 L1076.4,81.6
 L1080.4,81.4 L1084.4,81.2 L1088.5,81.1 L1092.5,80.9 L1096.5,80.7 L1100.5,80.6 L1104.6,80.4 L1108.6,80.3
 L1112.6,80.1 L1116.6,80.0 L1120.6,

In [ ]:
// foreach(var lst in PlotTableSH[6]) {
//     foreach(var grp in lst.dataGroups) {
//         Plot2Ddata dat = new Plot2Ddata();
//         dat.AddDataGroup(grp);
//         string name = "plotData/ModeDecomposition/m4/" + grp.Name + ".txt";
//         dat.SaveToTextFile(name);
//     }
//     }

### All-in-one plot

In [48]:
// for(int i = 0; i < PlotTableSH.GetLength(0); i++) {
//    PlotTableSH[i,0].XrangeMin = 0;
//    PlotTableSH[i,0].XrangeMax = 2.5;
// }

In [ ]:
// var gp = PlotTableSH.ToGnuplot();
// gp.PlotSVG(xRes:1800,yRes:1500)

In [ ]:
//Note: you must create sub-directory "latex" first!
//var cl = gp.PlotCairolatex(xSize:32,ySize:18);
//cl.WriteMinimalCompileableExample("latex\\plot.tex");

In [ ]:
foreach(var S in wmg.Sessions) {
    //var EI = S.Export().WithSupersampling(1).Do();
}

# Plotting energies

In [ ]:
string[] energyKeys = new string[] {"kineticEnergy", "surfaceEnergy", "totalEnergy"}; //, "kineticDissipation"};
double surfEnrgy0 = Math.PI;    // surface energy for a sphere with r = 1 and sigma = 1 on a quarter domain

In [ ]:
Plot2Ddata[,] PlotTableEnrgy = new Plot2Ddata[caseS.Length, 2];
for(int iRow = 0; iRow < PlotTableEnrgy.GetLength(0); iRow++) { // loop over cases
for(int iCol = 0; iCol < PlotTableEnrgy.GetLength(1); iCol++) {
for(int iGrd = 0; iGrd < gridS.Length; iGrd++) {
for(int iAna = 0; iAna < ShouldAnaInit.Length; iAna++) {
for(int iNew = 0; iNew < useNewton.Length; iNew++) {
   
    string _case = caseS[iRow];
    string _grd = gridS[iGrd];

    //ISessionInfo SI = null;
    var SIs = wmg.Sessions.Where(sess => sess.Name.Contains(_case) 
                                      && sess.Name.Contains(_grd)
                                      && (sess.Name.Contains("_thirdOrderInit") == ShouldAnaInit[iAna])
                                      && (sess.Name.Contains("_Newton") == useNewton[iNew]));
    
    if(SIs.Count() > 1) {
        foreach(var s in SIs)
            Console.WriteLine(s);
    }

    if(SIs.IsNullOrEmpty()) 
    continue;
    
    ISessionInfo SI = SIs.Single();
    if(SI == null)
        continue;
    Console.WriteLine(SI.Name);

    IDictionary<string, IList<double>> tab = null;
    try {
        tab = SI.ReadTabulatedTextFileAsDoubles("EnergyLogValues.txt", '\t');
    } catch (Exception e) {
        Console.Error.WriteLine(e.Message);
        Console.WriteLine("Skipping " + SI);
        
        // var dummyPlot = new Plot2Ddata();
        // dummyPlot.AddDataGroup("empty", new double[]{ 0, 1 }, new double[] { 0, 1 });
        // PlotTableEnrgy[iRow,iCol] = dummyPlot;
        continue;
    }
    
    var plot = new Plot2Ddata();
    var time = tab["time"];

    if(iCol == 0) {
        {
            var fmt = new PlotFormat();
            fmt.Style = Styles.Lines; 
            fmt.LineWidth = 3;
            fmt.LineColor = LineColors.Blue;
            if (ShouldAnaInit[iAna]) 
                fmt.DashType = DashTypes.Dashed;
            else  
                fmt.DashType = DashTypes.Solid;          

            string name = "surfaceEnergy"+_grd;
            // if(iAMR > 0)
            //     name = name + "-amr";
            if(ShouldAnaInit[iAna])
                name = name + "-3OrdInit";
            if(useNewton[iNew])
                name = name + "-Newton";

            List<double> procSurfEnrgy = new List<double>();    
            foreach(double sEnrgyVal in tab["surfaceEnergy"]) {
                procSurfEnrgy.Add(sEnrgyVal - surfEnrgy0);
            }    
            
            plot.AddDataGroup(name, time, procSurfEnrgy, fmt);
        }
        {
            var fmt = new PlotFormat();
            fmt.Style = Styles.Lines; 
            fmt.LineWidth = 3;
            fmt.LineColor = LineColors.Red;
            if (ShouldAnaInit[iAna]) 
                fmt.DashType = DashTypes.Dashed;
            else  
                fmt.DashType = DashTypes.Solid;          

            string name = "kineticEnergy"+_grd;
            // if(iAMR > 0)
            //     name = name + "-amr";
            if(ShouldAnaInit[iAna])
                name = name + "-3OrdInit";
            if(useNewton[iNew])
                name = name + "-Newton";
            
            plot.AddDataGroup(name, time, tab["kineticEnergy"], fmt);
        }
    }

    if(iCol == 1) {

        var fmt = new PlotFormat();
        fmt.Style = Styles.Lines; 
        fmt.LineWidth = 3;
        fmt.LineColor = LineColors.Blue;
        if (ShouldAnaInit[iAna]) 
            fmt.DashType = DashTypes.Dashed;
        else  
            fmt.DashType = DashTypes.Solid;

        string name = "totalEnergy"+_grd;
        // if(iAMR > 0)
        //     name = name + "-amr";
        if(ShouldAnaInit[iAna])
            name = name + "-3OrdInit";
        if(useNewton[iNew])
            name = name + "-Newton";
        
        plot.AddDataGroup(name, time, tab["totalEnergy"], fmt);
    }
    
   
    //plot.ShowLegend = iCol == 2 && iRow == 2;
    plot.ShowLegend = true;
    
    //plot.Title = SI.Name;
        
    if(PlotTableEnrgy[iRow,iCol]==null) {
        PlotTableEnrgy[iRow,iCol] = plot;
    } else {
        PlotTableEnrgy[iRow,iCol] = PlotTableEnrgy[iRow,iCol].Merge(plot);
    }
        
    PlotTableEnrgy[iRow,iCol].XrangeMin = 0;
    PlotTableEnrgy[iRow,iCol].XrangeMax = 2;
}
}
}
}
}

OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta04
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta02
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m2_Oh01_eta01
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m3_Oh01_eta04
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta04
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01_thirdOrderInit
OD3D_J686k3_wallBC_amr1_m4_Oh01_eta01

## Case1 (m2_Oh01_eta04)

In [ ]:
var gp = PlotTableEnrgy[0,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.2 
 
 
 
 
 0.4 
 
 
 
 
 0.6 
 
 
 
 
 0.8 
 
 
 
 
 1 
 
 
 
 
 1.2 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergy-3OrdInit 
 
 
 
 
 surfaceEnergy-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M433.1,57.1 L486.5,57.1 M53.9,485.1 L55.6,483.4 L57.4,479.7 L59.1,474.7 L60.9,468.9 L62.6,463.4
 L64.3,458.2 L66.1,453.4 L67.8,448.8 L69.6,444.4 L71.3,440.1 L73.0,440.8 L74.8,436.0 L76.5,430.9
 L78.3,426.3 L80.0,421.8 L81.7,419.9 L83.5,416.8 L85.2,414.8 L87.0,412.1 L88.7,410.1 L90.5,407.5
 L92.2,405.5 L93.9,403.2 L95.7,405.3 L97.4,404.5 L99.2,402.2 L100.9,398.3 L102.6,398.4 L104.4,398.7
 L106.1,398.1 L107.9,394.7 L109.6,393.1 L111.3,391.4 L113.1,391.9 L114.8,391.1 L116.6,390.5 L118.3,390.1
 L120.0,389.7 L121.8,389.5 L123.5,389.8 L125.3,389.6 L127.0,389.3 L128.7,388.7 L130.5,388.4 L132.2,388.9
 L134.0,389.3 L135.7,391.0 L137.4,390.2 L139.2,391.1 L140.9,391.0 L142.7,390.4 L144.4,393.0 L146.1,393.0
 L147.9,391.2 L149.6,391.6 L151.4,391.1 L153.1,391.7 L154.8,392.4 L156.6,393.1 L158.3,393.8 L160.1,394.5
 L161.8,395.2 L163.6,396.0 L165.3,396.8 L167.0,397.6 L168.8,398.4 L170.5,399.2 L172.3,400.0 L174.0,400.9
 L175.7,401.7 L177.5,402.6 L179.2,403.5 L181.0,404.3 L182.7,405.2 L184.4,406.1 L186.2,407.1 L187.9,408.0
 L189.7,409.0 L191.4,409.9 L193.1,410.8 L194.9,411.7 L196.6,412.6 L198.4,413.5 L200.1,414.3 L201.8,415.2
 L203.6,416.1 L205.3,417.0 L207.1,417.9 L208.8,418.9 L210.5,419.9 L212.3,421.1 L214.0,422.1 L215.8,423.2
 L217.5,424.2 L219.2,425.3 L221.0,426.5 L222.7,427.6 L224.5,428.8 L226.2,430.0 L228.0,431.3 L229.7,432.5
 L231.4,433.8 L233.2,435.2 L234.9,436.6 L236.7,438.0 L238.4,439.4 L240.1,440.8 L241.9,442.4 L243.6,443.9
 L245.4,445.5 L247.1,447.1 L248.8,448.7 L250.6,450.4 L252.3,452.1 L254.1,453.8 L255.8,455.5 L257.5,457.2
 L259.3,459.0 L261.0,460.7 L262.8,462.4 L264.5,464.2 L266.2,465.9 L268.0,467.6 L269.7,469.5 L271.5,471.2
 L273.2,472.9 L274.9,474.7 L276.7,476.4 L278.4,478.0 L280.2,479.6 L281.9,481.2 L283.6,482.9 L285.4,484.5
 L287.1,486.0 L288.9,487.5 L290.6,489.0 L292.3,490.4 L294.1,491.8 L295.8,493.2 L297.6,494.6 L299.3,495.8
 L301.1,497.1 L302.8,498.3 L304.5,499.4 L306.3,500.5 L308.0,501.6 L309.8,502.6 L311.5,503.6 L313.2,504.6
 L315.0,505.5 L316.7,506.4 L318.5,507.3 L320.2,508.1 L321.9,508.9 L323.7,509.7 L325.4,510.5 L327.2,511.1
 L328.9,511.8 L330.6,512.4 L332.4,513.0 L334.1,513.6 L335.9,514.2 L337.6,514.8 L339.3,515.4 L341.1,515.8
 L342.8,516.3 L344.6,516.7 L346.3,517.2 L348.0,517.6 L349.8,518.1 L351.5,518.5 L353.3,518.9 L355.0,519.4
 L356.7,519.8 L358.5,520.4 L360.2,520.8 L362.0,521.3 L363.7,521.8 L365.4,522.4 L367.2,522.9 L368.9,523.5
 L370.7,524.1 L372.4,524.8 L374.2,525.5 L375.9,526.2 L377.6,526.9 L379.4,527.6 L381.1,528.4 L382.9,529.2
 L384.6,530.0 L386.3,530.9 L388.1,531.7 L389.8,532.7 L391.6,533.6 L393.3,534.6 L395.0,535.5 L396.8,536.5
 '/> 
 
 kineticEnergy-3OrdInit 
 
 
 kineticEnergy-3OrdInit 
 
 
 
	<path stroke='rgb(255, 0, 0)' stroke-dasharray='7.5,12.0' d='M433.1,81.1 L486.5,81.1 M53.9,62.9 L55.6,135.8 L57.4,197.6 L59.1,245.1 L60.9,283.4 L62.6,315.1
 L64.3,341.2 L66.1,363.5 L67.8,382.7 L69.6,399.4 L71.3,414.1 L73.0,426.7 L74.8,438.0 L76.5,447.9
 L78.3,456.8 L80.0,464.6 L81.7,471.8 L83.5,478.2 L85.2,484.0 L87.0,489.2 L88.7,494.0 L90.5,498.4
 L92.2,502.4 L93.9,506.1 L95.7,509.5 L97.4,512.6 L99.2,515.5 L100.9,518.2 L102.6,520.7 L104.4,523.0
 L106.1,525.0 L107.9,527.0 L109.6,528.8 L111.3,530.5 L113.1,532.1 L114.8,533.6 L116.6,534.9 L118.3,536.2
 L120.0,537.3 L121.8,538.4 L123.5,539.4 L125.3,540.3 L127.0,517.4 L128.7,533.6 L130.5,538.5 L1

In [ ]:
var gp = PlotTableEnrgy[0,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3 
 
 
 
 
 3.2 
 
 
 
 
 3.4 
 
 
 
 
 3.6 
 
 
 
 
 3.8 
 
 
 
 
 4 
 
 
 
 
 4.2 
 
 
 
 
 4.4 
 
 
 
 
 4.6 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergy-3OrdInit 
 
 
 
 
 totalEnergy-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M455.3,57.1 L508.7,57.1 M53.9,82.2 L55.6,135.7 L57.4,179.2 L59.1,211.2 L60.9,235.5 L62.6,255.2
 L64.3,270.9 L66.1,284.0 L67.8,294.9 L69.6,304.2 L71.3,311.9 L73.0,321.9 L74.8,326.8 L76.5,330.4
 L78.3,333.6 L80.0,336.1 L81.7,340.1 L83.5,342.5 L85.2,345.3 L87.0,347.3 L88.7,349.3 L90.5,350.7
 L92.2,352.2 L93.9,353.2 L95.7,357.4 L97.4,359.1 L99.2,359.6 L100.9,358.7 L102.6,360.6 L104.4,362.6
 L106.1,363.6 L107.9,362.6 L109.6,362.8 L111.3,362.8 L113.1,364.3 L114.8,364.8 L116.6,365.4 L118.3,366.0
 L120.0,366.6 L121.8,367.2 L123.5,368.2 L125.3,368.7 L127.0,351.3 L128.7,363.0 L130.5,366.5 L132.2,368.4
 L134.0,369.6 L135.7,371.7 L137.4,371.7 L139.2,372.8 L140.9,373.1 L142.7,372.8 L144.4,375.1 L146.1,375.4
 L147.9,374.4 L149.6,374.9 L151.4,374.7 L153.1,375.2 L154.8,375.8 L156.6,376.5 L158.3,377.1 L160.1,377.7
 L161.8,378.2 L163.6,378.8 L165.3,379.5 L167.0,380.1 L168.8,380.7 L170.5,381.3 L172.3,381.9 L174.0,382.5
 L175.7,383.1 L177.5,383.7 L179.2,384.3 L181.0,384.9 L182.7,385.6 L184.4,386.2 L186.2,386.8 L187.9,387.4
 L189.7,388.1 L191.4,388.5 L193.1,389.3 L194.9,389.9 L196.6,390.4 L198.4,391.0 L200.1,391.5 L201.8,392.0
 L203.6,392.5 L205.3,393.0 L207.1,393.5 L208.8,394.0 L210.5,394.6 L212.3,395.2 L214.0,395.7 L215.8,396.1
 L217.5,396.6 L219.2,397.1 L221.0,397.6 L222.7,398.0 L224.5,398.5 L226.2,399.0 L228.0,399.4 L229.7,399.9
 L231.4,400.3 L233.2,400.9 L234.9,401.3 L236.7,401.8 L238.4,402.2 L240.1,402.7 L241.9,403.2 L243.6,403.7
 L245.4,404.3 L247.1,404.8 L248.8,405.4 L250.6,405.9 L252.3,406.5 L254.1,407.1 L255.8,407.7 L257.5,408.3
 L259.3,409.0 L261.0,409.6 L262.8,410.2 L264.5,410.9 L266.2,411.5 L268.0,412.2 L269.7,413.0 L271.5,413.7
 L273.2,414.4 L274.9,415.2 L276.7,415.9 L278.4,416.6 L280.2,417.4 L281.9,418.1 L283.6,418.9 L285.4,419.7
 L287.1,420.5 L288.9,421.2 L290.6,422.0 L292.3,422.8 L294.1,423.5 L295.8,424.3 L297.6,425.1 L299.3,425.8
 L301.1,426.6 L302.8,427.3 L304.5,428.0 L306.3,428.8 L308.0,429.5 L309.8,430.2 L311.5,430.9 L313.2,431.6
 L315.0,432.3 L316.7,433.0 L318.5,433.7 L320.2,434.3 L321.9,435.0 L323.7,435.7 L325.4,436.3 L327.2,436.9
 L328.9,437.5 L330.6,438.1 L332.4,438.7 L334.1,439.3 L335.9,439.8 L337.6,440.4 L339.3,441.0 L341.1,441.5
 L342.8,441.9 L344.6,442.4 L346.3,442.9 L348.0,443.3 L349.8,443.8 L351.5,444.2 L353.3,444.6 L355.0,445.0
 L356.7,445.4 L358.5,445.8 L360.2,446.1 L362.0,446.5 L363.7,446.8 L365.4,447.2 L367.2,447.5 L368.9,447.8
 L370.7,448.2 L372.4,448.5 L374.2,448.9 L375.9,449.2 L377.6,449.6 L379.4,449.9 L381.1,450.2 L382.9,450.6
 L384.6,450.9 L386.3,451.3 L388.1,451.6 L389.8,452.0 L391.6,452.4 L393.3,452.8 L395.0,453.2 L396.8,453.6
 '/> 
 
 totalEnergy 
 
 
 totalEnergy 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M455.3,81.1 L508.7,81.1 M53.9,458.1 L55.6,458.1 L57.4,458.1 L59.1,458.1 L60.9,458.1 L62.6,458.1
 L64.3,458.1 L66.1,458.1 L67.8,458.1 L69.6,458.1 L71.3,458.1 L73.0,458.1 L74.8,458.2 L76.5,458.2
 L78.3,458.2 L80.0,458.2 L81.7,458.3 L83.5,458.3 L85.2,458.4 L87.0,458.4 L88.7,458.5 L90.5,458.5
 L92.2,458.6 L93.9,458.6 L95.7,458.7 L97.4,458.7 L99.2,458.8 L100.9,458.9 L102.6,458.9 L104.4,459.0
 L106.1,459.1 L107.9,459.1 L109.6,459.2 L111.3,459.3 L113.1,459.4 L114.8,459.4 L116.6,459.5 L118.3,459.6
 L120.0,459.7 L121.8,459.8 L123.5,459.9 L125.3,460.0 L127.0,460.1 L128.7,460.2 L130.5,460.2 L132.2,

## Case 2 (m2_Oh01_eta02)

In [ ]:
var gp = PlotTableEnrgy[1,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.005 
 
 
 
 
 0.01 
 
 
 
 
 0.015 
 
 
 
 
 0.02 
 
 
 
 
 0.025 
 
 
 
 
 0.03 
 
 
 
 
 0.035 
 
 
 
 
 0.04 
 
 
 
 
 0.045 
 
 
 
 
 0.05 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergy-3OrdInit 
 
 
 
 
 surfaceEnergy-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M433.1,57.1 L486.5,57.1 M70.5,53.7 L72.2,53.5 L73.9,53.5 L75.6,53.6 L77.3,53.8 L79.0,54.1
 L80.7,54.7 L82.4,55.3 L84.1,56.1 L85.8,57.1 L87.5,58.2 L89.2,59.5 L90.9,60.9 L92.6,62.5
 L94.3,64.1 L96.0,66.0 L97.7,68.0 L99.4,70.1 L101.1,72.4 L102.8,74.8 L104.5,77.3 L106.2,79.9
 L107.9,82.6 L109.6,85.5 L111.3,88.5 L113.0,91.5 L114.7,94.7 L116.4,98.0 L118.1,101.3 L119.8,104.8
 L121.5,108.3 L123.2,112.0 L124.9,115.7 L126.6,119.5 L128.3,123.3 L130.0,127.3 L131.7,131.3 L133.4,135.4
 L135.1,139.6 L136.8,143.9 L138.5,148.0 L140.2,152.5 L141.9,157.0 L143.6,161.5 L145.3,166.1 L147.0,170.8
 L148.7,175.5 L150.4,180.1 L152.1,185.0 L153.8,189.9 L155.5,194.9 L157.1,199.7 L158.8,204.9 L160.5,210.0
 L162.2,215.1 L163.9,220.3 L165.6,225.4 L167.3,230.8 L169.0,236.1 L170.7,241.5 L172.4,246.9 L174.1,252.2
 L175.8,257.7 L177.5,263.2 L179.2,268.6 L180.9,274.2 L182.6,279.8 L184.3,285.4 L186.0,291.0 L187.7,296.6
 L189.4,302.2 L191.1,307.8 L192.8,313.5 L194.5,319.1 L196.2,324.7 L197.9,330.4 L199.6,336.0 L201.3,341.6
 L203.0,347.2 L204.7,352.8 L206.4,358.4 L208.1,363.9 L209.8,369.3 L211.5,374.9 L213.2,380.2 L214.9,385.8
 L216.6,391.2 L218.3,396.4 L220.0,401.8 L221.7,407.1 L223.4,412.2 L225.1,417.5 L226.8,422.6 L228.5,427.7
 L230.2,432.7 L231.9,437.7 L233.6,442.6 L235.3,447.4 L237.0,452.1 L238.7,456.6 L240.4,461.1 L242.1,465.8
 L243.8,470.2 L245.5,474.6 L247.2,478.9 L248.9,482.9 L250.6,487.1 L252.3,491.1 L254.0,495.0 L255.7,498.7
 L257.4,502.5 L259.1,506.2 L260.8,509.7 L262.5,513.0 L264.2,516.4 L265.9,519.6 L267.6,522.7 L269.3,525.7
 L271.0,528.6 L272.7,531.3 L274.4,534.0 L276.1,536.3 L277.8,538.9 L279.5,541.0 L281.2,543.3 L282.9,545.4
 L284.6,547.3 L286.3,549.1 L288.0,550.8 L289.7,552.3 L291.4,553.7 L293.1,555.0 L294.8,556.2 L296.5,557.3
 L298.2,558.2 L299.9,559.0 L301.6,559.7 L303.3,560.3 L305.0,560.8 L306.7,561.1 L308.4,561.3 L310.1,561.5
 L311.8,561.5 L313.5,561.3 L315.2,561.1 L316.9,560.8 L318.6,560.3 L320.3,559.7 L322.0,559.1 L323.7,558.3
 L325.4,557.4 L327.0,556.4 L328.7,555.4 L330.4,554.2 L332.1,552.9 L333.8,551.4 L335.5,549.9 L337.2,548.3
 L338.9,546.8 L340.6,545.1 L342.3,543.3 L344.0,541.4 L345.7,539.4 L347.4,537.3 L349.1,535.2 L350.8,533.0
 L352.5,530.7 L354.2,528.4 L355.9,526.0 L357.6,523.5 L359.3,521.0 L361.0,518.4 L362.7,515.8 L364.4,513.1
 L366.1,510.1 L367.8,507.4 L369.5,504.6 L371.2,501.8 L372.9,498.9 L374.6,496.0 L376.3,493.0 L378.0,490.0
 L379.7,487.0 L381.4,484.0 L383.1,481.0 L384.8,478.0 L386.5,474.9 L388.2,471.8 L389.9,468.8 L391.6,465.7
 L393.3,462.7 L395.0,459.6 L396.7,456.6 L398.4,453.5 L400.1,450.5 L401.8,447.5 L403.5,444.5 L405.2,441.6
 L406.9,438.6 L408.6,435.7 L410.3,432.9 L412.0,430.0 L413.7,427.2 L415.4,424.5 L417.1,421.8 L418.8,419.1
 L420.5,416.5 L422.2,413.9 L423.9,411.4 L425.6,408.9 L427.3,406.5 L429.0,404.2 L430.7,401.5 L432.4,399.4
 L434.1,397.3 L435.8,395.0 L437.5,393.1 L439.2,391.2 L440.9,389.4 L442.6,387.6 L444.3,385.9 L446.0,384.3
 L447.7,382.7 L449.4,381.3 L451.1,379.9 L452.8,378.5 L454.5,377.3 L456.2,376.2 L457.9,375.1 L459.6,374.1
 L461.3,373.2 L463.0,372.4 L464.7,371.7 L466.4,371.1 L468.1,370.5 L469.8,370.0 L471.5,369.7 L473.2,369.4
 L474.9,369.2 L476.6,369.1 L478.3,369.0 L480.0,369.1 L481.7,369.2 L483.4,369.5 L485.1,369.8 L486.8,

In [ ]:
var gp = PlotTableEnrgy[1,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.15 
 
 
 
 
 3.155 
 
 
 
 
 3.16 
 
 
 
 
 3.165 
 
 
 
 
 3.17 
 
 
 
 
 3.175 
 
 
 
 
 3.18 
 
 
 
 
 3.185 
 
 
 
 
 3.19 
 
 
 
 
 3.195 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergy-3OrdInit 
 
 
 
 
 totalEnergy-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M455.3,57.1 L508.7,57.1 M70.5,83.2 L72.2,84.4 L73.9,85.4 L75.6,86.3 L77.3,87.0 L79.0,87.5
 L80.7,88.0 L82.4,88.5 L84.1,88.9 L85.8,89.3 L87.5,89.7 L89.2,90.1 L90.9,90.4 L92.6,90.8
 L94.3,91.1 L96.0,91.5 L97.7,91.9 L99.4,92.3 L101.1,92.8 L102.8,93.2 L104.5,93.7 L106.2,94.2
 L107.9,94.7 L109.6,95.2 L111.3,95.7 L113.0,96.3 L114.7,96.9 L116.4,97.5 L118.1,98.1 L119.8,98.8
 L121.5,99.5 L123.2,100.1 L124.9,100.9 L126.6,101.6 L128.3,102.3 L130.0,103.1 L131.7,103.9 L133.4,104.7
 L135.1,105.6 L136.8,106.4 L138.5,107.1 L140.2,108.1 L141.9,109.0 L143.6,110.0 L145.3,111.0 L147.0,112.0
 L148.7,113.0 L150.4,113.8 L152.1,115.0 L153.8,116.1 L155.5,117.2 L157.1,118.2 L158.8,119.5 L160.5,120.7
 L162.2,121.9 L163.9,123.1 L165.6,124.2 L167.3,125.6 L169.0,127.0 L170.7,128.4 L172.4,129.7 L174.1,131.0
 L175.8,132.6 L177.5,134.1 L179.2,135.4 L180.9,137.1 L182.6,138.7 L184.3,140.3 L186.0,142.0 L187.7,143.7
 L189.4,145.4 L191.1,147.1 L192.8,148.9 L194.5,150.8 L196.2,152.6 L197.9,154.5 L199.6,156.5 L201.3,158.4
 L203.0,160.4 L204.7,162.5 L206.4,164.5 L208.1,166.6 L209.8,168.6 L211.5,170.9 L213.2,173.0 L214.9,175.4
 L216.6,177.7 L218.3,179.8 L220.0,182.3 L221.7,184.7 L223.4,186.9 L225.1,189.5 L226.8,192.0 L228.5,194.6
 L230.2,197.1 L231.9,199.7 L233.6,202.3 L235.3,204.9 L237.0,207.5 L238.7,209.9 L240.4,212.6 L242.1,215.4
 L243.8,218.2 L245.5,221.0 L247.2,223.8 L248.9,226.4 L250.6,229.3 L252.3,232.2 L254.0,235.0 L255.7,237.8
 L257.4,240.8 L259.1,243.7 L260.8,246.6 L262.5,249.4 L264.2,252.5 L265.9,255.4 L267.6,258.4 L269.3,261.4
 L271.0,264.3 L272.7,267.3 L274.4,270.3 L276.1,273.1 L277.8,276.2 L279.5,279.0 L281.2,282.1 L282.9,285.1
 L284.6,288.0 L286.3,291.0 L288.0,293.9 L289.7,296.8 L291.4,299.7 L293.1,302.6 L294.8,305.5 L296.5,308.4
 L298.2,311.2 L299.9,314.1 L301.6,316.9 L303.3,319.7 L305.0,322.5 L306.7,325.3 L308.4,328.0 L310.1,330.7
 L311.8,333.5 L313.5,336.1 L315.2,338.8 L316.9,341.5 L318.6,344.1 L320.3,346.7 L322.0,349.2 L323.7,351.8
 L325.4,354.3 L327.0,356.6 L328.7,359.2 L330.4,361.6 L332.1,364.0 L333.8,366.2 L335.5,368.5 L337.2,370.7
 L338.9,373.1 L340.6,375.3 L342.3,377.5 L344.0,379.7 L345.7,381.8 L347.4,383.8 L349.1,385.9 L350.8,387.8
 L352.5,389.8 L354.2,391.7 L355.9,393.6 L357.6,395.4 L359.3,397.2 L361.0,398.9 L362.7,400.6 L364.4,402.3
 L366.1,403.7 L367.8,405.4 L369.5,406.9 L371.2,408.5 L372.9,409.9 L374.6,411.4 L376.3,412.7 L378.0,414.1
 L379.7,415.4 L381.4,416.7 L383.1,417.9 L384.8,419.1 L386.5,420.2 L388.2,421.3 L389.9,422.4 L391.6,423.5
 L393.3,424.5 L395.0,425.4 L396.7,426.4 L398.4,427.2 L400.1,428.1 L401.8,428.9 L403.5,429.7 L405.2,430.5
 L406.9,431.2 L408.6,431.9 L410.3,432.6 L412.0,433.2 L413.7,433.8 L415.4,434.3 L417.1,434.9 L418.8,435.4
 L420.5,435.9 L422.2,436.3 L423.9,436.8 L425.6,437.2 L427.3,437.6 L429.0,437.9 L430.7,437.8 L432.4,438.2
 L434.1,438.6 L435.8,438.6 L437.5,439.0 L439.2,439.3 L440.9,439.6 L442.6,439.8 L444.3,440.1 L446.0,440.3
 L447.7,440.4 L449.4,440.6 L451.1,440.8 L452.8,440.9 L454.5,441.0 L456.2,441.1 L457.9,441.3 L459.6,441.4
 L461.3,441.5 L463.0,441.6 L464.7,441.6 L466.4,441.7 L468.1,441.8 L469.8,441.9 L471.5,442.0 L473.2,442.0
 L474.9,442.1 L476.6,442.2 L478.3,442.3 L480.0,442.4 L481.7,442.4 L483.4,442.5 L485.1,442.6 L486.8,442.7
 L488.5,442.8 L490.2,44

## Case 3 (m2_Oh01_eta01)

In [ ]:
var gp = PlotTableEnrgy[2,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 0.002 
 
 
 
 
 0.004 
 
 
 
 
 0.006 
 
 
 
 
 0.008 
 
 
 
 
 0.01 
 
 
 
 
 0.012 
 
 
 
 
 0.014 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergy-3OrdInit 
 
 
 
 
 surfaceEnergy-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M433.1,57.1 L486.5,57.1 M70.5,96.5 L72.2,96.4 L73.9,96.6 L75.6,96.9 L77.3,97.4 L79.0,98.0
 L80.7,98.8 L82.4,99.7 L84.1,100.8 L85.8,102.0 L87.5,103.2 L89.2,104.8 L90.9,106.5 L92.6,108.4
 L94.3,110.3 L96.0,112.4 L97.7,114.7 L99.4,117.0 L101.1,119.5 L102.8,122.2 L104.5,124.9 L106.2,127.8
 L107.9,130.8 L109.6,133.9 L111.3,137.2 L113.0,140.5 L114.7,144.0 L116.4,147.6 L118.1,151.3 L119.8,155.1
 L121.5,159.0 L123.2,163.0 L124.9,167.1 L126.6,171.3 L128.3,175.6 L130.0,179.9 L131.7,184.4 L133.4,188.9
 L135.1,193.5 L136.8,198.2 L138.5,203.0 L140.2,208.0 L141.9,213.0 L143.6,217.9 L145.3,222.9 L147.0,228.0
 L148.7,232.9 L150.4,238.2 L152.1,243.5 L153.8,248.8 L155.5,254.1 L157.1,259.4 L158.8,264.8 L160.5,270.0
 L162.2,275.6 L163.9,281.1 L165.6,286.6 L167.3,292.1 L169.0,297.4 L170.7,303.1 L172.4,308.7 L174.1,314.2
 L175.8,319.8 L177.5,325.2 L179.2,330.9 L180.9,336.5 L182.6,342.0 L184.3,347.5 L186.0,352.8 L187.7,358.5
 L189.4,364.0 L191.1,369.5 L192.8,374.9 L194.5,380.3 L196.2,385.7 L197.9,391.0 L199.6,396.3 L201.3,401.5
 L203.0,406.7 L204.7,411.9 L206.4,417.0 L208.1,422.0 L209.8,426.7 L211.5,431.8 L213.2,436.8 L214.9,441.6
 L216.6,446.3 L218.3,451.0 L220.0,455.6 L221.7,460.1 L223.4,464.5 L225.1,468.6 L226.8,473.2 L228.5,477.4
 L230.2,481.5 L231.9,485.2 L233.6,489.4 L235.3,493.3 L237.0,497.0 L238.7,500.7 L240.4,504.2 L242.1,507.6
 L243.8,511.0 L245.5,514.2 L247.2,517.4 L248.9,520.4 L250.6,523.3 L252.3,526.1 L254.0,528.8 L255.7,531.4
 L257.4,533.9 L259.1,536.3 L260.8,538.5 L262.5,540.7 L264.2,542.7 L265.9,544.6 L267.6,546.4 L269.3,548.1
 L271.0,549.7 L272.7,551.1 L274.4,552.5 L276.1,553.7 L277.8,554.8 L279.5,555.8 L281.2,556.7 L282.9,557.5
 L284.6,558.1 L286.3,558.7 L288.0,559.1 L289.7,559.5 L291.4,559.7 L293.1,559.8 L294.8,559.8 L296.5,559.7
 L298.2,559.5 L299.9,559.1 L301.6,558.7 L303.3,558.2 L305.0,557.6 L306.7,556.9 L308.4,556.1 L310.1,555.1
 L311.8,554.1 L313.5,553.0 L315.2,551.9 L316.9,550.6 L318.6,549.2 L320.3,547.8 L322.0,546.3 L323.7,544.6
 L325.4,542.6 L327.0,541.0 L328.7,539.3 L330.4,537.4 L332.1,535.5 L333.8,533.5 L335.5,531.4 L337.2,529.2
 L338.9,527.0 L340.6,524.1 L342.3,521.8 L344.0,519.6 L345.7,517.3 L347.4,514.3 L349.1,512.1 L350.8,509.6
 L352.5,507.1 L354.2,504.5 L355.9,501.9 L357.6,499.2 L359.3,496.5 L361.0,493.8 L362.7,491.0 L364.4,488.3
 L366.1,485.5 L367.8,482.7 L369.5,479.9 L371.2,477.1 L372.9,474.2 L374.6,471.4 L376.3,468.6 L378.0,465.8
 L379.7,463.0 L381.4,460.2 L383.1,457.4 L384.8,454.6 L386.5,451.8 L388.2,449.1 L389.9,446.4 L391.6,443.7
 L393.3,441.0 L395.0,438.4 L396.7,435.8 L398.4,433.2 L400.1,430.7 L401.8,428.2 L403.5,425.7 L405.2,423.3
 L406.9,420.9 L408.6,418.6 L410.3,416.3 L412.0,414.1 L413.7,411.9 L415.4,409.8 L417.1,407.8 L418.8,405.8
 L420.5,403.8 L422.2,401.9 L423.9,400.1 L425.6,398.3 L427.3,396.7 L429.0,395.0 L430.7,393.5 L432.4,392.0
 L434.1,390.5 L435.8,389.2 L437.5,387.9 L439.2,386.7 L440.9,384.9 L442.6,384.0 L444.3,383.1 L446.0,382.3
 L447.7,381.5 L449.4,380.7 L451.1,380.1 L452.8,379.5 L454.5,379.0 L456.2,378.5 L457.9,378.1 L459.6,377.9
 L461.3,377.6 L463.0,377.5 L464.7,377.4 L466.4,377.5 L468.1,377.6 L469.8,377.7 L471.5,378.0 L473.2,378.3
 L474.9,378.7 L476.6,379.1 L478.3,379.7 L480.0,380.3 L481.7,381.0 L483.4,381.7 L485.1,382.6 L486.8,383.5
 L488.5,384.4 L490.2,385.4 L491.9,386.5 L493.6,3

In [ ]:
var gp = PlotTableEnrgy[2,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.142 
 
 
 
 
 3.144 
 
 
 
 
 3.146 
 
 
 
 
 3.148 
 
 
 
 
 3.15 
 
 
 
 
 3.152 
 
 
 
 
 3.154 
 
 
 
 
 3.156 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergy-3OrdInit 
 
 
 
 
 totalEnergy-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M455.3,57.1 L508.7,57.1 M70.5,105.7 L72.2,105.8 L73.9,105.9 L75.6,106.1 L77.3,106.2 L79.0,106.3
 L80.7,106.5 L82.4,106.6 L84.1,106.8 L85.8,107.0 L87.5,107.0 L89.2,107.3 L90.9,107.5 L92.6,107.8
 L94.3,108.0 L96.0,108.3 L97.7,108.6 L99.4,108.9 L101.1,109.2 L102.8,109.5 L104.5,109.9 L106.2,110.2
 L107.9,110.6 L109.6,111.1 L111.3,111.5 L113.0,112.0 L114.7,112.5 L116.4,113.0 L118.1,113.5 L119.8,114.1
 L121.5,114.7 L123.2,115.3 L124.9,115.9 L126.6,116.6 L128.3,117.3 L130.0,118.0 L131.7,118.8 L133.4,119.6
 L135.1,120.4 L136.8,121.2 L138.5,122.1 L140.2,123.1 L141.9,124.2 L143.6,125.1 L145.3,126.1 L147.0,127.1
 L148.7,128.0 L150.4,129.2 L152.1,130.3 L153.8,131.4 L155.5,132.6 L157.1,133.8 L158.8,135.0 L160.5,136.1
 L162.2,137.5 L163.9,138.9 L165.6,140.2 L167.3,141.6 L169.0,142.7 L170.7,144.4 L172.4,145.9 L174.1,147.4
 L175.8,148.9 L177.5,150.3 L179.2,152.0 L180.9,153.7 L182.6,155.3 L184.3,157.0 L186.0,158.5 L187.7,160.4
 L189.4,162.2 L191.1,164.0 L192.8,165.8 L194.5,167.7 L196.2,169.6 L197.9,171.5 L199.6,173.4 L201.3,175.4
 L203.0,177.4 L204.7,179.4 L206.4,181.4 L208.1,183.5 L209.8,185.2 L211.5,187.5 L213.2,189.7 L214.9,191.9
 L216.6,194.0 L218.3,196.2 L220.0,198.4 L221.7,200.6 L223.4,202.8 L225.1,204.8 L226.8,207.3 L228.5,209.7
 L230.2,212.0 L231.9,213.9 L233.6,216.5 L235.3,218.9 L237.0,221.2 L238.7,223.6 L240.4,225.9 L242.1,228.3
 L243.8,230.7 L245.5,233.0 L247.2,235.4 L248.9,237.8 L250.6,240.2 L252.3,242.6 L254.0,244.9 L255.7,247.3
 L257.4,249.7 L259.1,252.1 L260.8,254.5 L262.5,256.9 L264.2,259.3 L265.9,261.6 L267.6,264.0 L269.3,266.4
 L271.0,268.7 L272.7,271.1 L274.4,273.4 L276.1,275.7 L277.8,278.1 L279.5,280.4 L281.2,282.7 L282.9,284.9
 L284.6,287.2 L286.3,289.5 L288.0,291.7 L289.7,293.9 L291.4,296.1 L293.1,298.3 L294.8,300.5 L296.5,302.7
 L298.2,304.8 L299.9,306.9 L301.6,309.0 L303.3,311.1 L305.0,313.1 L306.7,315.2 L308.4,317.2 L310.1,319.2
 L311.8,321.1 L313.5,323.1 L315.2,325.0 L316.9,326.8 L318.6,328.7 L320.3,330.5 L322.0,332.3 L323.7,334.1
 L325.4,335.5 L327.0,337.4 L328.7,339.2 L330.4,340.9 L332.1,342.5 L333.8,344.2 L335.5,345.7 L337.2,347.3
 L338.9,348.8 L340.6,349.6 L342.3,351.1 L344.0,352.8 L345.7,354.2 L347.4,355.1 L349.1,356.6 L350.8,358.1
 L352.5,359.4 L354.2,360.7 L355.9,361.9 L357.6,363.1 L359.3,364.2 L361.0,365.3 L362.7,366.4 L364.4,367.5
 L366.1,368.5 L367.8,369.5 L369.5,370.4 L371.2,371.4 L372.9,372.3 L374.6,373.1 L376.3,374.0 L378.0,374.8
 L379.7,375.5 L381.4,376.3 L383.1,377.0 L384.8,377.7 L386.5,378.4 L388.2,379.0 L389.9,379.6 L391.6,380.2
 L393.3,380.8 L395.0,381.3 L396.7,381.9 L398.4,382.3 L400.1,382.8 L401.8,383.3 L403.5,383.7 L405.2,384.1
 L406.9,384.5 L408.6,384.9 L410.3,385.2 L412.0,385.5 L413.7,385.8 L415.4,386.1 L417.1,386.4 L418.8,386.7
 L420.5,386.9 L422.2,387.1 L423.9,387.4 L425.6,387.6 L427.3,387.7 L429.0,387.9 L430.7,388.1 L432.4,388.2
 L434.1,388.4 L435.8,388.5 L437.5,388.6 L439.2,388.7 L440.9,388.2 L442.6,388.5 L444.3,388.7 L446.0,388.8
 L447.7,388.9 L449.4,389.0 L451.1,389.1 L452.8,389.2 L454.5,389.3 L456.2,389.3 L457.9,389.4 L459.6,389.5
 L461.3,389.6 L463.0,389.6 L464.7,389.7 L466.4,389.8 L468.1,389.9 L469.8,389.9 L471.5,390.0 L473.2,390.1
 L474.9,390.2 L476.6,390.3 L478.3,390.4 L480.0,390.5 L481.7,390.7 L483.4,390.8 L485.1,390.9 L486.8,391.1
 L488.5,391.2 L490.2,391.4 L491.9,391.5 

## Case 4 (m3_Oh01_eta04)

In [ ]:
var gp = PlotTableEnrgy[3,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergy-3OrdInit 
 
 
 
 
 surfaceEnergy-3OrdInit 
 
 
 
 
 
 kineticEnergy-3OrdInit 
 
 
 kineticEnergy-3OrdInit 
 
 
 
 
 
 surfaceEnergy 
 
 
 surfaceEnergy 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M433.1,105.1 L486.5,105.1 M45.6,549.9 L47.4,549.9 L49.1,550.0 L50.9,550.0 L52.6,550.1 L54.4,550.2
 L56.2,550.3 L57.9,550.4 L59.7,550.5 L61.5,550.7 L63.2,550.9 L65.0,551.0 L66.7,551.2 L68.5,551.4
 L70.3,551.6 L72.0,551.8 L73.8,552.1 L75.5,552.3 L77.3,552.5 L79.1,552.8 L80.8,553.0 L82.6,553.3
 L84.3,553.5 L86.1,553.8 L87.9,554.1 L89.6,554.3 L91.4,554.6 L93.2,554.9 L94.9,555.2 L96.7,555.4
 L98.4,555.7 L100.2,556.0 L102.0,556.3 L103.7,556.5 L105.5,556.8 L107.2,557.1 L109.0,557.4 L110.8,557.6
 L112.5,557.9 L114.3,558.2 L116.1,558.4 L117.8,558.7 L119.6,558.9 L121.3,559.2 L123.1,559.4 L124.9,559.6
 L126.6,559.9 L128.4,560.1 L130.1,560.3 L131.9,560.5 L133.7,560.7 L135.4,560.9 L137.2,561.1 L138.9,561.3
 L140.7,561.5 L142.5,561.6 L144.2,561.8 L146.0,561.9 L147.8,562.1 L149.5,562.2 L151.3,562.4 L153.0,562.5
 L154.8,562.6 L156.6,562.7 L158.3,562.8 L160.1,562.9 L161.8,563.0 L163.6,563.1 L165.4,563.1 L167.1,563.2
 L168.9,563.3 L170.6,563.3 L172.4,563.4 L174.2,563.4 L175.9,563.4 L177.7,563.4 L179.5,563.5 L181.2,563.5
 L183.0,563.5 L184.7,563.5 L186.5,563.5 L188.3,563.4 L190.0,563.4 L191.8,563.4 L193.5,563.4 L195.3,563.3
 L197.1,563.3 L198.8,563.3 L200.6,563.2 L202.4,563.1 L204.1,563.1 L205.9,563.0 L207.6,563.0 L209.4,562.9
 L211.2,562.8 L212.9,562.7 L214.7,562.7 L216.4,562.6 L218.2,562.5 L220.0,562.4 L221.7,562.3 L223.5,562.2
 L225.2,562.2 L227.0,562.1 L228.8,562.0 L230.5,561.9 L232.3,561.8 L234.1,561.7 L235.8,561.6 L237.6,561.5
 L239.3,561.4 L241.1,561.4 L242.9,561.3 L244.6,561.2 L246.4,561.1 L248.1,561.0 L249.9,561.0 L251.7,560.9
 L253.4,560.8 L255.2,560.8 L257.0,560.7 L258.7,560.7 L260.5,560.6 L262.2,560.6 L264.0,560.5 L265.8,560.5
 L267.5,560.5 L269.3,560.4 L271.0,560.4 L272.8,560.4 L274.6,560.4 L276.3,560.4 L278.1,560.4 L279.8,560.4
 L281.6,560.4 L283.4,560.4 L285.1,560.4 L286.9,560.4 L288.7,560.5 L290.4,560.5 L292.2,560.5 L293.9,560.6
 L295.7,560.6 L297.5,560.6 L299.2,560.7 L301.0,560.7 L302.7,560.8 L304.5,560.8 L306.3,560.9 L308.0,561.0
 L309.8,561.0 L311.5,561.1 L313.3,561.1 L315.1,561.2 L316.8,561.3 L318.6,561.3 L320.4,561.4 L322.1,561.5
 L323.9,561.6 L325.6,561.6 L327.4,561.7 L329.2,561.8 L330.9,561.9 L332.7,561.9 L334.4,562.0 L336.2,562.1
 L338.0,562.1 L339.7,562.2 L341.5,562.3 L343.3,562.4 L345.0,562.4 L346.8,562.5 L348.5,562.6 L350.3,562.6
 L352.1,562.7 L353.8,562.8 L355.6,562.8 L357.3,562.9 L359.1,563.0 L360.9,563.0 L362.6,563.1 L364.4,563.2
 L366.1,563.2 L367.9,563.2 L369.7,563.3 L371.4,563.3 L373.2,563.4 L375.0,563.4 L376.7,563.5 L378.5,563.5
 L380.2,563.6 L382.0,563.6 L383.8,563.6 L385.5,563.7 L387.3,563.7 L389.0,563.7 L390.8,563.8 L392.6,563.8
 L394.3,563.8 L396.1,563.8 L397.9,563.9 L399.6,563.9 L401.4,563.9 L403.1,563.9 L404.9,563.9 L406.7,563.9
 L408.4,563.9 L410.2,563.9 L411.9,563.9 L413.7,563.9 L415.5,563.9 L417.2,563.9 L419.0,563.9 L420.7,563.9
 L422.5,563.9 L424.3,563.9 L426.0,563.9 L427.8,563.8 L429.6,563.8 L431.3,563.8 L433.1,563.8 L434.8,563.8
 L436.6,563.8 L438.4,563.7 L440.1,563.7 L441.9,563.7 L443.6,563.7 L445.4,563.6 L447.2,563.6 L448.9,563.6
 L450.7,563.6 L452.4,563.5 L454.2,563.5 L456.0,563.5 L457.7,563.5 L459.5,563.4 L461.3,563.4 L463.0,563.4
 L464.8,563.4 L466.5,563.3 L468.3,563.3 L470.1,563.3 L471.8,563.3 L473.6,563.3 L475.3,563.2 L477.1,563.2
 L478.9,563.2 L4

In [ ]:
var gp = PlotTableEnrgy[3,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 14 
 
 
 
 
 16 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergy-3OrdInit 
 
 
 
 
 totalEnergy-3OrdInit 
 
 
 
 
 
 totalEnergy 
 
 
 totalEnergy 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M455.3,81.1 L508.7,81.1 M45.6,508.9 L47.4,508.9 L49.1,508.9 L50.9,508.9 L52.6,508.9 L54.4,508.9
 L56.2,508.9 L57.9,508.9 L59.7,508.9 L61.5,509.0 L63.2,509.0 L65.0,509.0 L66.7,509.0 L68.5,509.1
 L70.3,509.1 L72.0,509.2 L73.8,509.2 L75.5,509.3 L77.3,509.3 L79.1,509.4 L80.8,509.4 L82.6,509.5
 L84.3,509.6 L86.1,509.6 L87.9,509.7 L89.6,509.8 L91.4,509.9 L93.2,509.9 L94.9,510.0 L96.7,510.1
 L98.4,510.2 L100.2,510.3 L102.0,510.4 L103.7,510.5 L105.5,510.6 L107.2,510.7 L109.0,510.8 L110.8,510.9
 L112.5,511.0 L114.3,511.1 L116.1,511.2 L117.8,511.3 L119.6,511.4 L121.3,511.5 L123.1,511.6 L124.9,511.8
 L126.6,511.9 L128.4,512.0 L130.1,512.1 L131.9,512.2 L133.7,512.3 L135.4,512.5 L137.2,512.6 L138.9,512.7
 L140.7,512.8 L142.5,512.9 L144.2,513.1 L146.0,513.2 L147.8,513.3 L149.5,513.4 L151.3,513.5 L153.0,513.7
 L154.8,513.8 L156.6,513.9 L158.3,514.0 L160.1,514.1 L161.8,514.2 L163.6,514.3 L165.4,514.5 L167.1,514.6
 L168.9,514.7 L170.6,514.8 L172.4,514.9 L174.2,515.0 L175.9,515.1 L177.7,515.2 L179.5,515.3 L181.2,515.4
 L183.0,515.5 L184.7,515.6 L186.5,515.7 L188.3,515.8 L190.0,515.9 L191.8,516.0 L193.5,516.1 L195.3,516.1
 L197.1,516.2 L198.8,516.3 L200.6,516.4 L202.4,516.5 L204.1,516.5 L205.9,516.6 L207.6,516.7 L209.4,516.7
 L211.2,516.8 L212.9,516.9 L214.7,516.9 L216.4,517.0 L218.2,517.0 L220.0,517.1 L221.7,517.1 L223.5,517.2
 L225.2,517.2 L227.0,517.3 L228.8,517.3 L230.5,517.4 L232.3,517.4 L234.1,517.4 L235.8,517.5 L237.6,517.5
 L239.3,517.5 L241.1,517.5 L242.9,517.5 L244.6,517.6 L246.4,517.6 L248.1,517.6 L249.9,517.6 L251.7,517.6
 L253.4,517.6 L255.2,517.6 L257.0,517.7 L258.7,517.7 L260.5,517.7 L262.2,517.7 L264.0,517.7 L265.8,517.7
 L267.5,517.7 L269.3,517.7 L271.0,517.7 L272.8,517.7 L274.6,517.7 L276.3,517.7 L278.1,517.7 L279.8,517.7
 L281.6,517.7 L283.4,517.7 L285.1,517.7 L286.9,517.7 L288.7,517.8 L290.4,517.8 L292.2,517.8 L293.9,517.8
 L295.7,517.8 L297.5,517.8 L299.2,517.8 L301.0,517.8 L302.7,517.8 L304.5,517.8 L306.3,517.9 L308.0,517.9
 L309.8,517.9 L311.5,517.9 L313.3,517.9 L315.1,517.9 L316.8,518.0 L318.6,518.0 L320.4,518.0 L322.1,518.0
 L323.9,518.0 L325.6,518.1 L327.4,518.1 L329.2,518.1 L330.9,518.1 L332.7,518.2 L334.4,518.2 L336.2,518.2
 L338.0,518.2 L339.7,518.3 L341.5,518.3 L343.3,518.3 L345.0,518.4 L346.8,518.4 L348.5,518.4 L350.3,518.4
 L352.1,518.5 L353.8,518.5 L355.6,518.5 L357.3,518.6 L359.1,518.6 L360.9,518.6 L362.6,518.7 L364.4,518.7
 L366.1,518.7 L367.9,518.7 L369.7,518.8 L371.4,518.8 L373.2,518.8 L375.0,518.9 L376.7,518.9 L378.5,518.9
 L380.2,519.0 L382.0,519.0 L383.8,519.0 L385.5,519.1 L387.3,519.1 L389.0,519.1 L390.8,519.2 L392.6,519.2
 L394.3,519.2 L396.1,519.3 L397.9,519.3 L399.6,519.3 L401.4,519.4 L403.1,519.4 L404.9,519.4 L406.7,519.5
 L408.4,519.5 L410.2,519.5 L411.9,519.5 L413.7,519.6 L415.5,519.6 L417.2,519.6 L419.0,519.6 L420.7,519.7
 L422.5,519.7 L424.3,519.7 L426.0,519.7 L427.8,519.7 L429.6,519.8 L431.3,519.8 L433.1,519.8 L434.8,519.8
 L436.6,519.8 L438.4,519.9 L440.1,519.9 L441.9,519.9 L443.6,519.9 L445.4,519.9 L447.2,519.9 L448.9,519.9
 L450.7,519.9 L452.4,520.0 L454.2,520.0 L456.0,520.0 L457.7,520.0 L459.5,520.0 L461.3,520.0 L463.0,520.0
 L464.8,520.0 L466.5,520.0 L468.3,520.0 L470.1,520.0 L471.8,520.0 L473.6,520.0 L475.3,520.0 L477.1,520.0
 L478.9,520.0 L480.6,520.0 L482.4,520.0 L484.2,520.0 L485.9,520.1 

## Case 5 (m4_Oh01_eta04)

In [ ]:
var gp = PlotTableEnrgy[4,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergy-3OrdInit 
 
 
 
 
 surfaceEnergy-3OrdInit 
 
 
 
 
 
 kineticEnergy-3OrdInit 
 
 
 kineticEnergy-3OrdInit 
 
 
 
 
 
 surfaceEnergy 
 
 
 surfaceEnergy 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M433.1,105.1 L486.5,105.1 M53.9,563.1 L55.6,563.1 L57.4,563.1 L59.1,563.1 L60.9,563.1 L62.6,563.1
 L64.3,563.1 L66.1,563.1 L67.8,563.1 L69.6,563.1 L71.3,563.2 L73.0,563.2 L74.8,563.2 L76.5,563.2
 L78.3,563.2 L80.0,563.3 L81.7,563.3 L83.5,563.3 L85.2,563.3 L87.0,563.3 L88.7,563.4 L90.5,563.4
 L92.2,563.4 L93.9,563.4 L95.7,563.5 L97.4,563.5 L99.2,563.5 L100.9,563.5 L102.6,563.6 L104.4,563.6
 L106.1,563.6 L107.9,563.6 L109.6,563.6 L111.3,563.7 L113.1,563.7 L114.8,563.7 L116.6,563.7 L118.3,563.8
 L120.0,563.8 L121.8,563.8 L123.5,563.8 L125.3,563.8 L127.0,563.8 L128.7,563.9 L130.5,563.9 L132.2,563.9
 L134.0,563.9 L135.7,563.9 L137.4,563.9 L139.2,563.9 L140.9,563.9 L142.7,563.9 L144.4,563.9 L146.1,563.9
 L147.9,564.0 L149.6,564.0 L151.4,564.0 L153.1,564.0 L154.8,564.0 L156.6,564.0 L158.3,564.0 L160.1,564.0
 L161.8,564.0 L163.6,564.0 L165.3,564.0 L167.0,564.0 L168.8,563.9 L170.5,563.9 L172.3,563.9 L174.0,563.9
 L175.7,563.9 L177.5,563.9 L179.2,563.9 L181.0,563.9 L182.7,563.9 L184.4,563.9 L186.2,563.9 L187.9,563.9
 L189.7,563.9 L191.4,563.9 L193.1,563.8 L194.9,563.8 L196.6,563.8 L198.4,563.8 L200.1,563.8 L201.8,563.8
 L203.6,563.8 L205.3,563.8 L207.1,563.8 L208.8,563.8 L210.5,563.8 L212.3,563.8 L214.0,563.8 L215.8,563.8
 L217.5,563.8 L219.2,563.8 L221.0,563.8 L222.7,563.8 L224.5,563.8 L226.2,563.8 L228.0,563.8 L229.7,563.8
 L231.4,563.8 L233.2,563.8 L234.9,563.8 L236.7,563.8 L238.4,563.8 L240.1,563.9 L241.9,563.9 L243.6,563.9
 L245.4,563.9 L247.1,563.9 L248.8,563.9 L250.6,563.9 L252.3,563.9 L254.1,563.9 L255.8,563.9 L257.5,563.9
 L259.3,563.9 L261.0,563.9 L262.8,563.9 L264.5,563.9 L266.2,563.9 L268.0,563.9 L269.7,563.9 L271.5,564.0
 L273.2,564.0 L274.9,564.0 L276.7,564.0 L278.4,564.0 L280.2,564.0 L281.9,564.0 L283.6,564.0 L285.4,564.0
 L287.1,564.0 L288.9,564.0 L290.6,564.0 L292.3,564.0 L294.1,564.0 L295.8,564.0 L297.6,564.0 L299.3,564.0
 L301.1,564.0 L302.8,564.0 L304.5,564.0 L306.3,564.0 L308.0,564.0 L309.8,564.0 L311.5,564.0 L313.2,564.0
 L315.0,564.0 L316.7,564.0 L318.5,564.0 L320.2,564.0 L321.9,564.0 L323.7,564.0 L325.4,564.0 L327.2,564.0
 L328.9,564.0 L330.6,564.0 L332.4,564.0 L334.1,564.0 L335.9,564.0 L337.6,564.0 L339.3,564.0 L341.1,564.0
 L342.8,564.0 L344.6,564.0 L346.3,564.0 L348.0,564.0 L349.8,564.0 L351.5,564.0 L353.3,564.0 L355.0,564.0
 L356.7,564.0 L358.5,564.0 L360.2,564.0 L362.0,564.0 L363.7,564.0 L365.4,564.0 L367.2,564.0 L368.9,564.0
 L370.7,564.0 L372.4,564.0 L374.2,564.0 L375.9,564.0 L377.6,564.0 L379.4,564.0 L381.1,564.0 L382.9,564.0
 L384.6,564.0 L386.3,564.0 L388.1,564.0 L389.8,564.0 L391.6,564.0 L393.3,564.0 L395.0,564.0 L396.8,564.0
 L398.5,564.0 L400.3,564.0 L402.0,564.0 L403.7,564.0 L405.5,564.0 L407.2,564.0 L409.0,564.0 L410.7,564.0
 L412.4,564.0 L414.2,564.0 L415.9,564.0 L417.7,564.0 L419.4,564.0 L421.1,564.0 L422.9,564.0 L424.6,564.0
 L426.4,564.0 L428.1,564.0 L429.8,564.0 L431.6,564.0 L433.3,564.0 L435.1,564.0 L436.8,564.0 L438.6,564.0
 L440.3,564.0 L442.0,564.0 L443.8,564.0 L445.5,564.0 L447.3,564.0 L449.0,564.0 L450.7,564.0 L452.5,564.0
 L454.2,564.0 L456.0,564.0 L457.7,564.0 L459.4,564.0 L461.2,564.0 L462.9,564.0 L464.7,564.0 L466.4,564.0
 L468.1,564.0 L469.9,564.0 L471.6,564.0 L473.4,564.0 L475.1,564.0 L476.8,564.0 L478.6,564.0 L480.3,564.0
 L482.1,564.0 L483.8,564.0 

In [ ]:
var gp = PlotTableEnrgy[4,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 50 
 
 
 
 
 100 
 
 
 
 
 150 
 
 
 
 
 200 
 
 
 
 
 250 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergy-3OrdInit 
 
 
 
 
 totalEnergy-3OrdInit 
 
 
 
 
 
 totalEnergy 
 
 
 totalEnergy 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M455.3,81.1 L508.7,81.1 M53.9,556.4 L55.6,556.4 L57.4,556.4 L59.1,556.4 L60.9,556.4 L62.6,556.4
 L64.3,556.4 L66.1,556.4 L67.8,556.4 L69.6,556.4 L71.3,556.4 L73.0,556.5 L74.8,556.5 L76.5,556.5
 L78.3,556.5 L80.0,556.5 L81.7,556.5 L83.5,556.5 L85.2,556.5 L87.0,556.5 L88.7,556.5 L90.5,556.5
 L92.2,556.5 L93.9,556.5 L95.7,556.5 L97.4,556.6 L99.2,556.6 L100.9,556.6 L102.6,556.6 L104.4,556.6
 L106.1,556.6 L107.9,556.6 L109.6,556.6 L111.3,556.6 L113.1,556.7 L114.8,556.7 L116.6,556.7 L118.3,556.7
 L120.0,556.7 L121.8,556.7 L123.5,556.7 L125.3,556.8 L127.0,556.8 L128.7,556.8 L130.5,556.8 L132.2,556.8
 L134.0,556.8 L135.7,556.8 L137.4,556.8 L139.2,556.9 L140.9,556.9 L142.7,556.9 L144.4,556.9 L146.1,556.9
 L147.9,556.9 L149.6,556.9 L151.4,557.0 L153.1,557.0 L154.8,557.0 L156.6,557.0 L158.3,557.0 L160.1,557.0
 L161.8,557.0 L163.6,557.0 L165.3,557.0 L167.0,557.1 L168.8,557.1 L170.5,557.1 L172.3,557.1 L174.0,557.1
 L175.7,557.1 L177.5,557.1 L179.2,557.1 L181.0,557.1 L182.7,557.1 L184.4,557.1 L186.2,557.1 L187.9,557.1
 L189.7,557.1 L191.4,557.1 L193.1,557.1 L194.9,557.1 L196.6,557.1 L198.4,557.1 L200.1,557.1 L201.8,557.2
 L203.6,557.2 L205.3,557.2 L207.1,557.2 L208.8,557.2 L210.5,557.2 L212.3,557.2 L214.0,557.2 L215.8,557.2
 L217.5,557.2 L219.2,557.2 L221.0,557.2 L222.7,557.2 L224.5,557.2 L226.2,557.2 L228.0,557.2 L229.7,557.2
 L231.4,557.2 L233.2,557.2 L234.9,557.2 L236.7,557.2 L238.4,557.2 L240.1,557.2 L241.9,557.2 L243.6,557.2
 L245.4,557.2 L247.1,557.2 L248.8,557.2 L250.6,557.2 L252.3,557.2 L254.1,557.2 L255.8,557.2 L257.5,557.2
 L259.3,557.2 L261.0,557.2 L262.8,557.2 L264.5,557.2 L266.2,557.2 L268.0,557.2 L269.7,557.2 L271.5,557.2
 L273.2,557.2 L274.9,557.2 L276.7,557.2 L278.4,557.2 L280.2,557.3 L281.9,557.3 L283.6,557.3 L285.4,557.3
 L287.1,557.3 L288.9,557.3 L290.6,557.3 L292.3,557.3 L294.1,557.3 L295.8,557.3 L297.6,557.3 L299.3,557.3
 L301.1,557.3 L302.8,557.3 L304.5,557.3 L306.3,557.3 L308.0,557.3 L309.8,557.3 L311.5,557.3 L313.2,557.3
 L315.0,557.3 L316.7,557.3 L318.5,557.3 L320.2,557.3 L321.9,557.3 L323.7,557.3 L325.4,557.3 L327.2,557.3
 L328.9,557.3 L330.6,557.3 L332.4,557.3 L334.1,557.3 L335.9,557.3 L337.6,557.3 L339.3,557.3 L341.1,557.3
 L342.8,557.3 L344.6,557.3 L346.3,557.3 L348.0,557.3 L349.8,557.3 L351.5,557.3 L353.3,557.3 L355.0,557.3
 L356.7,557.3 L358.5,557.3 L360.2,557.3 L362.0,557.3 L363.7,557.3 L365.4,557.3 L367.2,557.3 L368.9,557.3
 L370.7,557.3 L372.4,557.3 L374.2,557.3 L375.9,557.3 L377.6,557.3 L379.4,557.3 L381.1,557.3 L382.9,557.3
 L384.6,557.3 L386.3,557.3 L388.1,557.3 L389.8,557.3 L391.6,557.3 L393.3,557.3 L395.0,557.3 L396.8,557.3
 L398.5,557.3 L400.3,557.3 L402.0,557.3 L403.7,557.3 L405.5,557.3 L407.2,557.3 L409.0,557.3 L410.7,557.3
 L412.4,557.3 L414.2,557.3 L415.9,557.3 L417.7,557.3 L419.4,557.3 L421.1,557.3 L422.9,557.3 L424.6,557.3
 L426.4,557.3 L428.1,557.3 L429.8,557.3 L431.6,557.3 L433.3,557.3 L435.1,557.3 L436.8,557.3 L438.6,557.3
 L440.3,557.3 L442.0,557.3 L443.8,557.3 L445.5,557.3 L447.3,557.3 L449.0,557.3 L450.7,557.3 L452.5,557.3
 L454.2,557.3 L456.0,557.3 L457.7,557.3 L459.4,557.3 L461.2,557.3 L462.9,557.3 L464.7,557.3 L466.4,557.3
 L468.1,557.3 L469.9,557.3 L471.6,557.3 L473.4,557.3 L475.1,557.3 L476.8,557.3 L478.6,557.3 L480.3,557.3
 L482.1,557.3 L483.8,557.3 L485.5,557.3 L487.3,557.3 L489.0,557.3 L490.8,557.3 L492.5,557.3 L494.2,55

## Case 6 (m4_Oh01_eta01)

In [ ]:
var gp = PlotTableEnrgy[5,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.005 
 
 
 
 
 0 
 
 
 
 
 0.005 
 
 
 
 
 0.01 
 
 
 
 
 0.015 
 
 
 
 
 0.02 
 
 
 
 
 0.025 
 
 
 
 
 0.03 
 
 
 
 
 0.035 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergy-3OrdInit 
 
 
 
 
 surfaceEnergy-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M433.1,57.1 L486.5,57.1 M70.5,88.1 L72.2,87.8 L73.9,88.8 L75.6,91.1 L77.3,94.5 L79.0,99.0
 L80.7,104.4 L82.4,111.0 L84.1,118.5 L85.8,126.9 L87.5,136.1 L89.2,146.0 L90.9,156.7 L92.6,168.0
 L94.3,179.7 L96.0,192.1 L97.7,204.8 L99.4,218.0 L101.1,231.4 L102.8,245.0 L104.5,258.7 L106.2,272.5
 L107.9,286.3 L109.6,300.1 L111.3,313.8 L113.0,327.2 L114.7,340.5 L116.4,353.2 L118.1,365.9 L119.8,378.2
 L121.5,390.0 L123.2,401.3 L124.9,412.1 L126.6,422.3 L128.3,431.9 L130.0,440.9 L131.7,449.3 L133.4,457.0
 L135.1,464.0 L136.8,470.4 L138.5,476.0 L140.2,481.0 L141.9,485.3 L143.6,488.9 L145.3,491.8 L147.0,494.1
 L148.7,495.8 L150.4,496.8 L152.1,497.2 L153.8,497.1 L155.5,496.4 L157.1,495.3 L158.8,493.4 L160.5,491.5
 L162.2,489.1 L163.9,486.1 L165.6,483.1 L167.3,479.7 L169.0,476.1 L170.7,472.2 L172.4,468.2 L174.1,464.1
 L175.8,459.9 L177.5,455.7 L179.2,451.4 L180.9,447.2 L182.6,442.7 L184.3,438.9 L186.0,435.0 L187.7,431.3
 L189.4,427.7 L191.1,424.4 L192.8,421.3 L194.5,418.4 L196.2,415.8 L197.9,413.5 L199.6,411.5 L201.3,409.7
 L203.0,408.3 L204.7,407.2 L206.4,406.4 L208.1,405.9 L209.8,405.8 L211.5,405.9 L213.2,406.3 L214.9,407.1
 L216.6,408.1 L218.3,409.4 L220.0,410.9 L221.7,412.7 L223.4,414.7 L225.1,416.9 L226.8,419.3 L228.5,421.9
 L230.2,424.7 L231.9,427.5 L233.6,430.5 L235.3,433.6 L237.0,436.8 L238.7,440.0 L240.4,443.3 L242.1,446.5
 L243.8,449.6 L245.5,453.0 L247.2,456.3 L248.9,459.5 L250.6,462.6 L252.3,465.6 L254.0,468.6 L255.7,471.4
 L257.4,474.2 L259.1,476.8 L260.8,479.3 L262.5,481.6 L264.2,483.8 L265.9,485.8 L267.6,487.7 L269.3,489.4
 L271.0,491.0 L272.7,492.3 L274.4,493.5 L276.1,494.6 L277.8,495.5 L279.5,496.2 L281.2,496.8 L282.9,497.3
 L284.6,497.6 L286.3,497.8 L288.0,497.8 L289.7,497.7 L291.4,497.5 L293.1,497.2 L294.8,496.8 L296.5,496.3
 L298.2,495.7 L299.9,495.1 L301.6,494.4 L303.3,493.6 L305.0,492.8 L306.7,491.9 L308.4,491.0 L310.1,490.1
 L311.8,489.2 L313.5,488.3 L315.2,487.3 L316.9,486.4 L318.6,485.5 L320.3,484.7 L322.0,483.8 L323.7,483.0
 L325.4,482.2 L327.0,481.5 L328.7,480.8 L330.4,480.2 L332.1,479.7 L333.8,479.2 L335.5,478.7 L337.2,478.3
 L338.9,478.0 L340.6,477.8 L342.3,477.6 L344.0,477.5 L345.7,477.5 L347.4,477.5 L349.1,477.6 L350.8,477.7
 L352.5,477.9 L354.2,478.2 L355.9,478.5 L357.6,478.9 L359.3,479.3 L361.0,479.8 L362.7,480.3 L364.4,480.9
 L366.1,481.5 L367.8,482.1 L369.5,482.8 L371.2,483.5 L372.9,484.1 L374.6,484.9 L376.3,485.6 L378.0,486.3
 L379.7,487.1 L381.4,487.8 L383.1,488.5 L384.8,489.2 L386.5,489.9 L388.2,490.6 L389.9,491.3 L391.6,492.0
 L393.3,492.6 L395.0,493.2 L396.7,493.8 L398.4,494.4 L400.1,494.9 L401.8,495.4 L403.5,495.8 L405.2,496.3
 L406.9,496.6 L408.6,497.0 L410.3,497.3 L412.0,497.6 L413.7,497.8 L415.4,498.0 L417.1,498.2 L418.8,498.3
 L420.5,498.4 L422.2,498.5 L423.9,498.5 L425.6,498.5 L427.3,498.5 L429.0,498.5 L430.7,498.4 L432.4,498.3
 L434.1,498.2 L435.8,498.0 L437.5,497.9 L439.2,497.7 L440.9,497.5 L442.6,497.4 L444.3,497.2 L446.0,496.9
 L447.7,496.7 L449.4,496.5 L451.1,496.3 L452.8,496.1 L454.5,495.9 L456.2,495.7 L457.9,495.5 L459.6,495.3
 L461.3,495.1 L463.0,494.9 L464.7,494.8 L466.4,494.5 L468.1,494.4 L469.8,494.3 L471.5,494.2 L473.2,494.1
 L474.9,494.1 L476.6,494.0 L478.3,494.0 L480.0,493.9 L481.7,493.9 L483.4,493.9 L485.1,494.0 L486.8,494.0
 L488.5,494.0 L490.2,

In [ ]:
var gp = PlotTableEnrgy[5,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.14 
 
 
 
 
 3.145 
 
 
 
 
 3.15 
 
 
 
 
 3.155 
 
 
 
 
 3.16 
 
 
 
 
 3.165 
 
 
 
 
 3.17 
 
 
 
 
 3.175 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergy-3OrdInit 
 
 
 
 
 totalEnergy-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M455.3,57.1 L508.7,57.1 M70.5,50.5 L72.2,51.5 L73.9,52.7 L75.6,54.0 L77.3,55.5 L79.0,57.1
 L80.7,59.0 L82.4,61.1 L84.1,63.6 L85.8,66.4 L87.5,69.5 L89.2,72.9 L90.9,76.6 L92.6,80.7
 L94.3,85.0 L96.0,89.8 L97.7,95.0 L99.4,100.4 L101.1,106.2 L102.8,112.4 L104.5,118.8 L106.2,125.5
 L107.9,132.6 L109.6,139.9 L111.3,147.4 L113.0,155.3 L114.7,163.3 L116.4,171.3 L118.1,179.8 L119.8,188.4
 L121.5,197.1 L123.2,205.9 L124.9,214.8 L126.6,223.7 L128.3,232.7 L130.0,241.6 L131.7,250.5 L133.4,259.4
 L135.1,268.1 L136.8,276.8 L138.5,285.3 L140.2,293.7 L141.9,301.9 L143.6,309.9 L145.3,317.7 L147.0,325.2
 L148.7,332.5 L150.4,339.6 L152.1,346.4 L153.8,352.9 L155.5,359.1 L157.1,365.0 L158.8,370.4 L160.5,375.8
 L162.2,380.9 L163.9,385.4 L165.6,389.9 L167.3,394.0 L169.0,397.8 L170.7,401.3 L172.4,404.5 L174.1,407.5
 L175.8,410.1 L177.5,412.5 L179.2,414.7 L180.9,416.6 L182.6,417.9 L184.3,419.6 L186.0,421.0 L187.7,422.1
 L189.4,423.2 L191.1,424.0 L192.8,424.8 L194.5,425.4 L196.2,425.9 L197.9,426.4 L199.6,426.8 L201.3,427.1
 L203.0,427.4 L204.7,427.7 L206.4,428.0 L208.1,428.3 L209.8,428.6 L211.5,428.9 L213.2,429.3 L214.9,429.7
 L216.6,430.2 L218.3,430.8 L220.0,431.4 L221.7,432.0 L223.4,432.8 L225.1,433.6 L226.8,434.6 L228.5,435.6
 L230.2,436.7 L231.9,437.8 L233.6,439.1 L235.3,440.5 L237.0,441.9 L238.7,443.4 L240.4,445.0 L242.1,446.7
 L243.8,448.2 L245.5,450.1 L247.2,452.0 L248.9,453.9 L250.6,455.8 L252.3,457.8 L254.0,459.9 L255.7,461.9
 L257.4,464.0 L259.1,466.0 L260.8,468.1 L262.5,470.2 L264.2,472.3 L265.9,474.4 L267.6,476.4 L269.3,478.4
 L271.0,480.4 L272.7,482.2 L274.4,484.2 L276.1,486.0 L277.8,487.8 L279.5,489.6 L281.2,491.3 L282.9,493.0
 L284.6,494.6 L286.3,496.1 L288.0,497.6 L289.7,499.0 L291.4,500.3 L293.1,501.6 L294.8,502.8 L296.5,503.9
 L298.2,505.0 L299.9,506.0 L301.6,506.9 L303.3,507.7 L305.0,508.5 L306.7,509.3 L308.4,509.9 L310.1,510.6
 L311.8,511.1 L313.5,511.6 L315.2,512.1 L316.9,512.5 L318.6,512.8 L320.3,513.2 L322.0,513.4 L323.7,513.7
 L325.4,513.9 L327.0,514.1 L328.7,514.2 L330.4,514.4 L332.1,514.5 L333.8,514.6 L335.5,514.7 L337.2,514.7
 L338.9,514.8 L340.6,514.9 L342.3,514.9 L344.0,515.0 L345.7,515.1 L347.4,515.1 L349.1,515.2 L350.8,515.3
 L352.5,515.4 L354.2,515.5 L355.9,515.6 L357.6,515.8 L359.3,515.9 L361.0,516.1 L362.7,516.3 L364.4,516.5
 L366.1,516.7 L367.8,516.9 L369.5,517.2 L371.2,517.5 L372.9,517.8 L374.6,518.1 L376.3,518.4 L378.0,518.8
 L379.7,519.1 L381.4,519.5 L383.1,519.9 L384.8,520.3 L386.5,520.7 L388.2,521.2 L389.9,521.6 L391.6,522.0
 L393.3,522.5 L395.0,522.9 L396.7,523.4 L398.4,523.9 L400.1,524.3 L401.8,524.8 L403.5,525.3 L405.2,525.7
 L406.9,526.2 L408.6,526.6 L410.3,527.0 L412.0,527.5 L413.7,527.9 L415.4,528.3 L417.1,528.7 L418.8,529.1
 L420.5,529.5 L422.2,529.8 L423.9,530.2 L425.6,530.5 L427.3,530.8 L429.0,531.2 L430.7,531.4 L432.4,531.7
 L434.1,532.0 L435.8,532.2 L437.5,532.5 L439.2,532.7 L440.9,532.9 L442.6,533.0 L444.3,533.2 L446.0,533.4
 L447.7,533.5 L449.4,533.6 L451.1,533.8 L452.8,533.9 L454.5,534.0 L456.2,534.0 L457.9,534.1 L459.6,534.2
 L461.3,534.2 L463.0,534.3 L464.7,534.3 L466.4,534.2 L468.1,534.3 L469.8,534.4 L471.5,534.5 L473.2,534.5
 L474.9,534.5 L476.6,534.6 L478.3,534.6 L480.0,534.6 L481.7,534.6 L483.4,534.6 L485.1,534.7 L486.8,534.7
 L488.5,534.7 L490.2,534.8 L491.9,534.8 L493.6,534.8 L495.2,

## Case 7 (m4_Oh056_eta005)

In [ ]:
var gp = PlotTableEnrgy[6,0].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -0.01 
 
 
 
 
 0 
 
 
 
 
 0.01 
 
 
 
 
 0.02 
 
 
 
 
 0.03 
 
 
 
 
 0.04 
 
 
 
 
 0.05 
 
 
 
 
 0.06 
 
 
 
 
 0.07 
 
 
 
 
 0.08 
 
 
 
 
 0.09 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 surfaceEnergyJ686-3OrdInit 
 
 
 
 
 surfaceEnergyJ686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M388.7,57.1 L442.1,57.1 M62.2,469.9 L63.9,469.9 L65.6,470.0 L67.4,470.1 L69.1,470.3 L70.8,470.6
 L72.5,471.0 L74.2,471.3 L76.0,471.8 L77.7,472.3 L79.4,472.8 L81.1,473.4 L82.8,474.0 L84.6,474.7
 L86.3,475.4 L88.0,476.1 L89.7,476.8 L91.4,477.6 L93.2,478.4 L94.9,479.1 L96.6,479.9 L98.3,480.7
 L100.0,481.5 L101.8,482.3 L103.5,483.2 L105.2,484.0 L106.9,484.8 L108.6,485.5 L110.4,486.3 L112.1,487.1
 L113.8,487.8 L115.5,488.6 L117.2,489.3 L119.0,490.0 L120.7,490.7 L122.4,491.4 L124.1,492.1 L125.8,492.7
 L127.6,493.3 L129.3,493.9 L131.0,494.5 L132.7,495.1 L134.4,495.7 L136.1,496.2 L137.9,496.7 L139.6,497.2
 L141.3,497.7 L143.0,498.1 L144.7,498.6 L146.5,499.0 L148.2,499.4 L149.9,499.8 L151.6,500.2 L153.3,500.5
 L155.1,500.9 L156.8,501.2 L158.5,501.5 L160.2,501.8 L161.9,502.1 L163.7,502.4 L165.4,502.6 L167.1,502.9
 L168.8,503.1 L170.5,503.3 L172.3,503.6 L174.0,503.8 L175.7,504.0 L177.4,504.1 L179.1,504.3 L180.9,504.5
 L182.6,504.7 L184.3,504.8 L186.0,505.0 L187.7,505.1 L189.5,505.2 L191.2,505.4 L192.9,505.5 L194.6,505.6
 L196.3,505.7 L198.1,505.8 L199.8,505.9 L201.5,506.0 L203.2,506.1 L204.9,506.2 L206.7,506.3 L208.4,506.3
 L210.1,506.4 L211.8,506.5 L213.5,506.6 L215.3,506.6 L217.0,506.7 L218.7,506.8 L220.4,506.8 L222.1,506.9
 L223.9,507.0 L225.6,507.0 L227.3,507.1 L229.0,507.1 L230.7,507.2 L232.5,507.2 L234.2,507.3 L235.9,507.3
 L237.6,507.4 L239.3,507.4 L241.1,507.4 L242.8,507.5 L244.5,507.5 L246.2,507.6 L247.9,507.6 L249.7,507.7
 L251.4,507.7 L253.1,507.8 L254.8,507.8 L256.5,507.8 L258.3,507.9 L260.0,507.9 L261.7,508.0 L263.4,508.0
 L265.1,508.0 L266.9,508.1 L268.6,508.1 L270.3,508.2 L272.0,508.2 L273.7,508.2 L275.4,508.3 L277.2,508.3
 L278.9,508.4 L280.6,508.4 L282.3,508.4 L284.0,508.5 L285.8,508.5 L287.5,508.5 L289.2,508.6 L290.9,508.6
 L292.6,508.7 L294.4,508.7 L296.1,508.7 L297.8,508.8 L299.5,508.8 L301.2,508.8 L303.0,508.9 L304.7,508.9
 L306.4,509.0 L308.1,509.0 L309.8,509.0 L311.6,509.1 L313.3,509.1 L315.0,509.1 L316.7,509.2 L318.4,509.2
 L320.2,509.2 L321.9,509.3 L323.6,509.3 L325.3,509.3 L327.0,509.4 L328.8,509.4 L330.5,509.4 L332.2,509.5
 L333.9,509.5 L335.6,509.5 L337.4,509.6 L339.1,509.6 L340.8,509.6 L342.5,509.7 L344.2,509.7 L346.0,509.7
 L347.7,509.8 L349.4,509.8 L351.1,509.8 L352.8,509.9 L354.6,509.9 L356.3,509.9 L358.0,509.9 L359.7,510.0
 L361.4,510.0 L363.2,510.0 L364.9,510.1 L366.6,510.1 L368.3,510.1 L370.0,510.1 L371.8,510.2 L373.5,510.2
 L375.2,510.2 L376.9,510.2 L378.6,510.3 L380.4,510.3 L382.1,510.3 L383.8,510.3 L385.5,510.4 L387.2,510.4
 L389.0,510.4 L390.7,510.4 L392.4,510.5 L394.1,510.5 L395.8,510.5 L397.6,510.5 L399.3,510.5 L401.0,510.6
 L402.7,510.6 L404.4,510.6 L406.2,510.6 L407.9,510.6 L409.6,510.7 L411.3,510.7 L413.0,510.7 L414.7,510.7
 L416.5,510.7 L418.2,510.8 L419.9,510.8 L421.6,510.8 L423.3,510.8 L425.1,510.8 L426.8,510.8 L428.5,510.9
 L430.2,510.9 L431.9,510.9 L433.7,510.9 L435.4,510.9 L437.1,510.9 L438.8,510.9 L440.5,511.0 L442.3,511.0
 L444.0,511.0 L445.7,511.0 L447.4,511.0 L449.1,511.0 L450.9,511.0 L452.6,511.1 L454.3,511.1 L456.0,511.1
 L457.7,511.1 L459.5,511.1 L461.2,511.1 L462.9,511.1 L464.6,511.1 L466.3,511.1 L468.1,511.1 L469.8,511.2
 L471.5,511.2 L473.2,511.2 L474.9,511.2 L476.7,511.2 L478.4,511.2 L480.

In [ ]:
var gp = PlotTableEnrgy[6,1].ToGnuplot();
gp.PlotSVG()

Using gnuplot: C:\Users\smuda\AppData\Local\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16" Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 3.14 
 
 
 
 
 3.15 
 
 
 
 
 3.16 
 
 
 
 
 3.17 
 
 
 
 
 3.18 
 
 
 
 
 3.19 
 
 
 
 
 3.2 
 
 
 
 
 3.21 
 
 
 
 
 3.22 
 
 
 
 
 3.23 
 
 
 
 
 3.24 
 
 
 
 
 0 
 
 
 
 
 0.5 
 
 
 
 
 1 
 
 
 
 
 1.5 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 totalEnergyJ686-3OrdInit 
 
 
 
 
 totalEnergyJ686-3OrdInit 
 
 
 
	<path stroke='rgb( 0, 0, 255)' stroke-dasharray='7.5,12.0' d='M410.9,57.1 L464.3,57.1 M62.2,79.2 L63.9,145.1 L65.6,201.8 L67.4,250.0 L69.1,290.5 L70.8,324.3
 L72.5,352.7 L74.2,376.5 L76.0,396.6 L77.7,413.5 L79.4,427.8 L81.1,440.0 L82.8,450.4 L84.6,459.2
 L86.3,466.8 L88.0,473.4 L89.7,479.1 L91.4,484.0 L93.2,488.4 L94.9,492.2 L96.6,495.6 L98.3,498.6
 L100.0,501.4 L101.8,503.9 L103.5,506.2 L105.2,508.3 L106.9,510.2 L108.6,512.0 L110.4,513.7 L112.1,515.3
 L113.8,516.9 L115.5,518.3 L117.2,519.7 L119.0,521.0 L120.7,522.3 L122.4,523.5 L124.1,524.7 L125.8,525.8
 L127.6,526.9 L129.3,527.9 L131.0,528.9 L132.7,529.9 L134.4,530.9 L136.1,531.8 L137.9,532.7 L139.6,533.5
 L141.3,534.3 L143.0,535.1 L144.7,535.9 L146.5,536.6 L148.2,537.3 L149.9,538.0 L151.6,538.7 L153.3,539.3
 L155.1,539.9 L156.8,540.5 L158.5,541.1 L160.2,541.6 L161.9,542.1 L163.7,542.6 L165.4,543.1 L167.1,543.5
 L168.8,544.0 L170.5,544.4 L172.3,544.8 L174.0,545.1 L175.7,545.5 L177.4,545.9 L179.1,546.2 L180.9,546.5
 L182.6,546.8 L184.3,547.1 L186.0,547.3 L187.7,547.6 L189.5,547.8 L191.2,548.1 L192.9,548.3 L194.6,548.5
 L196.3,548.7 L198.1,548.9 L199.8,549.1 L201.5,549.2 L203.2,549.4 L204.9,549.6 L206.7,549.7 L208.4,549.8
 L210.1,550.0 L211.8,550.1 L213.5,550.2 L215.3,550.3 L217.0,550.4 L218.7,550.5 L220.4,550.6 L222.1,550.7
 L223.9,550.8 L225.6,550.9 L227.3,551.0 L229.0,551.0 L230.7,551.1 L232.5,551.2 L234.2,551.2 L235.9,551.3
 L237.6,551.4 L239.3,551.4 L241.1,551.4 L242.8,551.5 L244.5,551.5 L246.2,551.6 L247.9,551.6 L249.7,551.7
 L251.4,551.7 L253.1,551.8 L254.8,551.8 L256.5,551.9 L258.3,551.9 L260.0,551.9 L261.7,552.0 L263.4,552.0
 L265.1,552.0 L266.9,552.1 L268.6,552.1 L270.3,552.1 L272.0,552.2 L273.7,552.2 L275.4,552.2 L277.2,552.3
 L278.9,552.3 L280.6,552.3 L282.3,552.3 L284.0,552.4 L285.8,552.4 L287.5,552.4 L289.2,552.5 L290.9,552.5
 L292.6,552.5 L294.4,552.5 L296.1,552.6 L297.8,552.6 L299.5,552.6 L301.2,552.7 L303.0,552.7 L304.7,552.7
 L306.4,552.7 L308.1,552.8 L309.8,552.8 L311.6,552.8 L313.3,552.9 L315.0,552.9 L316.7,552.9 L318.4,552.9
 L320.2,553.0 L321.9,553.0 L323.6,553.0 L325.3,553.1 L327.0,553.1 L328.8,553.1 L330.5,553.2 L332.2,553.2
 L333.9,553.2 L335.6,553.2 L337.4,553.3 L339.1,553.3 L340.8,553.3 L342.5,553.4 L344.2,553.4 L346.0,553.4
 L347.7,553.4 L349.4,553.5 L351.1,553.5 L352.8,553.5 L354.6,553.6 L356.3,553.6 L358.0,553.6 L359.7,553.6
 L361.4,553.7 L363.2,553.7 L364.9,553.7 L366.6,553.8 L368.3,553.8 L370.0,553.8 L371.8,553.9 L373.5,553.9
 L375.2,553.9 L376.9,553.9 L378.6,554.0 L380.4,554.0 L382.1,554.0 L383.8,554.0 L385.5,554.1 L387.2,554.1
 L389.0,554.1 L390.7,554.2 L392.4,554.2 L394.1,554.2 L395.8,554.2 L397.6,554.3 L399.3,554.3 L401.0,554.3
 L402.7,554.3 L404.4,554.4 L406.2,554.4 L407.9,554.4 L409.6,554.4 L411.3,554.5 L413.0,554.5 L414.7,554.5
 L416.5,554.5 L418.2,554.6 L419.9,554.6 L421.6,554.6 L423.3,554.6 L425.1,554.7 L426.8,554.7 L428.5,554.7
 L430.2,554.7 L431.9,554.7 L433.7,554.8 L435.4,554.8 L437.1,554.8 L438.8,554.8 L440.5,554.8 L442.3,554.9
 L444.0,554.9 L445.7,554.9 L447.4,554.9 L449.1,554.9 L450.9,555.0 L452.6,555.0 L454.3,555.0 L456.0,555.0
 L457.7,555.0 L459.5,555.0 L461.2,555.1 L462.9,555.1 L464.6,555.1 L466.3,555.1 L468.1,555.1 L469.8,555.1
 L471.5,555.2 L473.2,555.2 L474.9,555.2 L476.7,555.2 L478.4,555.2 L480.1,55